In [ ]:
from ase.io import read, Trajectory, write
from ase.constraints import FixAtoms
import numpy as np
import os
import time
from datetime import datetime
from sklearn.linear_model import LinearRegression
from matlantis_features.features.md import ASEMDSystem, LangevinIntegrator, MDFeature, MDExtensionBase, NPTIntegrator
from matlantis_features.features.md.md_extensions import DeformScheduler
from pfp_api_client.pfp.estimator import Estimator, EstimatorCalcMode
from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
from matlantis_features.utils.calculators import get_calculator, pfp_estimator_fn
from matlantis_features.utils.atoms_util import convert_atoms_to_upper
from matlantis_features.atoms import MatlantisAtoms
import numpy as np
import pathlib
from ase import units
from ase.build import add_vacuum
import pandas as pd  # CSVファイル読み込み用

# =============================================
# ファイル・ディレクトリパス設定（一箇所で管理）
# =============================================

# ベースディレクトリ
BASE_DIR = "/home/jovyan/Kaori/MD/LiB_2/"

# ★★★ 入力ディレクトリの変更 ★★★
# 元のコード: INPUT_DIR = os.path.join(BASE_DIR, "optimized_structures")
INPUT_DIR = os.path.join(BASE_DIR, "structure/output/final_interfaces")

# 出力ディレクトリ
OUTPUT_DIR = os.path.join(BASE_DIR, "output")
COMPRESSION_DIR = os.path.join(OUTPUT_DIR, "compression")
HEAT_DIR = os.path.join(OUTPUT_DIR, "heat")
STRESS_STRAIN_DIR = os.path.join(OUTPUT_DIR, "stress_strain")
TENSILE_DIR = os.path.join(OUTPUT_DIR, "tensile")

# ディレクトリ作成
os.makedirs(INPUT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(COMPRESSION_DIR, exist_ok=True)
os.makedirs(HEAT_DIR, exist_ok=True)
os.makedirs(STRESS_STRAIN_DIR, exist_ok=True)
os.makedirs(TENSILE_DIR, exist_ok=True)

# ★★★ CSV読み込み設定の追加 ★★★
CSV_FILE_PATH = "/home/jovyan/Kaori/MD/LiB_2/structure/extracted_data_final_with_filenames.csv"

def get_input_traj_path(com, atom):
    """入力ファイルのパスを取得"""
    return os.path.join(INPUT_DIR, f"{com}_{atom}_interface_optimized.traj")
    
def get_input_file_path(com, atom):
    """入力ファイルのパスを取得"""
    return os.path.join(INPUT_DIR, f"{com}_{atom}_interface_optimized.xyz")

# ★★★ CSVからファイル名を取得する関数を追加 ★★★
def get_input_file_from_csv(original_filename):
    """CSVの original_filename から入力ファイルパスを取得"""
    return os.path.join(INPUT_DIR, original_filename)

def get_compression_file_path(base_name, pressure, comp_temp):
    """圧着シミュレーションファイルのパスを取得"""
    return os.path.join(COMPRESSION_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_compression.traj")

def get_equil_file_path(base_name, pressure, comp_temp):
    """平衡化ファイルのパスを取得"""
    return os.path.join(HEAT_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_equil.traj")

def get_high_temp_file_path(base_name, pressure, comp_temp, high_temp):
    """高温シミュレーションファイルのパスを取得"""
    return os.path.join(HEAT_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K.traj")

def get_tensile_file_path(base_name, pressure, comp_temp, high_temp, tensile_temperature):
    """引張シミュレーションファイルのパスを取得"""
    return os.path.join(TENSILE_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K_tensile{tensile_temperature:.0f}K_early_stop.traj")

def get_stress_strain_file_path(base_name, pressure, comp_temp, high_temp):
    """応力-歪みデータファイルのパスを取得"""
    return os.path.join(STRESS_STRAIN_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K_stress_strain.dat")

def get_tensile_analysis_file_path(base_name, pressure, comp_temp, high_temp):
    """引張解析ファイルのパスを取得"""
    return os.path.join(TENSILE_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K_tensile_analysis.dat")

def get_tensile_summary_file_path(base_name, pressure, comp_temp, high_temp):
    """引張結果サマリーファイルのパスを取得"""
    return os.path.join(TENSILE_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K_tensile_summary.txt")

# =============================================
# 早期終了システム設定
# =============================================

# 統一条件設定（公平な比較のため）
UNIFIED_CONDITIONS = {
    "strain_rate": 0.2,              # 1/ps（統一ひずみ速度）
    "vacuum_top": 0.0,               # Å（上部vacuum）
    "temperature": 300.0,             # K（引張温度固定）
    "max_strain": 3.0,                # 最大ひずみ（材料長基準）
    "timestep": 1.0,                  # fs
    "elastic_region": (0.005, 0.03),  # 弾性領域（0.5-3.0%）
    "quality_threshold": 0.9,         # R²値閾値
    "max_steps": 100000,              # 最大ステップ数
    "output_freq": 100,               # 出力頻度
    "fixed_layer_thickness": 15      # 固定層の相対厚さ
}

# 早期終了条件設定
EARLY_STOP_CONDITIONS = {
    "separation_gap": 10.0,            # Å 完全分離判定の隙間
    "max_strain_limit": 5.0,          # 最大ひずみ制限
    "min_stress_threshold": -1.0,     # 最小応力閾値（圧縮転換）
    "separation_wait_steps": 500,     # 分離後の待機ステップ
    "stress_drop_ratio": 0.6          # 破断検出用（記録のみ）
}

# =============================================
# 早期終了システムクラス
# =============================================

class EarlyTerminationException(Exception):
    """早期終了用のカスタム例外"""
    def __init__(self, reason, step, details=None):
        self.reason = reason
        self.step = step
        self.details = details or {}
        super().__init__(f"Early termination at step {step}: {reason}")

class ComprehensiveTensileAnalyzer(MDExtensionBase):
    """
    早期終了機能を無効化した包括的引張解析器
    """
    
    def __init__(self, initial_thickness, structure_info, conditions, source_file, output_file):
        self.initial_thickness = initial_thickness
        self.structure_info = structure_info
        self.conditions = conditions
        self.source_file = source_file
        self.output_file = output_file
        
        # 解析データ
        self.data = []
        self.young_modulus = None
        self.young_r2 = 0.0
        self.young_intercept = 0.0
        self.max_stress = 0.0
        self.max_stress_strain = 0.0
        self.fracture_detected = False
        self.fracture_step = None
        self.complete_separation = False
        self.separation_step = None
        
        # 早期終了関連（無効化）
        self.early_termination_requested = False
        self.termination_reason = None
        self.termination_details = {}
        
        # チェック用カウンタ（記録のみ）
        self.stress_drop_count = 0
        self.separation_wait_count = 0
        
        # ヘッダー書き込み
        with open(self.output_file, 'w') as f:
            f.write(f"# Integrated Tensile Analysis (No Early Termination)\n")
            f.write(f"# Source: {source_file}\n")
            f.write(f"# Unified Strain Rate: {conditions['strain_rate']} 1/ps\n")
            f.write(f"# Detection Conditions (Record Only):\n")
            f.write(f"#   Separation Gap: {EARLY_STOP_CONDITIONS['separation_gap']} A (RECORD)\n")
            f.write(f"#   Max Strain Limit: {EARLY_STOP_CONDITIONS['max_strain_limit']} (RECORD)\n")
            f.write(f"#   Min Stress Threshold: {EARLY_STOP_CONDITIONS['min_stress_threshold']} GPa (RECORD)\n")
            f.write(f"#   Stress Drop Detection: {EARLY_STOP_CONDITIONS['stress_drop_ratio']} (RECORD)\n")
            if UNIFIED_CONDITIONS["vacuum_top"] > 0:
                f.write(f"# Step Material_Strain Stress(GPa) YoungMod(GPa) Status Time(ps) Notes\n")
            else:
                f.write(f"# Step Cell_Strain Stress(GPa) YoungMod(GPa) Status Time(ps) Notes\n")

    def detect_events(self, system, material_strain, stress_zz_GPa, z_coords=None):
        """イベント検出（記録のみ、早期終了はしない）"""
        
        # 1. 完全分離チェック（vacuum有りの場合のみ）
        if UNIFIED_CONDITIONS["vacuum_top"] > 0 and z_coords is not None:
            if system.current_total_step % 100 == 0:
                z_median = np.median(z_coords)
                upper_z = z_coords[z_coords > z_median]
                lower_z = z_coords[z_coords <= z_median]
                
                if len(upper_z) > 0 and len(lower_z) > 0:
                    gap = np.min(upper_z) - np.max(lower_z)
                    if np.max(lower_z) >= 8.0:
                        if gap > EARLY_STOP_CONDITIONS["separation_gap"]:
                            if not self.complete_separation:
                                self.complete_separation = True
                                self.separation_step = system.current_total_step
                                print(f"      🔗 完全分離検出: Step {self.separation_step}, 隙間 {gap:.1f}Å（記録のみ）")
                                
                                self.termination_details.update({
                                    "separation_gap": gap,
                                    "separation_step": self.separation_step
                                })
                            
                            self.separation_wait_count += 1
        
        # 2. 応力急減チェック（破断検出）- 記録のみ
        if len(self.data) > 100 and self.max_stress > 0:
            recent_stresses = [d[2] for d in self.data[-50:]]
            if len(recent_stresses) > 0:
                recent_max_stress = max(recent_stresses)
                stress_ratio = recent_max_stress / self.max_stress
                
                if stress_ratio < EARLY_STOP_CONDITIONS["stress_drop_ratio"]:
                    if not self.fracture_detected:
                        self.fracture_detected = True
                        self.fracture_step = system.current_total_step
                        print(f"      ⚠️  破断検出: Step {self.fracture_step}, 応力比 {stress_ratio:.3f}（記録のみ）")
                        
                        self.termination_details.update({
                            "fracture_stress_ratio": stress_ratio,
                            "fracture_step": self.fracture_step
                        })
        
        # 3. 最大ひずみ制限チェック - 記録のみ
        if material_strain > EARLY_STOP_CONDITIONS["max_strain_limit"]:
            if "max_strain_exceeded" not in self.termination_details:
                print(f"      📊 最大ひずみ制限超過: {material_strain:.3f}（記録のみ）")
                self.termination_details["max_strain_exceeded"] = {
                    "strain": material_strain,
                    "step": system.current_total_step
                }
        
        # 4. 圧縮転換チェック - 記録のみ
        if stress_zz_GPa < EARLY_STOP_CONDITIONS["min_stress_threshold"]:
            if "compression_detected" not in self.termination_details:
                print(f"      ⬇️  圧縮転換検出: {stress_zz_GPa:.3f} GPa（記録のみ）")
                self.termination_details["compression_detected"] = {
                    "stress": stress_zz_GPa,
                    "step": system.current_total_step
                }
    
    def __call__(self, system, integrator):
        # ひずみ計算の分岐（vacuum有無による）
        if UNIFIED_CONDITIONS["vacuum_top"] > 0:
            # vacuum有り：材料長基準
            positions = system.ase_atoms.get_positions()
            z_coords = positions[:, 2]
            current_material_length = np.max(z_coords) - np.min(z_coords)
            material_strain = (current_material_length - self.initial_thickness) / self.initial_thickness
            strain_method = "material_length"
        else:
            # vacuum無し：セル長基準
            current_cell_length = system.ase_atoms.get_cell()[2, 2]
            material_strain = (current_cell_length - self.initial_thickness) / self.initial_thickness
            current_material_length = current_cell_length  # セル長=材料長
            strain_method = "cell_length"
            z_coords = None
        
        # 応力取得
        try:
            stress_tensor = system.ase_atoms.get_stress(voigt=False)
            stress_zz_GPa = stress_tensor[2, 2] * 160.21766208  # eV/Å³ to GPa
        except:
            stress_zz_GPa = 0.0
        
        # セル長
        cell_length = system.ase_atoms.get_cell()[2, 2]
        time_ps = system.current_total_step * self.conditions["timestep"] / 1000
        
        if UNIFIED_CONDITIONS["vacuum_top"] > 0:
            # vacuum有り：材料長も記録
            self.data.append([
                system.current_total_step, material_strain, stress_zz_GPa,
                current_material_length, cell_length, time_ps
            ])
        else:
            # vacuum無し：材料長記録なし（セル長のみ）
            self.data.append([
                system.current_total_step, material_strain, stress_zz_GPa,
                None, cell_length, time_ps
            ])
        
        # 最大応力更新
        if stress_zz_GPa > self.max_stress:
            self.max_stress = stress_zz_GPa
            self.max_stress_strain = material_strain
        
        # ヤング率計算
        status = "Loading"
        current_young = 0.0
        
        if len(self.data) > 50 and material_strain > 0.01 and self.young_modulus is None:
            strains = np.array([d[1] for d in self.data])
            stresses = np.array([d[2] for d in self.data])
            
            elastic_min, elastic_max = self.conditions["elastic_region"]
            elastic_mask = (strains > elastic_min) & (strains < elastic_max) & (stresses > 0)
            
            if np.sum(elastic_mask) > 20:
                elastic_strains = strains[elastic_mask]
                elastic_stresses = stresses[elastic_mask]
                
                X = elastic_strains.reshape(-1, 1)
                y = elastic_stresses
                
                model = LinearRegression().fit(X, y)
                predicted = model.predict(X)
                
                self.young_modulus = model.coef_[0]
                self.young_intercept = model.intercept_
                
                ss_res = np.sum((y - predicted) ** 2)
                ss_tot = np.sum((y - np.mean(y)) ** 2)
                self.young_r2 = 1 - (ss_res / ss_tot) if ss_tot > 0 else 0
                
                print(f"      ✅ ヤング率確定: {self.young_modulus:.1f} GPa (R² = {self.young_r2:.3f})")
        
        if self.young_modulus is not None:
            current_young = self.young_modulus
            if material_strain > self.conditions["elastic_region"][1]:
                status = "Plastic"
        
        # イベント検出（記録のみ、早期終了はしない）
        self.detect_events(system, material_strain, stress_zz_GPa, z_coords)
        
        # 状態更新
        if self.complete_separation:
            status = "Separated"
        elif self.fracture_detected:
            status = "Fractured"
        
        # 早期終了処理は無効化 - 常に計算を継続
        
        # ファイル出力（計算方法も記録）
        notes = f"Method:{strain_method}"
        if self.complete_separation:
            notes += f" Sep:{self.separation_wait_count}"
        if self.fracture_detected:
            notes += f" Frac"
        
        with open(self.output_file, 'a') as f:
            f.write(f"{system.current_total_step:<8} {material_strain:<14.6f} {stress_zz_GPa:<12.6f} ")
            f.write(f"{current_young:<12.1f} {status:<10} {time_ps:<10.3f} {notes}\n")
    
    def get_final_results(self):
        """最終結果を辞書形式で返す"""
        return {
            'young_modulus': self.young_modulus or 0.0,
            'young_r2': self.young_r2,
            'young_intercept': self.young_intercept,
            'max_stress': self.max_stress,
            'max_stress_strain': self.max_stress_strain,
            'fracture_detected': self.fracture_detected,
            'fracture_step': self.fracture_step,
            'complete_separation': self.complete_separation,
            'separation_step': self.separation_step,
            'total_data_points': len(self.data),
            'structure_info': self.structure_info,
            'conditions': self.conditions,
            'early_termination': False,  # 常にFalse
            'termination_reason': None,
            'termination_details': self.termination_details
        }

def run_md_without_early_termination(md_feature, system, extensions):
    """早期終了機能無しのMD実行ラッパー"""
    
    execution_info = {
        'completed_normally': False,
        'early_terminated': False,
        'termination_reason': None,
        'termination_step': None,
        'total_steps_run': 0,
        'execution_time': 0.0,
        'error_occurred': False,
        'error_message': None
    }
    
    start_time = time.time()
    
    try:
        print(f"🚀 MD計算開始（早期終了無効）")
        
        # MD実行（早期終了例外はキャッチしない）
        md_feature(system, extensions=extensions)
        
        # 正常完了
        execution_info['completed_normally'] = True
        execution_info['total_steps_run'] = UNIFIED_CONDITIONS["max_steps"]
        print(f"✅ MD計算正常完了")
        
    except Exception as e:
        # エラー終了のみキャッチ（EarlyTerminationExceptionもここでキャッチされるが、無視する）
        if "EarlyTerminationException" in str(type(e)):
            # 早期終了例外の場合は正常完了として扱う
            execution_info['completed_normally'] = True
            execution_info['total_steps_run'] = getattr(e, 'step', UNIFIED_CONDITIONS["max_steps"])
            print(f"✅ MD計算完了（早期終了イベント検出済み）")
        else:
            # 真のエラー終了
            execution_info['error_occurred'] = True
            execution_info['error_message'] = str(e)
            print(f"❌ MD計算エラー終了: {e}")
        
    finally:
        execution_info['execution_time'] = time.time() - start_time
    
    return execution_info

# =============================================
# シミュレーションパラメータ設定
# =============================================

# ★★★ 物質リストをCSVから読み込みに変更 ★★★
# パラメータの設定 - カスタマイズ可能
compression_temperatures = [300]  # 圧着シミュレーション温度（K）
compression_pressures = [0.001,0.005,0.01]  # 圧着圧力（GPa）
high_temperatures = [300]  # 高温シミュレーション温度（K）
tensile_temperature = 300.0  # 引張シミュレーション温度（K）

# =============================================
# 既存の関数群（変更なし）
# =============================================

def check_md_completed(traj_path, expected_steps, traj_freq):
    """シミュレーションが既に完了しているかチェックする関数"""
    try:
        if not os.path.exists(traj_path):
            return False
        
        traj = Trajectory(traj_path, 'r')
        num_frames = len(traj)
        expected_frames = expected_steps // traj_freq
        
        return num_frames >= expected_frames * 0.1
    except Exception:
        return False

class PrintCellShape(MDExtensionBase):
    def __init__(self, cell_log=None):
        self.cell_log = cell_log
        
    def __call__(self, system, integrator) -> None:
        cell_par = system.ase_atoms.cell.cellpar()
        istep = system.current_total_step
        if istep % 1000 == 0:
            print(f"Dyn step {istep:4d} a {cell_par[0]:3.2f} b {cell_par[1]:3.2f} c {cell_par[2]:3.2f} alpha {cell_par[3]:3.2f} beta {cell_par[4]:3.2f} gamma {cell_par[5]:3.2f} ")
        if self.cell_log is not None:
            self.cell_log.append(cell_par)

def rotate_atoms_to_lower(atoms):
    """原子系のZ座標を確認し、全原子を下端（Z=0付近）に移動させる"""
    ase_atoms = atoms.ase_atoms if hasattr(atoms, 'ase_atoms') else atoms
    
    positions = ase_atoms.get_positions()
    z_min = np.min(positions[:, 2])
    z_max = np.max(positions[:, 2])
    
    shift_z = -z_min
    translation = np.array([0, 0, shift_z])
    ase_atoms.translate(translation)
    z_min2 = np.min(positions[:, 2])
    z_max2 = np.max(positions[:, 2])
    
    # 原子の厚み（Z方向の範囲）を計算
    thickness = z_max2 - z_min2
    new_cell =  ase_atoms.get_cell()
    new_cell[2, 2] =  z_max+shift_z+0.5
    ase_atoms.set_cell(new_cell, scale_atoms=False)
    ase_atoms.wrap()
    print(f"原子をZ方向に {shift_z:.2f}Å 移動しました")
    print(f"Z座標範囲: {z_min:.2f} → {z_min+shift_z:.2f}, {z_max:.2f} → {z_max+shift_z:.2f}")
    
    return ase_atoms

def set_fixed_atoms_at_bottom(atoms, z_threshold=UNIFIED_CONDITIONS["fixed_layer_thickness"]):
    """Z座標が閾値以下の原子を固定する"""
    positions = atoms.get_positions()
    indices = [atom.index for atom in atoms if atom.position[2] <= z_threshold]
    
    print(f"固定する原子数: {len(indices)} / {len(atoms)}")
    
    constraint = FixAtoms(indices=indices)
    atoms.set_constraint(constraint)
    
    return atoms

class StressStrainRecorder(MDExtensionBase):
    """応力-歪み記録クラス"""
    def __init__(self, initial_length, output_file):
        self.initial_length = initial_length
        self.output_file = output_file
        self.data = []
        
    def __call__(self, system, integrator):
        current_cell = system.ase_atoms.get_cell()
        current_length = current_cell[2, 2] - 15
        strain = (current_length - self.initial_length) / self.initial_length
        
        try:
            stress = system.ase_atoms.get_stress()
            stress_zz = stress[2]
        except:
            stress_zz = 0.0
        
        self.data.append([system.current_total_step, strain, stress_zz])
        
        if len(self.data) % 10 == 0:
            np.savetxt(self.output_file, self.data, 
                      header="Step Strain Stress_ZZ", fmt="%.6f")

def run_md_with_early_termination(md_feature, system, extensions):
    """早期終了機能付きのMD実行ラッパー"""
    
    execution_info = {
        'completed_normally': False,
        'early_terminated': False,
        'termination_reason': None,
        'termination_step': None,
        'total_steps_run': 0,
        'execution_time': 0.0,
        'error_occurred': False,
        'error_message': None
    }
    
    start_time = time.time()
    
    try:
        print(f"🚀 MD計算開始（統一ひずみ速度 + 選択的早期終了）")
        
        # MD実行
        md_feature(system, extensions=extensions)
        
        # 正常完了
        execution_info['completed_normally'] = True
        execution_info['total_steps_run'] = UNIFIED_CONDITIONS["max_steps"]
        print(f"✅ MD計算正常完了")
        
    except EarlyTerminationException as e:
        # 早期終了（正常）
        execution_info['early_terminated'] = True
        execution_info['termination_reason'] = e.reason
        execution_info['termination_step'] = e.step
        execution_info['total_steps_run'] = e.step
        
        print(f"🛑 MD計算早期終了")
        print(f"   理由: {e.reason}")
        print(f"   ステップ: {e.step}")
        
    except Exception as e:
        # エラー終了
        execution_info['error_occurred'] = True
        execution_info['error_message'] = str(e)
        
        print(f"❌ MD計算エラー終了: {e}")
        
    finally:
        execution_info['execution_time'] = time.time() - start_time
    
    return execution_info

def prepare_structure_with_top_vacuum(high_temp_file, vacuum_top, fixed_layer_ratio):
    """構造準備（vacuum有無に対応）"""

    if not os.path.exists(high_temp_file):
        raise FileNotFoundError(f"ステップ2高温処理ファイルが見つかりません: {high_temp_file}")

    ase_atoms = read(high_temp_file, "-1")
    ase_atoms = rotate_atoms_to_lower(ase_atoms)

    # 構造解析
    positions = ase_atoms.get_positions()
    z_coords = positions[:, 2]
    z_min, z_max = np.min(z_coords), np.max(z_coords)
    material_thickness = z_max - z_min

    print(f"    構造解析:")
    print(f"      材料厚さ: {material_thickness:.2f}Å")
    print(f"      原子数: {len(ase_atoms)}")

    if vacuum_top > 0:
        # vacuum有りの場合：上部vacuum配置
        print(f"    設定: vacuum有り ({vacuum_top}Å)")
        shift_z = -z_min
        ase_atoms.translate([0, 0, shift_z])

        new_cell_z = material_thickness + vacuum_top
        cell = ase_atoms.get_cell()
        cell[2, 2] = new_cell_z
        ase_atoms.set_cell(cell)
        ase_atoms.wrap()

        # 固定原子設定（上部と下部を相対的な割合で固定）
        current_z_min = np.min(ase_atoms.get_positions()[:, 2])
        current_z_max = np.max(ase_atoms.get_positions()[:, 2])
        current_z_range = current_z_max - current_z_min
        
        fixed_layer_thickness = current_z_range * fixed_layer_ratio/100
        
        # 下部の固定範囲
        lower_fixed_threshold = current_z_min + fixed_layer_thickness
        
        # 上部の固定範囲
        upper_fixed_threshold = current_z_max - fixed_layer_thickness
        
        # 固定する原子のインデックスをリストアップ
        indices = [
            atom.index for atom in ase_atoms
            if atom.position[2] <= lower_fixed_threshold or atom.position[2] >= upper_fixed_threshold
        ]
        
        initial_reference = new_cell_z

    else:
        # vacuum無しの場合：セル長=材料長+マージン
        print(f"    設定: vacuum無し（セル長=材料長+マージン）")
        shift_z = -z_min
        ase_atoms.translate([0, 0, shift_z])

        # 材料長+マージンをセル長とする
        margin = 2.0
        new_cell_z = material_thickness + margin
        cell = ase_atoms.get_cell()
        cell[2, 2] = new_cell_z
        ase_atoms.set_cell(cell)
        ase_atoms.wrap()

        # 固定原子設定（上部と下部を相対的な割合で固定）
        current_z_min = np.min(ase_atoms.get_positions()[:, 2])
        current_z_max = np.max(ase_atoms.get_positions()[:, 2])
        current_z_range = current_z_max - current_z_min
        
        fixed_layer_thickness = current_z_range * fixed_layer_ratio/100
        
        # 下部の固定範囲
        lower_fixed_threshold = current_z_min + fixed_layer_thickness
        
        # 上部の固定範囲
        upper_fixed_threshold = current_z_max - fixed_layer_thickness
        
        # 固定する原子のインデックスをリストアップ
        indices = [
            atom.index for atom in ase_atoms
            if atom.position[2] <= lower_fixed_threshold or atom.position[2] >= upper_fixed_threshold
        ]
        
        initial_reference = new_cell_z

    # 構造情報
    structure_info = {
        "material_thickness": material_thickness,
        "vacuum_top": vacuum_top,
        "vacuum_bottom": 0.0,
        "total_cell_z": new_cell_z,
        "material_ratio": material_thickness / new_cell_z,
        "z_min_material": np.min(ase_atoms.get_positions()[:, 2]),
        "z_max_material": np.max(ase_atoms.get_positions()[:, 2]),
        "fixed_atoms": len(indices),
        "fixed_layer_thickness": fixed_layer_thickness,
        "fixed_ratio": len(indices) / len(ase_atoms),
        "strain_method": "material_length" if vacuum_top > 0 else "cell_length"
    }

    print(f"    配置完了:")
    print(f"      セルZ長: {new_cell_z:.2f}Å")
    print(f"      材料占有率: {structure_info['material_ratio']*100:.1f}%")
    print(f"      固定原子数: {len(indices)}/{len(ase_atoms)} ({structure_info['fixed_ratio']*100:.1f}%)")
    print(f"      ひずみ計算方法: {structure_info['strain_method']}")

    return ase_atoms, initial_reference, structure_info

def save_tensile_results_no_early_termination(results, summary_file, base_name, pressure, comp_temp, high_temp, execution_info):
    """引張計算結果をファイルに保存（早期終了無効化版）"""
    
    with open(summary_file, 'w') as f:
        f.write(f"引張計算結果（早期終了無効） - {base_name}\n")
        f.write("="*70 + "\n")
        f.write(f"計算完了時刻: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"計算時間: {execution_info['execution_time']/60:.1f} 分\n\n")
        
        f.write(f"条件設定:\n")
        f.write(f"  材料: {base_name}\n")
        f.write(f"  圧力: {pressure:.3f} GPa\n")
        f.write(f"  圧着温度: {comp_temp} K\n")
        f.write(f"  高温処理: {high_temp} K\n")
        f.write(f"  引張温度: {results['conditions']['temperature']} K\n\n")
        
        f.write(f"vacuum有無分岐システム:\n")
        if results['structure_info']['vacuum_top'] > 0:
            f.write(f"  ひずみ計算: 材料長基準（vacuum考慮）\n")
            f.write(f"  イベント検出: 分離・破断検出あり（記録のみ）\n")
        else:
            f.write(f"  ひずみ計算: セル長基準（vacuum無し）\n")
            f.write(f"  イベント検出: 基本イベント検出（記録のみ）\n")
        f.write(f"  計算方法: {results['structure_info']['strain_method']}\n\n")
        f.write(f"  ひずみ速度: {results['conditions']['strain_rate']} 1/ps (統一)\n")
        f.write(f"  上部vacuum: {results['structure_info']['vacuum_top']} Å\n")
        f.write(f"  下部vacuum: {results['structure_info']['vacuum_bottom']} Å\n")
        f.write(f"  固定層厚さ: {results['structure_info']['fixed_layer_thickness']}\n")
        f.write(f"  固定原子率: {results['structure_info']['fixed_ratio']*100:.1f}%\n")
        f.write(f"  材料占有率: {results['structure_info']['material_ratio']*100:.1f}%\n\n")
        
        f.write(f"検出設定（記録のみ、早期終了無し）:\n")
        f.write(f"  分離隙間閾値: {EARLY_STOP_CONDITIONS['separation_gap']} Å（記録のみ）\n")
        f.write(f"  最大ひずみ制限: {EARLY_STOP_CONDITIONS['max_strain_limit']}（記録のみ）\n")
        f.write(f"  圧縮転換閾値: {EARLY_STOP_CONDITIONS['min_stress_threshold']} GPa（記録のみ）\n")
        f.write(f"  破断検出: {EARLY_STOP_CONDITIONS['stress_drop_ratio']}（記録のみ）\n\n")
        
        f.write(f"実行結果:\n")
        if execution_info['completed_normally']:
            f.write(f"  実行状態: 正常完了（早期終了無効）\n")
            f.write(f"  実行ステップ数: {execution_info['total_steps_run']}\n")
        elif execution_info['error_occurred']:
            f.write(f"  実行状態: エラー終了\n")
            f.write(f"  エラー: {execution_info['error_message']}\n")
        f.write(f"\n")
        
        f.write(f"機械特性結果:\n")
        f.write(f"  ヤング率: {results['young_modulus']:.1f} GPa\n")
        f.write(f"  ヤング率信頼性: R² = {results['young_r2']:.3f}\n")
        f.write(f"  回帰y切片: {results['young_intercept']:.3f} GPa\n")
        f.write(f"  最大応力: {results['max_stress']:.2f} GPa\n")
        f.write(f"  最大応力時ひずみ: {results['max_stress_strain']:.3f}\n\n")
        
        # 品質評価
        if results['young_r2'] > 0.95:
            quality = "Excellent"
        elif results['young_r2'] > 0.9:
            quality = "Good"
        elif results['young_r2'] > 0.8:
            quality = "Fair"
        else:
            quality = "Poor"
        
        f.write(f"品質評価: {quality} (R² = {results['young_r2']:.3f})\n\n")
        
        # 破断・分離情報（記録のみ）
        f.write(f"破断・分離特性（記録のみ）:\n")
        if results['fracture_detected']:
            f.write(f"  破断検出: Step {results['fracture_step']} (記録のみ、計算は継続)\n")
        else:
            f.write(f"  破断検出: なし\n")
            
        if results['complete_separation']:
            f.write(f"  完全分離: Step {results['separation_step']} (記録のみ、計算は継続)\n")
        else:
            f.write(f"  完全分離: なし\n")
        
        # その他のイベント記録
        if results['termination_details']:
            f.write(f"  その他の検出イベント:\n")
            for event, details in results['termination_details'].items():
                if isinstance(details, dict):
                    f.write(f"    {event}: Step {details.get('step', 'N/A')} (記録のみ)\n")
                else:
                    f.write(f"    {event}: {details} (記録のみ)\n")
        
        f.write(f"\nデータ品質:\n")
        f.write(f"  総データ点数: {results['total_data_points']}\n")
        f.write(f"  計算効率: {execution_info['total_steps_run']}/{UNIFIED_CONDITIONS['max_steps']} ")
        f.write(f"({execution_info['total_steps_run']/UNIFIED_CONDITIONS['max_steps']*100:.1f}%)\n")
        f.write(f"  実行時間: {execution_info['execution_time']/60:.1f} 分\n")
        
        f.write(f"\n早期終了無効化システムの特徴:\n")
        f.write(f"  ✅ 統一ひずみ速度による公平な材料間比較\n")
        f.write(f"  ✅ 全ステップ実行による完全なデータ収集\n")
        f.write(f"  ✅ イベント検出は記録のみ（計算は継続）\n")
        f.write(f"  ✅ 材料長ひずみによる実験対応可能な解析\n")
        f.write(f"  ✅ vacuum設定による効率的な計算設定\n")
        f.write(f"  ✅ 完全なデータセット取得による詳細解析可能\n")

# エネルギー計算用の関数
estimator_fn = pfp_estimator_fn(model_version='v7.0.0', calc_mode='crystal_u0_plus_d3')

def fix_xy_overflow(atoms, method="expand_cell", margin=2.0):
    """XY方向のはみ出しを修正（安全版）"""
    try:
        positions = atoms.get_positions()
        cell = atoms.get_cell()
        
        if not _check_positions_validity(positions, "入力座標"):
            raise ValueError("入力座標に無効な値が含まれています")
        
        if not _check_cell_validity(cell, "入力セル"):
            raise ValueError("入力セルに無効な値が含まれています")
        
        min_coords = np.min(positions, axis=0)
        max_coords = np.max(positions, axis=0)
        
        if not _check_positions_validity(np.array([min_coords, max_coords]), "座標範囲"):
            raise ValueError("座標範囲の計算で無効な値が生成されました")
        
        print(f"修正前の状況:")
        print(f"  セルサイズ: X={cell[0,0]:.3f}, Y={cell[1,1]:.3f}, Z={cell[2,2]:.3f}")
        print(f"  原子範囲: X=[{min_coords[0]:.3f}, {max_coords[0]:.3f}], Y=[{min_coords[1]:.3f}, {max_coords[1]:.3f}]")
        
        x_overflow_low = min_coords[0] < -1e-6
        x_overflow_high = max_coords[0] > cell[0,0] + 1e-6
        y_overflow_low = min_coords[1] < -1e-6
        y_overflow_high = max_coords[1] > cell[1,1] + 1e-6
        
        if method == "expand_cell":
            new_cell = cell.copy()
            
            if x_overflow_low or x_overflow_high:
                coord_range_x = max_coords[0] - min_coords[0]
                
                if coord_range_x > 1000.0:
                    raise ValueError(f"X方向の原子範囲が異常に大きいです: {coord_range_x:.3f} Å")
                
                required_x = coord_range_x + margin
                max_cell_size = 500.0
                new_cell[0, 0] = min(max(cell[0,0], required_x), max_cell_size)
                print(f"  X軸セル拡張: {cell[0,0]:.3f} → {new_cell[0,0]:.3f}")
            
            if y_overflow_low or y_overflow_high:
                coord_range_y = max_coords[1] - min_coords[1]
                
                if coord_range_y > 1000.0:
                    raise ValueError(f"Y方向の原子範囲が異常に大きいです: {coord_range_y:.3f} Å")
                
                required_y = coord_range_y + margin
                max_cell_size = 500.0
                new_cell[1, 1] = min(max(cell[1,1], required_y), max_cell_size)
                print(f"  Y軸セル拡張: {cell[1,1]:.3f} → {new_cell[1,1]:.3f}")
            
            if not _check_cell_validity(new_cell, "新しいセル"):
                raise ValueError("新しいセルの計算で無効な値が生成されました")
            
            atoms.set_cell(new_cell, scale_atoms=False)
            
            if min_coords[0] < -1e-6 or min_coords[1] < -1e-6:
                shift = np.array([
                    max(0, -min_coords[0] + margin/2) if min_coords[0] < -1e-6 else 0,
                    max(0, -min_coords[1] + margin/2) if min_coords[1] < -1e-6 else 0,
                    0
                ])
                
                if np.any(np.abs(shift[:2]) > 100.0):
                    raise ValueError(f"異常に大きなシフトが必要です: {shift}")
                
                if not _check_positions_validity(shift.reshape(1, -1), "シフト量"):
                    raise ValueError("シフト量の計算で無効な値が生成されました")
                
                atoms.translate(shift)
                print(f"  原子移動: X={shift[0]:.3f}, Y={shift[1]:.3f}")
        
        elif method == "wrap_atoms":
            atoms.wrap()
            print(f"  原子をセル内にwrap実行")
        
        elif method == "center_atoms":
            atom_center = (min_coords + max_coords) / 2
            cell_center = np.array([cell[0,0]/2, cell[1,1]/2, cell[2,2]/2])
            
            shift = cell_center - atom_center
            shift[2] = 0
            
            if np.any(np.abs(shift[:2]) > 100.0):
                raise ValueError(f"異常に大きなシフトが必要です: {shift}")
            
            if not _check_positions_validity(shift.reshape(1, -1), "中央配置シフト"):
                raise ValueError("中央配置シフトの計算で無効な値が生成されました")
            
            atoms.translate(shift)
            print(f"  原子を中央配置: X={shift[0]:.3f}, Y={shift[1]:.3f}")
        
        new_positions = atoms.get_positions()
        
        if not _check_positions_validity(new_positions, "修正後座標"):
            raise ValueError("修正後の座標に無効な値が含まれています")
        
        new_min = np.min(new_positions, axis=0)
        new_max = np.max(new_positions, axis=0)
        new_cell = atoms.get_cell()
        
        print(f"修正後の状況:")
        print(f"  セルサイズ: X={new_cell[0,0]:.3f}, Y={new_cell[1,1]:.3f}, Z={new_cell[2,2]:.3f}")
        print(f"  原子範囲: X=[{new_min[0]:.3f}, {new_max[0]:.3f}], Y=[{new_min[1]:.3f}, {new_max[1]:.3f}]")
        
        x_ok = new_min[0] >= -1e-6 and new_max[0] <= new_cell[0,0] + 1e-6
        y_ok = new_min[1] >= -1e-6 and new_max[1] <= new_cell[1,1] + 1e-6
        
        if x_ok and y_ok:
            print(f"  ✅ XY方向はみ出し修正完了")
        else:
            print(f"  ⚠️  修正後もはみ出しあり: X={not x_ok}, Y={not y_ok}")
        
        return atoms
        
    except Exception as e:
        print(f"❌ fix_xy_overflow でエラーが発生しました: {e}")
        print(f"  元の atoms を返します（修正なし）")
        return atoms

def _check_positions_validity(positions, name=""):
    """座標の妥当性をチェックする補助関数"""
    if positions is None or len(positions) == 0:
        print(f"ERROR: {name} - 座標データが空です")
        return False
    
    if np.any(np.isinf(positions)):
        print(f"ERROR: {name} - 無限大値が検出されました")
        inf_indices = np.where(np.isinf(positions))
        print(f"  無限大値の位置: {list(zip(inf_indices[0], inf_indices[1]))}")
        return False
    
    if np.any(np.isnan(positions)):
        print(f"ERROR: {name} - NaN値が検出されました")
        nan_indices = np.where(np.isnan(positions))
        print(f"  NaN値の位置: {list(zip(nan_indices[0], nan_indices[1]))}")
        return False
    
    max_abs_value = np.max(np.abs(positions))
    if max_abs_value > 1e6:
        print(f"WARNING: {name} - 非常に大きな座標値が検出されました: {max_abs_value}")
        return False
    
    return True

def _check_cell_validity(cell, name=""):
    """セルの妥当性をチェックする補助関数"""
    if cell is None:
        print(f"ERROR: {name} - セルデータが空です")
        return False
    
    diagonal = np.diag(cell)
    
    if np.any(diagonal <= 0):
        print(f"ERROR: {name} - セルサイズが0以下です: {diagonal}")
        return False
    
    if np.any(np.isinf(diagonal)) or np.any(np.isnan(diagonal)):
        print(f"ERROR: {name} - セルサイズに無効な値があります: {diagonal}")
        return False
    
    if np.any(diagonal > 1000.0):
        print(f"WARNING: {name} - セルサイズが異常に大きいです: {diagonal}")
        return False
    
    return True

# =============================================
# メイン処理（CSV読み込み版）
# =============================================

print(f"入力CSV: {CSV_FILE_PATH}")
print(f"入力ディレクトリ: {INPUT_DIR}")
print(f"出力ディレクトリ: {OUTPUT_DIR}")
print("="*50)

# ★★★ CSVファイルの読み込み ★★★
try:
    df = pd.read_csv(CSV_FILE_PATH)
    print(f"✅ CSVから {len(df)} 件の計算対象を読み込みました。")
except FileNotFoundError:
    print(f"❌ エラー: CSVファイルが見つかりません: {CSV_FILE_PATH}")
    exit()

# ★★★ CSVの各行を処理 ★★★
for index, row in df.iterrows():
    original_filename = row['original_filename']
    com = row['com']
    atom = row['atom']
    base_name = f"{com}_{atom}"
    
    # ★★★ 入力ファイルパスをCSVから取得 ★★★
    input_file = get_input_file_from_csv(original_filename)
    
    # 入力ファイルの存在チェック
    if not os.path.exists(input_file):
        print(f"\n入力ファイルが見つかりません: {input_file}. スキップします...")
        continue
        
    print(f"\nStarting simulations for {base_name} ({original_filename})")
    
    # 各圧着温度に対してシミュレーション
    for comp_temp in compression_temperatures:
        # 各圧着圧力に対してシミュレーション
        for pressure in compression_pressures:
            compression_file = get_compression_file_path(base_name, pressure, comp_temp)
            
            # ステップ0: 圧着シミュレーション（温度・圧力可変）
            if not check_md_completed(compression_file, 10, 100):
                print(f"Running compression simulation (P={pressure:.3f}GPa, T={comp_temp:.0f}K) for {base_name}")
                
                # ★★★ trajファイルの代わりにxyzファイルから読み込み ★★★
                ase_atoms1 = read(input_file)
                ase_atoms = ase_atoms1.copy()
                
                ase_atoms = rotate_atoms_to_lower(ase_atoms)
                indices = [atom.index for atom in ase_atoms if atom.position[2] <= 3.0]
                c = FixAtoms(indices)
                ase_atoms.set_constraint(c)
                ase_atoms.set_pbc(True)
                ase_atoms.wrap()
                atoms = MatlantisAtoms.from_ase_atoms(ase_atoms)
                atoms.rotate_atoms_to_upper()
                t_step = 1.0
                press_ase = pressure * units.GPa
                atoms.calc = get_calculator(estimator_fn)
                system = ASEMDSystem(atoms)
                system.init_temperature(comp_temp)

                integrator = NPTIntegrator(
                    timestep=t_step,
                    temperature=comp_temp,
                    pressure=press_ase,
                    ttime=20*units.fs,
                    pfactor=2e6*units.GPa*(units.fs**2),
                    mask=np.array([[0,0,0], [0,0,0], [0,0,1]])
                )
                
                info = PrintCellShape()
                
                md = MDFeature(
                    integrator, 
                    n_run=8000, 
                    traj_file_name=compression_file,
                    traj_freq=100, 
                    estimator_fn=estimator_fn
                )
                
                md(system, extensions=[(info, 100)])
                print(f"Compression simulation (P={pressure:.3f}GPa, T={comp_temp:.0f}K) completed for {base_name}")
            else:
                print(f"Compression simulation (P={pressure:.3f}GPa, T={comp_temp:.0f}K) already completed for {base_name}")
            
            # 各高温シミュレーション温度に対して処理
            for high_temp in high_temperatures:
                equil_file = get_equil_file_path(base_name, pressure, comp_temp)
                high_temp_file = get_high_temp_file_path(base_name, pressure, comp_temp, high_temp)
                tensile_file = get_tensile_file_path(base_name, pressure, comp_temp, high_temp, tensile_temperature)
                    
                try:
                    # ステップ1: 短い平衡化（5 ps）
                    if not check_md_completed(equil_file, 5000, 100):
                        print(f"Running equilibration for {base_name} (P={pressure:.3f}GPa, T={comp_temp:.0f}K)")
                        
                        if os.path.exists(compression_file):
                            ase_atoms = read(compression_file, "-1")
                        else:
                            ase_atoms = read(input_file)
                        
                        ase_atoms = rotate_atoms_to_lower(ase_atoms)
                        indices = [atom.index for atom in ase_atoms if atom.position[2] <= 3.0]
                        c = FixAtoms(indices)
                        ase_atoms.set_constraint(c)
                        cell = ase_atoms.get_cell()
                        
                        ase_atoms.set_cell(cell, scale_atoms=True)
                        ase_atoms.wrap()
                        atoms = MatlantisAtoms.from_ase_atoms(ase_atoms)
                        atoms.rotate_atoms_to_upper()
                        system = ASEMDSystem(atoms)
                        system.init_temperature(300)
                        
                        integrator = LangevinIntegrator(timestep=1.0, temperature=300.0)
                        info = PrintCellShape()
                        
                        md = MDFeature(
                            integrator, 
                            n_run=3000, 
                            traj_file_name=equil_file,
                            traj_freq=100, 
                            estimator_fn=estimator_fn
                        )
                        
                        md(system, extensions=[(info, 1000)])
                        print(f"Equilibration completed for {base_name} (P={pressure:.3f}GPa, T={comp_temp:.0f}K)")
                    else:
                        print(f"Equilibration already completed for {base_name} (P={pressure:.3f}GPa, T={comp_temp:.0f}K)")
                    
                    # ステップ2: 高温シミュレーション（可変温度、10ps）
                    if not check_md_completed(high_temp_file, 10000, 100):
                        print(f"Running high temperature simulation ({high_temp}K) for {base_name} (P={pressure:.3f}GPa, T={comp_temp:.0f}K)")
                        
                        ase_atoms = read(equil_file, "-1")
                        ase_atoms = rotate_atoms_to_lower(ase_atoms)
                        indices = [atom.index for atom in ase_atoms if atom.position[2] <= 2.0]
                        c = FixAtoms(indices)
                        ase_atoms.set_constraint(c)
                        atoms = MatlantisAtoms.from_ase_atoms(ase_atoms)
                        atoms.rotate_atoms_to_upper()
                        system = ASEMDSystem(atoms)
                        system.init_temperature(high_temp)
                        
                        integrator = LangevinIntegrator(timestep=1.0, temperature=float(high_temp))
                        info = PrintCellShape()
                        
                        md = MDFeature(
                            integrator, 
                            n_run=10000, 
                            traj_file_name=high_temp_file,
                            traj_freq=100, 
                            estimator_fn=estimator_fn
                        )
                        
                        md(system, extensions=[(info, 500)])
                        print(f"High temperature simulation ({high_temp}K) completed for {base_name} (P={pressure:.3f}GPa, T={comp_temp:.0f}K)")
                    else:
                        print(f"High temperature simulation ({high_temp}K) already completed for {base_name} (P={pressure:.3f}GPa, T={comp_temp:.0f}K)")
                    
                    # ステップ3: 早期終了機能付き引張シミュレーション（温度300K固定）
                    analysis_file = get_tensile_analysis_file_path(base_name, pressure, comp_temp, high_temp)
                    summary_file = get_tensile_summary_file_path(base_name, pressure, comp_temp, high_temp)
                    
                    # 計算済みチェック
                    if check_md_completed(tensile_file, UNIFIED_CONDITIONS["max_steps"], UNIFIED_CONDITIONS["output_freq"]):
                        print(f"Tensile simulation already completed for {base_name} (P={pressure:.3f}GPa, CT={comp_temp}K, HT={high_temp}K)")
                        continue
                    
                    print(f"\n{'='*80}")
                    print(f"早期終了機能付き引張計算開始: {base_name}")
                    print(f"条件: P={pressure:.3f}GPa, CT={comp_temp}K, HT={high_temp}K")
                    print(f"統一ひずみ速度: {UNIFIED_CONDITIONS['strain_rate']} 1/ps")
                    print(f"{'='*80}")
                    
                    # 入力ファイル確認
                    if not os.path.exists(high_temp_file):
                        print(f"❌ ステップ2ファイルが見つかりません: {high_temp_file}")
                        continue
                    
                    # 1. 構造準備
                    print(f"\n📋 Step 1: 構造準備（vacuum設定: {UNIFIED_CONDITIONS['vacuum_top']}Å）")
                    ase_atoms, initial_reference, structure_info = prepare_structure_with_top_vacuum(
                        high_temp_file, 
                        UNIFIED_CONDITIONS["vacuum_top"],
                        UNIFIED_CONDITIONS["fixed_layer_thickness"]
                    )
                    
                    # 2. MD設定
                    print(f"\n📋 Step 2: MD計算設定（統一条件）")
                    atoms = MatlantisAtoms.from_ase_atoms(ase_atoms)
                    atoms.rotate_atoms_to_upper()
                    system = ASEMDSystem(atoms)
                    system.init_temperature(UNIFIED_CONDITIONS["temperature"])
                    
                    # 引張設定
                    strain_rate = UNIFIED_CONDITIONS["strain_rate"]
                    max_strain = UNIFIED_CONDITIONS["max_strain"]
                    timestep = UNIFIED_CONDITIONS["timestep"]
                    
                    total_steps = int(max_strain / (strain_rate * timestep * 1e-3))
                    total_steps = min(total_steps, UNIFIED_CONDITIONS["max_steps"])
                    
                    if UNIFIED_CONDITIONS["vacuum_top"] > 0:
                        # vacuum有り：材料長基準の最終長さ
                        final_material_thickness = structure_info["material_thickness"] * (1 + max_strain)
                        final_cell_length = final_material_thickness + UNIFIED_CONDITIONS["vacuum_top"]
                    else:
                        # vacuum無し：セル長基準の最終長さ
                        final_cell_length = initial_reference * (1 + max_strain)
                    
                    latt = np.array(system.ase_atoms.cell)
                    latt[2, 2] = final_cell_length
                    
                    print(f"  統一ひずみ速度: {strain_rate} 1/ps")
                    print(f"  総ステップ数: {total_steps}")
                    print(f"  推定時間: {total_steps * timestep / 1000:.1f} ps")
                    print(f"  ひずみ計算方法: {structure_info['strain_method']}")
                    
                    # ステップ3: 解析器セットアップ（早期終了無効化版）
                    print(f"\n📋 Step 3: 解析器セットアップ（早期終了無効）")
                    analyzer = ComprehensiveTensileAnalyzer(
                        initial_reference, structure_info, UNIFIED_CONDITIONS, high_temp_file, analysis_file
                    )
                    
                    # 4. MD実行（早期終了無効化版）
                    print(f"\n📋 Step 4: MD計算実行（早期終了無効）")
                    integrator = LangevinIntegrator(
                        timestep=timestep, 
                        temperature=UNIFIED_CONDITIONS["temperature"]
                    )
                    deform = DeformScheduler(latt, total_steps)
                    info = PrintCellShape()
                    
                    md = MDFeature(
                        integrator,
                        n_run=total_steps,
                        traj_file_name=tensile_file,
                        traj_freq=UNIFIED_CONDITIONS["output_freq"],
                        estimator_fn=estimator_fn
                    )
                    
                    extensions = [
                        (info, 100),
                        (deform, 1),
                        (analyzer, 10)
                    ]
                    
                    # 早期終了無効化のMD実行
                    execution_info = run_md_without_early_termination(md, system, extensions)
                    
                    # 5. 結果解析
                    print(f"\n📋 Step 5: 結果解析")
                    final_results = analyzer.get_final_results()
                    final_results['execution_info'] = execution_info
                    
                    # 6. 結果出力（修正版関数を使用）
                    print(f"\n📋 Step 6: 結果出力")
                    save_tensile_results_no_early_termination(
                        final_results, summary_file, base_name, pressure, comp_temp, high_temp, execution_info
                    )
                    
                    print(f"\n✅ 引張計算完了（早期終了無効）: {base_name}")
                    print(f"📊 結果サマリー:")
                    print(f"  ヤング率: {final_results['young_modulus']:.1f} GPa (R² = {final_results['young_r2']:.3f})")
                    print(f"  最大応力: {final_results['max_stress']:.2f} GPa")
                    print(f"  統一ひずみ速度: {final_results['conditions']['strain_rate']} 1/ps")
                    
                    if final_results['fracture_detected']:
                        print(f"  破断検出: Step {final_results['fracture_step']} (記録のみ)")
                    if final_results['complete_separation']:
                        print(f"  完全分離: Step {final_results['separation_step']} (記録のみ)")
                        
                    print(f"All simulations completed successfully for {base_name} (P={pressure:.3f}GPa, CT={comp_temp}K, HT={high_temp}K)")
                    
                except Exception as e:
                    print(f"An error occurred processing {base_name} (P={pressure:.3f}GPa, CT={comp_temp}K, HT={high_temp}K): {e}")
                    import traceback
                    traceback.print_exc()
                    continue

print("All processing complete.")
print(f"結果は以下のディレクトリに保存されました:")
print(f"- 圧着結果: {COMPRESSION_DIR}")
print(f"- 非圧着結果: {HEAT_DIR}")
print(f"- 応力-歪みデータ: {STRESS_STRAIN_DIR}")
print(f"- 早期終了機能付き引張結果: {TENSILE_DIR}")

print(f"\n🎯 早期終了機能付きシステムの特徴:")
print(f"✅ 統一ひずみ速度 ({UNIFIED_CONDITIONS['strain_rate']} 1/ps) による公平な材料間比較")
print(f"✅ 選択的早期終了による計算効率化")
print(f"✅ 破断検出は記録のみ（過度な早期終了を回避）")
print(f"✅ 材料長ひずみによる実験対応可能な解析")
print(f"✅ 上部vacuum配置による効率的な計算")
print(f"✅ 詳細な結果解析とサマリー出力")

print(f"\n📝 CSVベース処理の特徴:")
print(f"✅ CSVファイルからの自動ファイル名取得")
print(f"✅ ファイル存在チェックによる安全な処理")
print(f"✅ 元のコードの動作を完全に維持")
print(f"✅ 入力ディレクトリとファイル名のみ変更")

The MD trajectory will be saved at /home/jovyan/Kaori/MD/LiB_2/output/compression/Al_100_(1, 0, 0)_NMC111_Co_MntoNi_40_012_(0, 1, 2)_P0.001_T300K_compression.traj.
Note: The max disk size of /home/jovyan is about 98G.


入力CSV: /home/jovyan/Kaori/MD/LiB_2/structure/extracted_data_final_with_filenames.csv
入力ディレクトリ: /home/jovyan/Kaori/MD/LiB_2/structure/output/final_interfaces
出力ディレクトリ: /home/jovyan/Kaori/MD/LiB_2/output
✅ CSVから 723 件の計算対象を読み込みました。

Starting simulations for Al_100_(1, 0, 0)_NMC111_Co_MntoNi_40_012_(0, 1, 2) (Interface_Al_100_(1, 0, 0)_on_NMC111_Co_MntoNi_40_012_(0, 1, 2)_trimmed_initial.xyz)
Running compression simulation (P=0.001GPa, T=300K) for Al_100_(1, 0, 0)_NMC111_Co_MntoNi_40_012_(0, 1, 2)
原子をZ方向に -0.00Å 移動しました
Z座標範囲: 0.00 → 0.00, 35.29 → 35.29
Dyn step    0 a 12.00 b 12.00 c 35.79 alpha 90.00 beta 90.00 gamma 90.00 
Dyn step    0 a 12.00 b 12.00 c 35.79 alpha 90.00 beta 90.00 gamma 90.00 


In [ ]:
from ase.io import read, Trajectory, write
from ase.constraints import FixAtoms
import numpy as np
import os
import time
from datetime import datetime
from sklearn.linear_model import LinearRegression
from matlantis_features.features.md import ASEMDSystem, LangevinIntegrator, MDFeature, MDExtensionBase, NPTIntegrator
from matlantis_features.features.md.md_extensions import DeformScheduler
from pfp_api_client.pfp.estimator import Estimator, EstimatorCalcMode
from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
from matlantis_features.utils.calculators import get_calculator, pfp_estimator_fn
from matlantis_features.utils.atoms_util import convert_atoms_to_upper
from matlantis_features.atoms import MatlantisAtoms
import numpy as np
import pathlib
from ase import units
from ase.build import add_vacuum
import pandas as pd
import traceback # To print detailed error information

# =============================================
# ファイル・ディレクトリパス設定（一箇所で管理）
# =============================================

# ベースディレクトリ
BASE_DIR = "/home/jovyan/Kaori/MD/LiB_2/"

# ★★★ 入力ディレクトリの変更 ★★★
# 元のコード: INPUT_DIR = os.path.join(BASE_DIR, "optimized_structures")
INPUT_DIR = os.path.join(BASE_DIR, "structure/output/final_interfaces")

# 出力ディレクトリ
OUTPUT_DIR = os.path.join(BASE_DIR, "output")
COMPRESSION_DIR = os.path.join(OUTPUT_DIR, "compression")
HEAT_DIR = os.path.join(OUTPUT_DIR, "heat")
STRESS_STRAIN_DIR = os.path.join(OUTPUT_DIR, "stress_strain")
TENSILE_DIR = os.path.join(OUTPUT_DIR, "tensile")

# ディレクトリ作成
os.makedirs(INPUT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(COMPRESSION_DIR, exist_ok=True)
os.makedirs(HEAT_DIR, exist_ok=True)
os.makedirs(STRESS_STRAIN_DIR, exist_ok=True)
os.makedirs(TENSILE_DIR, exist_ok=True)

# ★★★ CSV読み込み設定の追加 ★★★
CSV_FILE_PATH = "/home/jovyan/Kaori/MD/LiB_2/structure/extracted_data_final_with_filenames.csv"

def get_input_traj_path(com, atom):
    """入力ファイルのパスを取得"""
    return os.path.join(INPUT_DIR, f"{com}_{atom}_interface_optimized.traj")
    
def get_input_file_path(com, atom):
    """入力ファイルのパスを取得"""
    return os.path.join(INPUT_DIR, f"{com}_{atom}_interface_optimized.xyz")

# ★★★ CSVからファイル名を取得する関数を追加 ★★★
def get_input_file_from_csv(original_filename):
    """CSVの original_filename から入力ファイルパスを取得"""
    return os.path.join(INPUT_DIR, original_filename)

def get_compression_file_path(base_name, pressure, comp_temp):
    """圧着シミュレーションファイルのパスを取得"""
    return os.path.join(COMPRESSION_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_compression.traj")

def get_equil_file_path(base_name, pressure, comp_temp):
    """平衡化ファイルのパスを取得"""
    return os.path.join(HEAT_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_equil.traj")

def get_high_temp_file_path(base_name, pressure, comp_temp, high_temp):
    """高温シミュレーションファイルのパスを取得"""
    return os.path.join(HEAT_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K.traj")

def get_tensile_file_path(base_name, pressure, comp_temp, high_temp, tensile_temperature):
    """引張シミュレーションファイルのパスを取得"""
    return os.path.join(TENSILE_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K_tensile{tensile_temperature:.0f}K_early_stop.traj")

def get_stress_strain_file_path(base_name, pressure, comp_temp, high_temp):
    """応力-歪みデータファイルのパスを取得"""
    return os.path.join(STRESS_STRAIN_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K_stress_strain.dat")

def get_tensile_analysis_file_path(base_name, pressure, comp_temp, high_temp):
    """引張解析ファイルのパスを取得"""
    return os.path.join(TENSILE_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K_tensile_analysis.dat")

def get_tensile_summary_file_path(base_name, pressure, comp_temp, high_temp):
    """引張結果サマリーファイルのパスを取得"""
    return os.path.join(TENSILE_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K_tensile_summary.txt")

# =============================================
# 早期終了システム設定
# =============================================

# 統一条件設定（公平な比較のため）
UNIFIED_CONDITIONS = {
    "strain_rate": 0.01,              # 1/ps（統一ひずみ速度）
    "vacuum_top": 0.0,               # Å（上部vacuum）
    "temperature": 300.0,             # K（引張温度固定）
    "max_strain": 3.0,                # 最大ひずみ（材料長基準）
    "timestep": 1.0,                  # fs
    "elastic_region": (0.005, 0.03),  # 弾性領域（0.5-3.0%）
    "quality_threshold": 0.9,         # R²値閾値
    "max_steps": 100000,              # 最大ステップ数
    "output_freq": 100,               # 出力頻度
    "fixed_layer_thickness": 15      # 固定層の相対厚さ
}

# 早期終了条件設定
EARLY_STOP_CONDITIONS = {
    "separation_gap": 10.0,            # Å 完全分離判定の隙間
    "max_strain_limit": 5.0,          # 最大ひずみ制限
    "min_stress_threshold": -1.0,     # 最小応力閾値（圧縮転換）
    "separation_wait_steps": 500,     # 分離後の待機ステップ
    "stress_drop_ratio": 0.6          # 破断検出用（記録のみ）
}

# =============================================
# 早期終了システムクラス
# =============================================

class EarlyTerminationException(Exception):
    """早期終了用のカスタム例外"""
    def __init__(self, reason, step, details=None):
        self.reason = reason
        self.step = step
        self.details = details or {}
        super().__init__(f"Early termination at step {step}: {reason}")

class ComprehensiveTensileAnalyzer(MDExtensionBase):
    """
    早期終了機能を無効化した包括的引張解析器
    """
    
    def __init__(self, initial_thickness, structure_info, conditions, source_file, output_file):
        self.initial_thickness = initial_thickness
        self.structure_info = structure_info
        self.conditions = conditions
        self.source_file = source_file
        self.output_file = output_file
        
        # 解析データ
        self.data = []
        self.young_modulus = None
        self.young_r2 = 0.0
        self.young_intercept = 0.0
        self.max_stress = 0.0
        self.max_stress_strain = 0.0
        self.fracture_detected = False
        self.fracture_step = None
        self.complete_separation = False
        self.separation_step = None
        
        # 早期終了関連（無効化）
        self.early_termination_requested = False
        self.termination_reason = None
        self.termination_details = {}
        
        # チェック用カウンタ（記録のみ）
        self.stress_drop_count = 0
        self.separation_wait_count = 0
        
        # ヘッダー書き込み
        with open(self.output_file, 'w') as f:
            f.write(f"# Integrated Tensile Analysis (No Early Termination)\n")
            f.write(f"# Source: {source_file}\n")
            f.write(f"# Unified Strain Rate: {conditions['strain_rate']} 1/ps\n")
            f.write(f"# Detection Conditions (Record Only):\n")
            f.write(f"#   Separation Gap: {EARLY_STOP_CONDITIONS['separation_gap']} A (RECORD)\n")
            f.write(f"#   Max Strain Limit: {EARLY_STOP_CONDITIONS['max_strain_limit']} (RECORD)\n")
            f.write(f"#   Min Stress Threshold: {EARLY_STOP_CONDITIONS['min_stress_threshold']} GPa (RECORD)\n")
            f.write(f"#   Stress Drop Detection: {EARLY_STOP_CONDITIONS['stress_drop_ratio']} (RECORD)\n")
            if UNIFIED_CONDITIONS["vacuum_top"] > 0:
                f.write(f"# Step Material_Strain Stress(GPa) YoungMod(GPa) Status Time(ps) Notes\n")
            else:
                f.write(f"# Step Cell_Strain Stress(GPa) YoungMod(GPa) Status Time(ps) Notes\n")

    def detect_events(self, system, material_strain, stress_zz_GPa, z_coords=None):
        """イベント検出（記録のみ、早期終了はしない）"""
        
        # 1. 完全分離チェック（vacuum有りの場合のみ）
        if UNIFIED_CONDITIONS["vacuum_top"] > 0 and z_coords is not None:
            if system.current_total_step % 100 == 0:
                z_median = np.median(z_coords)
                upper_z = z_coords[z_coords > z_median]
                lower_z = z_coords[z_coords <= z_median]
                
                if len(upper_z) > 0 and len(lower_z) > 0:
                    gap = np.min(upper_z) - np.max(lower_z)
                    if np.max(lower_z) >= 8.0:
                        if gap > EARLY_STOP_CONDITIONS["separation_gap"]:
                            if not self.complete_separation:
                                self.complete_separation = True
                                self.separation_step = system.current_total_step
                                print(f"      🔗 完全分離検出: Step {self.separation_step}, 隙間 {gap:.1f}Å（記録のみ）")
                                
                                self.termination_details.update({
                                    "separation_gap": gap,
                                    "separation_step": self.separation_step
                                })
                            
                            self.separation_wait_count += 1
        
        # 2. 応力急減チェック（破断検出）- 記録のみ
        if len(self.data) > 100 and self.max_stress > 0:
            recent_stresses = [d[2] for d in self.data[-50:]]
            if len(recent_stresses) > 0:
                recent_max_stress = max(recent_stresses)
                stress_ratio = recent_max_stress / self.max_stress
                
                if stress_ratio < EARLY_STOP_CONDITIONS["stress_drop_ratio"]:
                    if not self.fracture_detected:
                        self.fracture_detected = True
                        self.fracture_step = system.current_total_step
                        print(f"      ⚠️  破断検出: Step {self.fracture_step}, 応力比 {stress_ratio:.3f}（記録のみ）")
                        
                        self.termination_details.update({
                            "fracture_stress_ratio": stress_ratio,
                            "fracture_step": self.fracture_step
                        })
        
        # 3. 最大ひずみ制限チェック - 記録のみ
        if material_strain > EARLY_STOP_CONDITIONS["max_strain_limit"]:
            if "max_strain_exceeded" not in self.termination_details:
                print(f"      📊 最大ひずみ制限超過: {material_strain:.3f}（記録のみ）")
                self.termination_details["max_strain_exceeded"] = {
                    "strain": material_strain,
                    "step": system.current_total_step
                }
        
        # 4. 圧縮転換チェック - 記録のみ
        if stress_zz_GPa < EARLY_STOP_CONDITIONS["min_stress_threshold"]:
            if "compression_detected" not in self.termination_details:
                print(f"      ⬇️  圧縮転換検出: {stress_zz_GPa:.3f} GPa（記録のみ）")
                self.termination_details["compression_detected"] = {
                    "stress": stress_zz_GPa,
                    "step": system.current_total_step
                }
    
    def __call__(self, system, integrator):
        # ひずみ計算の分岐（vacuum有無による）
        if UNIFIED_CONDITIONS["vacuum_top"] > 0:
            # vacuum有り：材料長基準
            positions = system.ase_atoms.get_positions()
            z_coords = positions[:, 2]
            current_material_length = np.max(z_coords) - np.min(z_coords)
            material_strain = (current_material_length - self.initial_thickness) / self.initial_thickness
            strain_method = "material_length"
        else:
            # vacuum無し：セル長基準
            current_cell_length = system.ase_atoms.get_cell()[2, 2]
            material_strain = (current_cell_length - self.initial_thickness) / self.initial_thickness
            current_material_length = current_cell_length  # セル長=材料長
            strain_method = "cell_length"
            z_coords = None
        
        # 応力取得
        try:
            stress_tensor = system.ase_atoms.get_stress(voigt=False)
            stress_zz_GPa = stress_tensor[2, 2] * 160.21766208  # eV/Å³ to GPa
        except:
            stress_zz_GPa = 0.0
        
        # セル長
        cell_length = system.ase_atoms.get_cell()[2, 2]
        time_ps = system.current_total_step * self.conditions["timestep"] / 1000
        
        if UNIFIED_CONDITIONS["vacuum_top"] > 0:
            # vacuum有り：材料長も記録
            self.data.append([
                system.current_total_step, material_strain, stress_zz_GPa,
                current_material_length, cell_length, time_ps
            ])
        else:
            # vacuum無し：材料長記録なし（セル長のみ）
            self.data.append([
                system.current_total_step, material_strain, stress_zz_GPa,
                None, cell_length, time_ps
            ])
        
        # 最大応力更新
        if stress_zz_GPa > self.max_stress:
            self.max_stress = stress_zz_GPa
            self.max_stress_strain = material_strain
        
        # ヤング率計算
        status = "Loading"
        current_young = 0.0
        
        if len(self.data) > 50 and material_strain > 0.01 and self.young_modulus is None:
            strains = np.array([d[1] for d in self.data])
            stresses = np.array([d[2] for d in self.data])
            
            elastic_min, elastic_max = self.conditions["elastic_region"]
            elastic_mask = (strains > elastic_min) & (strains < elastic_max) & (stresses > 0)
            
            if np.sum(elastic_mask) > 20:
                elastic_strains = strains[elastic_mask]
                elastic_stresses = stresses[elastic_mask]
                
                X = elastic_strains.reshape(-1, 1)
                y = elastic_stresses
                
                model = LinearRegression().fit(X, y)
                predicted = model.predict(X)
                
                self.young_modulus = model.coef_[0]
                self.young_intercept = model.intercept_
                
                ss_res = np.sum((y - predicted) ** 2)
                ss_tot = np.sum((y - np.mean(y)) ** 2)
                self.young_r2 = 1 - (ss_res / ss_tot) if ss_tot > 0 else 0
                
                print(f"      ✅ ヤング率確定: {self.young_modulus:.1f} GPa (R² = {self.young_r2:.3f})")
        
        if self.young_modulus is not None:
            current_young = self.young_modulus
            if material_strain > self.conditions["elastic_region"][1]:
                status = "Plastic"
        
        # イベント検出（記録のみ、早期終了はしない）
        self.detect_events(system, material_strain, stress_zz_GPa, z_coords)
        
        # 状態更新
        if self.complete_separation:
            status = "Separated"
        elif self.fracture_detected:
            status = "Fractured"
        
        # 早期終了処理は無効化 - 常に計算を継続
        
        # ファイル出力（計算方法も記録）
        notes = f"Method:{strain_method}"
        if self.complete_separation:
            notes += f" Sep:{self.separation_wait_count}"
        if self.fracture_detected:
            notes += f" Frac"
        
        with open(self.output_file, 'a') as f:
            f.write(f"{system.current_total_step:<8} {material_strain:<14.6f} {stress_zz_GPa:<12.6f} ")
            f.write(f"{current_young:<12.1f} {status:<10} {time_ps:<10.3f} {notes}\n")
    
    def get_final_results(self):
        """最終結果を辞書形式で返す"""
        return {
            'young_modulus': self.young_modulus or 0.0,
            'young_r2': self.young_r2,
            'young_intercept': self.young_intercept,
            'max_stress': self.max_stress,
            'max_stress_strain': self.max_stress_strain,
            'fracture_detected': self.fracture_detected,
            'fracture_step': self.fracture_step,
            'complete_separation': self.complete_separation,
            'separation_step': self.separation_step,
            'total_data_points': len(self.data),
            'structure_info': self.structure_info,
            'conditions': self.conditions,
            'early_termination': False,  # 常にFalse
            'termination_reason': None,
            'termination_details': self.termination_details
        }

def run_md_without_early_termination(md_feature, system, extensions):
    """早期終了機能無しのMD実行ラッパー"""
    
    execution_info = {
        'completed_normally': False,
        'early_terminated': False,
        'termination_reason': None,
        'termination_step': None,
        'total_steps_run': 0,
        'execution_time': 0.0,
        'error_occurred': False,
        'error_message': None
    }
    
    start_time = time.time()
    
    try:
        print(f"🚀 MD計算開始（早期終了無効）")
        
        # MD実行（早期終了例外はキャッチしない）
        md_feature(system, extensions=extensions)
        
        # 正常完了
        execution_info['completed_normally'] = True
        execution_info['total_steps_run'] = UNIFIED_CONDITIONS["max_steps"]
        print(f"✅ MD計算正常完了")
        
    except Exception as e:
        # エラー終了のみキャッチ（EarlyTerminationExceptionもここでキャッチされるが、無視する）
        if "EarlyTerminationException" in str(type(e)):
            # 早期終了例外の場合は正常完了として扱う
            execution_info['completed_normally'] = True
            execution_info['total_steps_run'] = getattr(e, 'step', UNIFIED_CONDITIONS["max_steps"])
            print(f"✅ MD計算完了（早期終了イベント検出済み）")
        else:
            # 真のエラー終了
            execution_info['error_occurred'] = True
            execution_info['error_message'] = str(e)
            print(f"❌ MD計算エラー終了: {e}")
        
    finally:
        execution_info['execution_time'] = time.time() - start_time
    
    return execution_info

# =============================================
# シミュレーションパラメータ設定
# =============================================

# ★★★ 物質リストをCSVから読み込みに変更 ★★★
# パラメータの設定 - カスタマイズ可能
compression_temperatures = [300]  # 圧着シミュレーション温度（K）
compression_pressures = [0.001,0.005,0.01]  # 圧着圧力（GPa）
high_temperatures = [300]  # 高温シミュレーション温度（K）
tensile_temperature = 300.0  # 引張シミュレーション温度（K）

# =============================================
# 既存の関数群（変更なし）
# =============================================

def check_md_completed(traj_path, expected_steps, traj_freq):
    """シミュレーションが既に完了しているかチェックする関数"""
    try:
        if not os.path.exists(traj_path):
            return False
        
        traj = Trajectory(traj_path, 'r')
        num_frames = len(traj)
        expected_frames = expected_steps // traj_freq
        
        return num_frames >= expected_frames * 0.1
    except Exception:
        return False

class PrintCellShape(MDExtensionBase):
    def __init__(self, cell_log=None):
        self.cell_log = cell_log
        
    def __call__(self, system, integrator) -> None:
        cell_par = system.ase_atoms.cell.cellpar()
        istep = system.current_total_step
        if istep % 1000 == 0:
            print(f"Dyn step {istep:4d} a {cell_par[0]:3.2f} b {cell_par[1]:3.2f} c {cell_par[2]:3.2f} alpha {cell_par[3]:3.2f} beta {cell_par[4]:3.2f} gamma {cell_par[5]:3.2f} ")
        if self.cell_log is not None:
            self.cell_log.append(cell_par)

def rotate_atoms_to_lower(atoms):
    """原子系のZ座標を確認し、全原子を下端（Z=0付近）に移動させる"""
    ase_atoms = atoms.ase_atoms if hasattr(atoms, 'ase_atoms') else atoms
    
    positions = ase_atoms.get_positions()
    z_min = np.min(positions[:, 2])
    z_max = np.max(positions[:, 2])
    
    shift_z = -z_min
    translation = np.array([0, 0, shift_z])
    ase_atoms.translate(translation)
    z_min2 = np.min(positions[:, 2])
    z_max2 = np.max(positions[:, 2])
    
    # 原子の厚み（Z方向の範囲）を計算
    thickness = z_max2 - z_min2
    new_cell =  ase_atoms.get_cell()
    new_cell[2, 2] =  z_max+shift_z+0.5
    ase_atoms.set_cell(new_cell, scale_atoms=False)
    ase_atoms.wrap()
    print(f"原子をZ方向に {shift_z:.2f}Å 移動しました")
    print(f"Z座標範囲: {z_min:.2f} → {z_min+shift_z:.2f}, {z_max:.2f} → {z_max+shift_z:.2f}")
    
    return ase_atoms

def set_fixed_atoms_at_bottom(atoms, z_threshold=UNIFIED_CONDITIONS["fixed_layer_thickness"]):
    """Z座標が閾値以下の原子を固定する"""
    positions = atoms.get_positions()
    indices = [atom.index for atom in atoms if atom.position[2] <= z_threshold]
    
    print(f"固定する原子数: {len(indices)} / {len(atoms)}")
    
    constraint = FixAtoms(indices=indices)
    atoms.set_constraint(constraint)
    
    return atoms

class StressStrainRecorder(MDExtensionBase):
    """応力-歪み記録クラス"""
    def __init__(self, initial_length, output_file):
        self.initial_length = initial_length
        self.output_file = output_file
        self.data = []
        
    def __call__(self, system, integrator):
        current_cell = system.ase_atoms.get_cell()
        current_length = current_cell[2, 2] - 15
        strain = (current_length - self.initial_length) / self.initial_length
        
        try:
            stress = system.ase_atoms.get_stress()
            stress_zz = stress[2]
        except:
            stress_zz = 0.0
        
        self.data.append([system.current_total_step, strain, stress_zz])
        
        if len(self.data) % 10 == 0:
            np.savetxt(self.output_file, self.data, 
                      header="Step Strain Stress_ZZ", fmt="%.6f")

def run_md_with_early_termination(md_feature, system, extensions):
    """早期終了機能付きのMD実行ラッパー"""
    
    execution_info = {
        'completed_normally': False,
        'early_terminated': False,
        'termination_reason': None,
        'termination_step': None,
        'total_steps_run': 0,
        'execution_time': 0.0,
        'error_occurred': False,
        'error_message': None
    }
    
    start_time = time.time()
    
    try:
        print(f"🚀 MD計算開始（統一ひずみ速度 + 選択的早期終了）")
        
        # MD実行
        md_feature(system, extensions=extensions)
        
        # 正常完了
        execution_info['completed_normally'] = True
        execution_info['total_steps_run'] = UNIFIED_CONDITIONS["max_steps"]
        print(f"✅ MD計算正常完了")
        
    except EarlyTerminationException as e:
        # 早期終了（正常）
        execution_info['early_terminated'] = True
        execution_info['termination_reason'] = e.reason
        execution_info['termination_step'] = e.step
        execution_info['total_steps_run'] = e.step
        
        print(f"🛑 MD計算早期終了")
        print(f"   理由: {e.reason}")
        print(f"   ステップ: {e.step}")
        
    except Exception as e:
        # エラー終了
        execution_info['error_occurred'] = True
        execution_info['error_message'] = str(e)
        
        print(f"❌ MD計算エラー終了: {e}")
        
    finally:
        execution_info['execution_time'] = time.time() - start_time
    
    return execution_info

def prepare_structure_with_top_vacuum(high_temp_file, vacuum_top, fixed_layer_ratio):
    """構造準備（vacuum有無に対応）"""

    if not os.path.exists(high_temp_file):
        raise FileNotFoundError(f"ステップ2高温処理ファイルが見つかりません: {high_temp_file}")

    ase_atoms = read(high_temp_file, "-1")
    ase_atoms = rotate_atoms_to_lower(ase_atoms)

    # 構造解析
    positions = ase_atoms.get_positions()
    z_coords = positions[:, 2]
    z_min, z_max = np.min(z_coords), np.max(z_coords)
    material_thickness = z_max - z_min

    print(f"    構造解析:")
    print(f"      材料厚さ: {material_thickness:.2f}Å")
    print(f"      原子数: {len(ase_atoms)}")

    if vacuum_top > 0:
        # vacuum有りの場合：上部vacuum配置
        print(f"    設定: vacuum有り ({vacuum_top}Å)")
        shift_z = -z_min
        ase_atoms.translate([0, 0, shift_z])

        new_cell_z = material_thickness + vacuum_top
        cell = ase_atoms.get_cell()
        cell[2, 2] = new_cell_z
        ase_atoms.set_cell(cell)
        ase_atoms.wrap()

        # 固定原子設定（上部と下部を相対的な割合で固定）
        current_z_min = np.min(ase_atoms.get_positions()[:, 2])
        current_z_max = np.max(ase_atoms.get_positions()[:, 2])
        current_z_range = current_z_max - current_z_min
        
        fixed_layer_thickness = current_z_range * fixed_layer_ratio/100
        
        # 下部の固定範囲
        lower_fixed_threshold = current_z_min + fixed_layer_thickness
        
        # 上部の固定範囲
        upper_fixed_threshold = current_z_max - fixed_layer_thickness
        
        # 固定する原子のインデックスをリストアップ
        indices = [
            atom.index for atom in ase_atoms
            if atom.position[2] <= lower_fixed_threshold or atom.position[2] >= upper_fixed_threshold
        ]
        
        initial_reference = new_cell_z

    else:
        # vacuum無しの場合：セル長=材料長+マージン
        print(f"    設定: vacuum無し（セル長=材料長+マージン）")
        shift_z = -z_min
        ase_atoms.translate([0, 0, shift_z])

        # 材料長+マージンをセル長とする
        margin = 2.0
        new_cell_z = material_thickness + margin
        cell = ase_atoms.get_cell()
        cell[2, 2] = new_cell_z
        ase_atoms.set_cell(cell)
        ase_atoms.wrap()

        # 固定原子設定（上部と下部を相対的な割合で固定）
        current_z_min = np.min(ase_atoms.get_positions()[:, 2])
        current_z_max = np.max(ase_atoms.get_positions()[:, 2])
        current_z_range = current_z_max - current_z_min
        
        fixed_layer_thickness = current_z_range * fixed_layer_ratio/100
        
        # 下部の固定範囲
        lower_fixed_threshold = current_z_min + fixed_layer_thickness
        
        # 上部の固定範囲
        upper_fixed_threshold = current_z_max - fixed_layer_thickness
        
        # 固定する原子のインデックスをリストアップ
        indices = [
            atom.index for atom in ase_atoms
            if atom.position[2] <= lower_fixed_threshold or atom.position[2] >= upper_fixed_threshold
        ]
        
        initial_reference = new_cell_z

    # 構造情報
    structure_info = {
        "material_thickness": material_thickness,
        "vacuum_top": vacuum_top,
        "vacuum_bottom": 0.0,
        "total_cell_z": new_cell_z,
        "material_ratio": material_thickness / new_cell_z,
        "z_min_material": np.min(ase_atoms.get_positions()[:, 2]),
        "z_max_material": np.max(ase_atoms.get_positions()[:, 2]),
        "fixed_atoms": len(indices),
        "fixed_layer_thickness": fixed_layer_thickness,
        "fixed_ratio": len(indices) / len(ase_atoms),
        "strain_method": "material_length" if vacuum_top > 0 else "cell_length"
    }

    print(f"    配置完了:")
    print(f"      セルZ長: {new_cell_z:.2f}Å")
    print(f"      材料占有率: {structure_info['material_ratio']*100:.1f}%")
    print(f"      固定原子数: {len(indices)}/{len(ase_atoms)} ({structure_info['fixed_ratio']*100:.1f}%)")
    print(f"      ひずみ計算方法: {structure_info['strain_method']}")

    return ase_atoms, initial_reference, structure_info

def save_tensile_results_no_early_termination(results, summary_file, base_name, pressure, comp_temp, high_temp, execution_info):
    """引張計算結果をファイルに保存（早期終了無効化版）"""
    
    with open(summary_file, 'w') as f:
        f.write(f"引張計算結果（早期終了無効） - {base_name}\n")
        f.write("="*70 + "\n")
        f.write(f"計算完了時刻: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"計算時間: {execution_info['execution_time']/60:.1f} 分\n\n")
        
        f.write(f"条件設定:\n")
        f.write(f"  材料: {base_name}\n")
        f.write(f"  圧力: {pressure:.3f} GPa\n")
        f.write(f"  圧着温度: {comp_temp} K\n")
        f.write(f"  高温処理: {high_temp} K\n")
        f.write(f"  引張温度: {results['conditions']['temperature']} K\n\n")
        
        f.write(f"vacuum有無分岐システム:\n")
        if results['structure_info']['vacuum_top'] > 0:
            f.write(f"  ひずみ計算: 材料長基準（vacuum考慮）\n")
            f.write(f"  イベント検出: 分離・破断検出あり（記録のみ）\n")
        else:
            f.write(f"  ひずみ計算: セル長基準（vacuum無し）\n")
            f.write(f"  イベント検出: 基本イベント検出（記録のみ）\n")
        f.write(f"  計算方法: {results['structure_info']['strain_method']}\n\n")
        f.write(f"  ひずみ速度: {results['conditions']['strain_rate']} 1/ps (統一)\n")
        f.write(f"  上部vacuum: {results['structure_info']['vacuum_top']} Å\n")
        f.write(f"  下部vacuum: {results['structure_info']['vacuum_bottom']} Å\n")
        f.write(f"  固定層厚さ: {results['structure_info']['fixed_layer_thickness']}\n")
        f.write(f"  固定原子率: {results['structure_info']['fixed_ratio']*100:.1f}%\n")
        f.write(f"  材料占有率: {results['structure_info']['material_ratio']*100:.1f}%\n\n")
        
        f.write(f"検出設定（記録のみ、早期終了無し）:\n")
        f.write(f"  分離隙間閾値: {EARLY_STOP_CONDITIONS['separation_gap']} Å（記録のみ）\n")
        f.write(f"  最大ひずみ制限: {EARLY_STOP_CONDITIONS['max_strain_limit']}（記録のみ）\n")
        f.write(f"  圧縮転換閾値: {EARLY_STOP_CONDITIONS['min_stress_threshold']} GPa（記録のみ）\n")
        f.write(f"  破断検出: {EARLY_STOP_CONDITIONS['stress_drop_ratio']}（記録のみ）\n\n")
        
        f.write(f"実行結果:\n")
        if execution_info['completed_normally']:
            f.write(f"  実行状態: 正常完了（早期終了無効）\n")
            f.write(f"  実行ステップ数: {execution_info['total_steps_run']}\n")
        elif execution_info['error_occurred']:
            f.write(f"  実行状態: エラー終了\n")
            f.write(f"  エラー: {execution_info['error_message']}\n")
        f.write(f"\n")
        
        f.write(f"機械特性結果:\n")
        f.write(f"  ヤング率: {results['young_modulus']:.1f} GPa\n")
        f.write(f"  ヤング率信頼性: R² = {results['young_r2']:.3f}\n")
        f.write(f"  回帰y切片: {results['young_intercept']:.3f} GPa\n")
        f.write(f"  最大応力: {results['max_stress']:.2f} GPa\n")
        f.write(f"  最大応力時ひずみ: {results['max_stress_strain']:.3f}\n\n")
        
        # 品質評価
        if results['young_r2'] > 0.95:
            quality = "Excellent"
        elif results['young_r2'] > 0.9:
            quality = "Good"
        elif results['young_r2'] > 0.8:
            quality = "Fair"
        else:
            quality = "Poor"
        
        f.write(f"品質評価: {quality} (R² = {results['young_r2']:.3f})\n\n")
        
        # 破断・分離情報（記録のみ）
        f.write(f"破断・分離特性（記録のみ）:\n")
        if results['fracture_detected']:
            f.write(f"  破断検出: Step {results['fracture_step']} (記録のみ、計算は継続)\n")
        else:
            f.write(f"  破断検出: なし\n")
            
        if results['complete_separation']:
            f.write(f"  完全分離: Step {results['separation_step']} (記録のみ、計算は継続)\n")
        else:
            f.write(f"  完全分離: なし\n")
        
        # その他のイベント記録
        if results['termination_details']:
            f.write(f"  その他の検出イベント:\n")
            for event, details in results['termination_details'].items():
                if isinstance(details, dict):
                    f.write(f"    {event}: Step {details.get('step', 'N/A')} (記録のみ)\n")
                else:
                    f.write(f"    {event}: {details} (記録のみ)\n")
        
        f.write(f"\nデータ品質:\n")
        f.write(f"  総データ点数: {results['total_data_points']}\n")
        f.write(f"  計算効率: {execution_info['total_steps_run']}/{UNIFIED_CONDITIONS['max_steps']} ")
        f.write(f"({execution_info['total_steps_run']/UNIFIED_CONDITIONS['max_steps']*100:.1f}%)\n")
        f.write(f"  実行時間: {execution_info['execution_time']/60:.1f} 分\n")
        
        f.write(f"\n早期終了無効化システムの特徴:\n")
        f.write(f"  ✅ 統一ひずみ速度による公平な材料間比較\n")
        f.write(f"  ✅ 全ステップ実行による完全なデータ収集\n")
        f.write(f"  ✅ イベント検出は記録のみ（計算は継続）\n")
        f.write(f"  ✅ 材料長ひずみによる実験対応可能な解析\n")
        f.write(f"  ✅ vacuum設定による効率的な計算設定\n")
        f.write(f"  ✅ 完全なデータセット取得による詳細解析可能\n")

# エネルギー計算用の関数
estimator_fn = pfp_estimator_fn(model_version='v7.0.0', calc_mode='crystal_u0_plus_d3')

def fix_xy_overflow(atoms, method="expand_cell", margin=2.0):
    """XY方向のはみ出しを修正（安全版）"""
    try:
        positions = atoms.get_positions()
        cell = atoms.get_cell()
        
        if not _check_positions_validity(positions, "入力座標"):
            raise ValueError("入力座標に無効な値が含まれています")
        
        if not _check_cell_validity(cell, "入力セル"):
            raise ValueError("入力セルに無効な値が含まれています")
        
        min_coords = np.min(positions, axis=0)
        max_coords = np.max(positions, axis=0)
        
        if not _check_positions_validity(np.array([min_coords, max_coords]), "座標範囲"):
            raise ValueError("座標範囲の計算で無効な値が生成されました")
        
        print(f"修正前の状況:")
        print(f"  セルサイズ: X={cell[0,0]:.3f}, Y={cell[1,1]:.3f}, Z={cell[2,2]:.3f}")
        print(f"  原子範囲: X=[{min_coords[0]:.3f}, {max_coords[0]:.3f}], Y=[{min_coords[1]:.3f}, {max_coords[1]:.3f}]")
        
        x_overflow_low = min_coords[0] < -1e-6
        x_overflow_high = max_coords[0] > cell[0,0] + 1e-6
        y_overflow_low = min_coords[1] < -1e-6
        y_overflow_high = max_coords[1] > cell[1,1] + 1e-6
        
        if method == "expand_cell":
            new_cell = cell.copy()
            
            if x_overflow_low or x_overflow_high:
                coord_range_x = max_coords[0] - min_coords[0]
                
                if coord_range_x > 1000.0:
                    raise ValueError(f"X方向の原子範囲が異常に大きいです: {coord_range_x:.3f} Å")
                
                required_x = coord_range_x + margin
                max_cell_size = 500.0
                new_cell[0, 0] = min(max(cell[0,0], required_x), max_cell_size)
                print(f"  X軸セル拡張: {cell[0,0]:.3f} → {new_cell[0,0]:.3f}")
            
            if y_overflow_low or y_overflow_high:
                coord_range_y = max_coords[1] - min_coords[1]
                
                if coord_range_y > 1000.0:
                    raise ValueError(f"Y方向の原子範囲が異常に大きいです: {coord_range_y:.3f} Å")
                
                required_y = coord_range_y + margin
                max_cell_size = 500.0
                new_cell[1, 1] = min(max(cell[1,1], required_y), max_cell_size)
                print(f"  Y軸セル拡張: {cell[1,1]:.3f} → {new_cell[1,1]:.3f}")
            
            if not _check_cell_validity(new_cell, "新しいセル"):
                raise ValueError("新しいセルの計算で無効な値が生成されました")
            
            atoms.set_cell(new_cell, scale_atoms=False)
            
            if min_coords[0] < -1e-6 or min_coords[1] < -1e-6:
                shift = np.array([
                    max(0, -min_coords[0] + margin/2) if min_coords[0] < -1e-6 else 0,
                    max(0, -min_coords[1] + margin/2) if min_coords[1] < -1e-6 else 0,
                    0
                ])
                
                if np.any(np.abs(shift[:2]) > 100.0):
                    raise ValueError(f"異常に大きなシフトが必要です: {shift}")
                
                if not _check_positions_validity(shift.reshape(1, -1), "シフト量"):
                    raise ValueError("シフト量の計算で無効な値が生成されました")
                
                atoms.translate(shift)
                print(f"  原子移動: X={shift[0]:.3f}, Y={shift[1]:.3f}")
        
        elif method == "wrap_atoms":
            atoms.wrap()
            print(f"  原子をセル内にwrap実行")
        
        elif method == "center_atoms":
            atom_center = (min_coords + max_coords) / 2
            cell_center = np.array([cell[0,0]/2, cell[1,1]/2, cell[2,2]/2])
            
            shift = cell_center - atom_center
            shift[2] = 0
            
            if np.any(np.abs(shift[:2]) > 100.0):
                raise ValueError(f"異常に大きなシフトが必要です: {shift}")
            
            if not _check_positions_validity(shift.reshape(1, -1), "中央配置シフト"):
                raise ValueError("中央配置シフトの計算で無効な値が生成されました")
            
            atoms.translate(shift)
            print(f"  原子を中央配置: X={shift[0]:.3f}, Y={shift[1]:.3f}")
        
        new_positions = atoms.get_positions()
        
        if not _check_positions_validity(new_positions, "修正後座標"):
            raise ValueError("修正後の座標に無効な値が含まれています")
        
        new_min = np.min(new_positions, axis=0)
        new_max = np.max(new_positions, axis=0)
        new_cell = atoms.get_cell()
        
        print(f"修正後の状況:")
        print(f"  セルサイズ: X={new_cell[0,0]:.3f}, Y={new_cell[1,1]:.3f}, Z={new_cell[2,2]:.3f}")
        print(f"  原子範囲: X=[{new_min[0]:.3f}, {new_max[0]:.3f}], Y=[{new_min[1]:.3f}, {new_max[1]:.3f}]")
        
        x_ok = new_min[0] >= -1e-6 and new_max[0] <= new_cell[0,0] + 1e-6
        y_ok = new_min[1] >= -1e-6 and new_max[1] <= new_cell[1,1] + 1e-6
        
        if x_ok and y_ok:
            print(f"  ✅ XY方向はみ出し修正完了")
        else:
            print(f"  ⚠️  修正後もはみ出しあり: X={not x_ok}, Y={not y_ok}")
        
        return atoms
        
    except Exception as e:
        print(f"❌ fix_xy_overflow でエラーが発生しました: {e}")
        print(f"  元の atoms を返します（修正なし）")
        return atoms

def _check_positions_validity(positions, name=""):
    """座標の妥当性をチェックする補助関数"""
    if positions is None or len(positions) == 0:
        print(f"ERROR: {name} - 座標データが空です")
        return False
    
    if np.any(np.isinf(positions)):
        print(f"ERROR: {name} - 無限大値が検出されました")
        inf_indices = np.where(np.isinf(positions))
        print(f"  無限大値の位置: {list(zip(inf_indices[0], inf_indices[1]))}")
        return False
    
    if np.any(np.isnan(positions)):
        print(f"ERROR: {name} - NaN値が検出されました")
        nan_indices = np.where(np.isnan(positions))
        print(f"  NaN値の位置: {list(zip(nan_indices[0], nan_indices[1]))}")
        return False
    
    max_abs_value = np.max(np.abs(positions))
    if max_abs_value > 1e6:
        print(f"WARNING: {name} - 非常に大きな座標値が検出されました: {max_abs_value}")
        return False
    
    return True

def _check_cell_validity(cell, name=""):
    """セルの妥当性をチェックする補助関数"""
    if cell is None:
        print(f"ERROR: {name} - セルデータが空です")
        return False
    
    diagonal = np.diag(cell)
    
    if np.any(diagonal <= 0):
        print(f"ERROR: {name} - セルサイズが0以下です: {diagonal}")
        return False
    
    if np.any(np.isinf(diagonal)) or np.any(np.isnan(diagonal)):
        print(f"ERROR: {name} - セルサイズに無効な値があります: {diagonal}")
        return False
    
    if np.any(diagonal > 1000.0):
        print(f"WARNING: {name} - セルサイズが異常に大きいです: {diagonal}")
        return False
    
    return True

# =============================================
# メイン処理（CSV読み込み版）
# =============================================

print(f"入力CSV: {CSV_FILE_PATH}")
print(f"入力ディレクトリ: {INPUT_DIR}")
print(f"出力ディレクトリ: {OUTPUT_DIR}")
print("="*50)

# ★★★ CSVファイルの読み込み ★★★
try:
    df = pd.read_csv(CSV_FILE_PATH)
    print(f"✅ CSVから {len(df)} 件の計算対象を読み込みました。")
except FileNotFoundError:
    print(f"❌ エラー: CSVファイルが見つかりません: {CSV_FILE_PATH}")
    exit()

# ★★★ CSVの各行を処理 ★★★
for index, row in df.iterrows():
    original_filename = row['original_filename']
    com = row['com']
    atom = row['atom']
    base_name = f"{com}_{atom}"
    
    # ★★★ 入力ファイルパスをCSVから取得 ★★★
    input_file = get_input_file_from_csv(original_filename)
    
    # 入力ファイルの存在チェック
    if not os.path.exists(input_file):
        print(f"\n[SKIP] 入力ファイルが見つかりません: {input_file}")
        continue
    
    # =========================================================================
    # ★★★★★★★★★★★★★★★★★★★ 変更点 1: 条件チェック ★★★★★★★★★★★★★★★★★★★
    # =========================================================================
    # 計算実行前に、Al, O, F以外の原子が100個以上あるかチェック
    try:
        temp_atoms = read(input_file)
        symbols = temp_atoms.get_chemical_symbols()
        # Al, O, F 以外の原子をカウント
        other_atoms_count = sum(1 for s in symbols if s not in ['Al', 'O', 'F'])

        if other_atoms_count < 100:
            print(f"\n[SKIP] {base_name}: Al,O,F以外の原子が{other_atoms_count}個のためスキップ (閾値: 100個)")
            continue # このファイルの処理を中断し、CSVの次の行へ
    except Exception as e:
        print(f"\n[SKIP] ファイル読み込みまたは原子数チェック中にエラー発生: {input_file}. Error: {e}")
        continue # エラーが発生した場合もスキップして次のファイルへ
    # =========================================================================
        
    print(f"\n▶️  Simulations for {base_name} ({original_filename})")
    
    # 各圧着温度に対してシミュレーション
    for comp_temp in compression_temperatures:
        # 各圧着圧力に対してシミュレーション
        for pressure in compression_pressures:
            # =========================================================================
            # ★★★★★★★★★★★★★★ 変更点 2: 堅牢なエラーハンドリング ★★★★★★★★★★★★★★★
            # =========================================================================
            # 各パラメータセットのシミュレーション全体をtry-exceptで囲む
            try:
                compression_file = get_compression_file_path(base_name, pressure, comp_temp)
                
                # ステップ0: 圧着シミュレーション（温度・圧力可変）
                if not check_md_completed(compression_file, 10, 100):
                    print(f"Running compression simulation (P={pressure:.3f}GPa, T={comp_temp:.0f}K) for {base_name}")
                    
                    ase_atoms1 = read(input_file)
                    ase_atoms = ase_atoms1.copy()
                    
                    ase_atoms = rotate_atoms_to_lower(ase_atoms)
                    indices = [atom.index for atom in ase_atoms if atom.position[2] <= 3.0]
                    c = FixAtoms(indices)
                    ase_atoms.set_constraint(c)
                    ase_atoms.set_pbc(True)
                    ase_atoms.wrap()
                    atoms = MatlantisAtoms.from_ase_atoms(ase_atoms)
                    atoms.rotate_atoms_to_upper()
                    t_step = 1.0
                    press_ase = pressure * units.GPa
                    atoms.calc = get_calculator(estimator_fn)
                    system = ASEMDSystem(atoms)
                    system.init_temperature(comp_temp)

                    integrator = NPTIntegrator(
                        timestep=t_step,
                        temperature=comp_temp,
                        pressure=press_ase,
                        ttime=20*units.fs,
                        pfactor=2e6*units.GPa*(units.fs**2),
                        mask=np.array([[0,0,0], [0,0,0], [0,0,1]])
                    )
                    
                    info = PrintCellShape()
                    
                    md = MDFeature(
                        integrator, 
                        n_run=8000, 
                        traj_file_name=compression_file,
                        traj_freq=100, 
                        estimator_fn=estimator_fn
                    )
                    
                    md(system, extensions=[(info, 100)])
                    print(f"Compression simulation (P={pressure:.3f}GPa, T={comp_temp:.0f}K) completed for {base_name}")
                else:
                    print(f"Compression simulation (P={pressure:.3f}GPa, T={comp_temp:.0f}K) already completed for {base_name}")
                
                # 各高温シミュレーション温度に対して処理
                for high_temp in high_temperatures:
                    equil_file = get_equil_file_path(base_name, pressure, comp_temp)
                    high_temp_file = get_high_temp_file_path(base_name, pressure, comp_temp, high_temp)
                    tensile_file = get_tensile_file_path(base_name, pressure, comp_temp, high_temp, tensile_temperature)
                        
                    # ステップ1: 短い平衡化（5 ps）
                    if not check_md_completed(equil_file, 5000, 100):
                        print(f"Running equilibration for {base_name} (P={pressure:.3f}GPa, T={comp_temp:.0f}K)")
                        
                        if os.path.exists(compression_file):
                            ase_atoms = read(compression_file, "-1")
                        else:
                            ase_atoms = read(input_file)
                        
                        ase_atoms = rotate_atoms_to_lower(ase_atoms)
                        indices = [atom.index for atom in ase_atoms if atom.position[2] <= 3.0]
                        c = FixAtoms(indices)
                        ase_atoms.set_constraint(c)
                        cell = ase_atoms.get_cell()
                        
                        ase_atoms.set_cell(cell, scale_atoms=True)
                        ase_atoms.wrap()
                        atoms = MatlantisAtoms.from_ase_atoms(ase_atoms)
                        atoms.rotate_atoms_to_upper()
                        system = ASEMDSystem(atoms)
                        system.init_temperature(300)
                        
                        integrator = LangevinIntegrator(timestep=1.0, temperature=300.0)
                        info = PrintCellShape()
                        
                        md = MDFeature(
                            integrator, 
                            n_run=3000, 
                            traj_file_name=equil_file,
                            traj_freq=100, 
                            estimator_fn=estimator_fn
                        )
                        
                        md(system, extensions=[(info, 1000)])
                        print(f"Equilibration completed for {base_name} (P={pressure:.3f}GPa, T={comp_temp:.0f}K)")
                    else:
                        print(f"Equilibration already completed for {base_name} (P={pressure:.3f}GPa, T={comp_temp:.0f}K)")
                    
                    # ステップ2: 高温シミュレーション（可変温度、10ps）
                    if not check_md_completed(high_temp_file, 10000, 100):
                        print(f"Running high temperature simulation ({high_temp}K) for {base_name} (P={pressure:.3f}GPa, T={comp_temp:.0f}K)")
                        
                        ase_atoms = read(equil_file, "-1")
                        ase_atoms = rotate_atoms_to_lower(ase_atoms)
                        indices = [atom.index for atom in ase_atoms if atom.position[2] <= 2.0]
                        c = FixAtoms(indices)
                        ase_atoms.set_constraint(c)
                        atoms = MatlantisAtoms.from_ase_atoms(ase_atoms)
                        atoms.rotate_atoms_to_upper()
                        system = ASEMDSystem(atoms)
                        system.init_temperature(high_temp)
                        
                        integrator = LangevinIntegrator(timestep=1.0, temperature=float(high_temp))
                        info = PrintCellShape()
                        
                        md = MDFeature(
                            integrator, 
                            n_run=10000, 
                            traj_file_name=high_temp_file,
                            traj_freq=100, 
                            estimator_fn=estimator_fn
                        )
                        
                        md(system, extensions=[(info, 500)])
                        print(f"High temperature simulation ({high_temp}K) completed for {base_name} (P={pressure:.3f}GPa, T={comp_temp:.0f}K)")
                    else:
                        print(f"High temperature simulation ({high_temp}K) already completed for {base_name} (P={pressure:.3f}GPa, T={comp_temp:.0f}K)")
                    
                    # ステップ3: 早期終了機能付き引張シミュレーション（温度300K固定）
                    analysis_file = get_tensile_analysis_file_path(base_name, pressure, comp_temp, high_temp)
                    summary_file = get_tensile_summary_file_path(base_name, pressure, comp_temp, high_temp)
                    
                    # 計算済みチェック
                    if check_md_completed(tensile_file, UNIFIED_CONDITIONS["max_steps"], UNIFIED_CONDITIONS["output_freq"]):
                        print(f"Tensile simulation already completed for {base_name} (P={pressure:.3f}GPa, CT={comp_temp}K, HT={high_temp}K)")
                        continue
                    
                    print(f"\n{'='*80}")
                    print(f"早期終了機能付き引張計算開始: {base_name}")
                    print(f"条件: P={pressure:.3f}GPa, CT={comp_temp}K, HT={high_temp}K")
                    print(f"統一ひずみ速度: {UNIFIED_CONDITIONS['strain_rate']} 1/ps")
                    print(f"{'='*80}")
                    
                    if not os.path.exists(high_temp_file):
                        print(f"❌ ステップ2ファイルが見つかりません: {high_temp_file}")
                        continue
                    
                    print(f"\n📋 Step 1: 構造準備（vacuum設定: {UNIFIED_CONDITIONS['vacuum_top']}Å）")
                    ase_atoms, initial_reference, structure_info = prepare_structure_with_top_vacuum(
                        high_temp_file, 
                        UNIFIED_CONDITIONS["vacuum_top"],
                        UNIFIED_CONDITIONS["fixed_layer_thickness"]
                    )
                    
                    print(f"\n📋 Step 2: MD計算設定（統一条件）")
                    atoms = MatlantisAtoms.from_ase_atoms(ase_atoms)
                    atoms.rotate_atoms_to_upper()
                    system = ASEMDSystem(atoms)
                    system.init_temperature(UNIFIED_CONDITIONS["temperature"])
                    
                    strain_rate = UNIFIED_CONDITIONS["strain_rate"]
                    max_strain = UNIFIED_CONDITIONS["max_strain"]
                    timestep = UNIFIED_CONDITIONS["timestep"]
                    
                    total_steps = int(max_strain / (strain_rate * timestep * 1e-3))
                    total_steps = min(total_steps, UNIFIED_CONDITIONS["max_steps"])
                    
                    if UNIFIED_CONDITIONS["vacuum_top"] > 0:
                        final_material_thickness = structure_info["material_thickness"] * (1 + max_strain)
                        final_cell_length = final_material_thickness + UNIFIED_CONDITIONS["vacuum_top"]
                    else:
                        final_cell_length = initial_reference * (1 + max_strain)
                    
                    latt = np.array(system.ase_atoms.cell)
                    latt[2, 2] = final_cell_length
                    
                    print(f"  統一ひずみ速度: {strain_rate} 1/ps")
                    print(f"  総ステップ数: {total_steps}")
                    print(f"  推定時間: {total_steps * timestep / 1000:.1f} ps")
                    print(f"  ひずみ計算方法: {structure_info['strain_method']}")
                    
                    print(f"\n📋 Step 3: 解析器セットアップ（早期終了無効）")
                    analyzer = ComprehensiveTensileAnalyzer(
                        initial_reference, structure_info, UNIFIED_CONDITIONS, high_temp_file, analysis_file
                    )
                    
                    print(f"\n📋 Step 4: MD計算実行（早期終了無効）")
                    integrator = LangevinIntegrator(
                        timestep=timestep, 
                        temperature=UNIFIED_CONDITIONS["temperature"]
                    )
                    deform = DeformScheduler(latt, total_steps)
                    info = PrintCellShape()
                    
                    md = MDFeature(
                        integrator,
                        n_run=total_steps,
                        traj_file_name=tensile_file,
                        traj_freq=UNIFIED_CONDITIONS["output_freq"],
                        estimator_fn=estimator_fn
                    )
                    
                    extensions = [
                        (info, 100),
                        (deform, 1),
                        (analyzer, 10)
                    ]
                    
                    execution_info = run_md_without_early_termination(md, system, extensions)
                    
                    print(f"\n📋 Step 5: 結果解析")
                    final_results = analyzer.get_final_results()
                    final_results['execution_info'] = execution_info
                    
                    print(f"\n📋 Step 6: 結果出力")
                    save_tensile_results_no_early_termination(
                        final_results, summary_file, base_name, pressure, comp_temp, high_temp, execution_info
                    )
                    
                    print(f"\n✅ 引張計算完了（早期終了無効）: {base_name}")
                    print(f"📊 結果サマリー:")
                    print(f"  ヤング率: {final_results['young_modulus']:.1f} GPa (R² = {final_results['young_r2']:.3f})")
                    print(f"  最大応力: {final_results['max_stress']:.2f} GPa")
                    print(f"  統一ひずみ速度: {final_results['conditions']['strain_rate']} 1/ps")
                    
                    if final_results['fracture_detected']:
                        print(f"  破断検出: Step {final_results['fracture_step']} (記録のみ)")
                    if final_results['complete_separation']:
                        print(f"  完全分離: Step {final_results['separation_step']} (記録のみ)")
                        
                    print(f"All simulations completed successfully for {base_name} (P={pressure:.3f}GPa, CT={comp_temp}K, HT={high_temp}K)")
            
            except Exception as e:
                # エラーが発生した場合、情報を表示して次の計算へ進む
                print(f"\n❌ [ERROR] An error occurred processing {base_name} (P={pressure:.3f}GPa, T={comp_temp:.0f}K).")
                print("   Skipping to the next calculation set...")
                print("   Error details:")
                traceback.print_exc() # 詳細なエラー情報をコンソールに出力
                continue # この pressure, temp の組み合わせをスキップ

print("\n\nAll processing complete.")
print(f"結果は以下のディレクトリに保存されました:")
print(f"- 圧着結果: {COMPRESSION_DIR}")
print(f"- 非圧着結果: {HEAT_DIR}")
print(f"- 応力-歪みデータ: {STRESS_STRAIN_DIR}")
print(f"- 早期終了機能付き引張結果: {TENSILE_DIR}")

print(f"\n🎯 早期終了機能付きシステムの特徴:")
print(f"✅ 統一ひずみ速度 ({UNIFIED_CONDITIONS['strain_rate']} 1/ps) による公平な材料間比較")
print(f"✅ 選択的早期終了による計算効率化")
print(f"✅ 破断検出は記録のみ（過度な早期終了を回避）")
print(f"✅ 材料長ひずみによる実験対応可能な解析")
print(f"✅ 上部vacuum配置による効率的な計算")
print(f"✅ 詳細な結果解析とサマリー出力")

print(f"\n📝 CSVベース処理の特徴:")
print(f"✅ CSVファイルからの自動ファイル名取得")
print(f"✅ ファイル存在チェックによる安全な処理")
print(f"✅ 元のコードの動作を完全に維持")
print(f"✅ 入力ディレクトリとファイル名のみ変更")

入力CSV: /home/jovyan/Kaori/MD/LiB_2/structure/extracted_data_final_with_filenames.csv
入力ディレクトリ: /home/jovyan/Kaori/MD/LiB_2/structure/output/final_interfaces
出力ディレクトリ: /home/jovyan/Kaori/MD/LiB_2/output
✅ CSVから 723 件の計算対象を読み込みました。

▶️  Simulations for Al_100_(1, 0, 0)_NMC111_Co_MntoNi_40_012_(0, 1, 2) (Interface_Al_100_(1, 0, 0)_on_NMC111_Co_MntoNi_40_012_(0, 1, 2)_trimmed_initial.xyz)
Compression simulation (P=0.001GPa, T=300K) already completed for Al_100_(1, 0, 0)_NMC111_Co_MntoNi_40_012_(0, 1, 2)
Equilibration already completed for Al_100_(1, 0, 0)_NMC111_Co_MntoNi_40_012_(0, 1, 2) (P=0.001GPa, T=300K)
High temperature simulation (300K) already completed for Al_100_(1, 0, 0)_NMC111_Co_MntoNi_40_012_(0, 1, 2) (P=0.001GPa, T=300K)
Tensile simulation already completed for Al_100_(1, 0, 0)_NMC111_Co_MntoNi_40_012_(0, 1, 2) (P=0.001GPa, CT=300K, HT=300K)
Compression simulation (P=0.005GPa, T=300K) already completed for Al_100_(1, 0, 0)_NMC111_Co_MntoNi_40_012_(0, 1, 2)
Equilibration alre

The MD trajectory will be saved at /home/jovyan/Kaori/MD/LiB_2/output/compression/Al2O3_10-12_(1, 0, 2)_NMC111_CotoNi_80_001_(0, 0, 1)_P0.001_T300K_compression.traj.
Note: The max disk size of /home/jovyan is about 98G.


Dyn step    0 a 12.00 b 12.00 c 32.70 alpha 90.00 beta 90.00 gamma 90.00 
Dyn step    0 a 12.00 b 12.00 c 32.70 alpha 90.00 beta 90.00 gamma 90.00 


Traceback (most recent call last):
  File "/tmp/ipykernel_33/150896995.py", line 1045, in <module>
    md(system, extensions=[(info, 100)])
  File "/home/jovyan/.py311/lib/python3.11/site-packages/matlantis_features/features/base.py", line 131, in decorated
    res = derived_call(self, *args_list, **kwargs_new)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jovyan/.py311/lib/python3.11/site-packages/matlantis_features/features/md/md.py", line 353, in __call__
    sim.run(n_run_steps=self.n_run)
  File "/home/jovyan/.py311/lib/python3.11/site-packages/matlantis_features/features/md/ase_simulation.py", line 192, in run
    dyn.run(steps=n_run_steps)
  File "/home/jovyan/.py311/lib/python3.11/site-packages/ase/md/npt.py", line 315, in run
    for complete in self.irun(steps):
  File "/home/jovyan/.py311/lib/python3.11/site-packages/ase/md/npt.py", line 299, in irun
    yield from super().irun(steps)
  File "/home/jovyan/.py311/lib/python3.11/site-packages/ase/optimiz


❌ [ERROR] An error occurred processing Al2O3_10-12_(1, 0, 2)_NMC111_CotoNi_80_001_(0, 0, 1) (P=0.001GPa, T=300K).
   Skipping to the next calculation set...
   Error details:
Running compression simulation (P=0.005GPa, T=300K) for Al2O3_10-12_(1, 0, 2)_NMC111_CotoNi_80_001_(0, 0, 1)
原子をZ方向に -4.57Å 移動しました
Z座標範囲: 4.57 → 0.00, 36.77 → 32.20
Dyn step    0 a 12.00 b 12.00 c 32.70 alpha 90.00 beta 90.00 gamma 90.00 
Dyn step    0 a 12.00 b 12.00 c 32.70 alpha 90.00 beta 90.00 gamma 90.00 


...


❌ [ERROR] An error occurred processing Al2O3_10-12_(1, 0, 2)_NMC111_CotoNi_80_100_(1, 0, 0) (P=0.001GPa, T=300K).
   Skipping to the next calculation set...
   Error details:
Running compression simulation (P=0.005GPa, T=300K) for Al2O3_10-12_(1, 0, 2)_NMC111_CotoNi_80_100_(1, 0, 0)
原子をZ方向に -4.56Å 移動しました
Z座標範囲: 4.56 → 0.00, 36.68 → 32.11
Dyn step    0 a 12.00 b 12.00 c 32.61 alpha 90.00 beta 90.00 gamma 90.00 
Dyn step    0 a 12.00 b 12.00 c 32.61 alpha 90.00 beta 90.00 gamma 90.00 


Traceback (most recent call last):
  File "/tmp/ipykernel_33/150896995.py", line 1045, in <module>
    md(system, extensions=[(info, 100)])
  File "/home/jovyan/.py311/lib/python3.11/site-packages/matlantis_features/features/base.py", line 131, in decorated
    res = derived_call(self, *args_list, **kwargs_new)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jovyan/.py311/lib/python3.11/site-packages/matlantis_features/features/md/md.py", line 353, in __call__
    sim.run(n_run_steps=self.n_run)
  File "/home/jovyan/.py311/lib/python3.11/site-packages/matlantis_features/features/md/ase_simulation.py", line 192, in run
    dyn.run(steps=n_run_steps)
  File "/home/jovyan/.py311/lib/python3.11/site-packages/ase/md/npt.py", line 315, in run
    for complete in self.irun(steps):
  File "/home/jovyan/.py311/lib/python3.11/site-packages/ase/md/npt.py", line 299, in irun
    yield from super().irun(steps)
  File "/home/jovyan/.py311/lib/python3.11/site-packages/ase/optimiz


❌ [ERROR] An error occurred processing Al2O3_10-12_(1, 0, 2)_NMC111_CotoNi_80_100_(1, 0, 0) (P=0.005GPa, T=300K).
   Skipping to the next calculation set...
   Error details:
Running compression simulation (P=0.010GPa, T=300K) for Al2O3_10-12_(1, 0, 2)_NMC111_CotoNi_80_100_(1, 0, 0)
原子をZ方向に -4.56Å 移動しました
Z座標範囲: 4.56 → 0.00, 36.68 → 32.11
Dyn step    0 a 12.00 b 12.00 c 32.61 alpha 90.00 beta 90.00 gamma 90.00 
Dyn step    0 a 12.00 b 12.00 c 32.61 alpha 90.00 beta 90.00 gamma 90.00 


Traceback (most recent call last):
  File "/tmp/ipykernel_33/150896995.py", line 1045, in <module>
    md(system, extensions=[(info, 100)])
  File "/home/jovyan/.py311/lib/python3.11/site-packages/matlantis_features/features/base.py", line 131, in decorated
    res = derived_call(self, *args_list, **kwargs_new)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jovyan/.py311/lib/python3.11/site-packages/matlantis_features/features/md/md.py", line 353, in __call__
    sim.run(n_run_steps=self.n_run)
  File "/home/jovyan/.py311/lib/python3.11/site-packages/matlantis_features/features/md/ase_simulation.py", line 192, in run
    dyn.run(steps=n_run_steps)
  File "/home/jovyan/.py311/lib/python3.11/site-packages/ase/md/npt.py", line 315, in run
    for complete in self.irun(steps):
  File "/home/jovyan/.py311/lib/python3.11/site-packages/ase/md/npt.py", line 299, in irun
    yield from super().irun(steps)
  File "/home/jovyan/.py311/lib/python3.11/site-packages/ase/optimiz


❌ [ERROR] An error occurred processing Al2O3_10-12_(1, 0, 2)_NMC111_CotoNi_80_100_(1, 0, 0) (P=0.010GPa, T=300K).
   Skipping to the next calculation set...
   Error details:

[SKIP] Al_100_(1, 0, 0)_NMC111_CotoNi_80: Al,O,F以外の原子が12個のためスキップ (閾値: 100個)

▶️  Simulations for Al2O3_NMC111_pristine_001_(0, 0, 1) (Interface_Al2O3_on_NMC111_pristine_001_(0, 0, 1)_trimmed.xyz)
Compression simulation (P=0.001GPa, T=300K) already completed for Al2O3_NMC111_pristine_001_(0, 0, 1)
Equilibration already completed for Al2O3_NMC111_pristine_001_(0, 0, 1) (P=0.001GPa, T=300K)
High temperature simulation (300K) already completed for Al2O3_NMC111_pristine_001_(0, 0, 1) (P=0.001GPa, T=300K)
Tensile simulation already completed for Al2O3_NMC111_pristine_001_(0, 0, 1) (P=0.001GPa, CT=300K, HT=300K)
Compression simulation (P=0.005GPa, T=300K) already completed for Al2O3_NMC111_pristine_001_(0, 0, 1)
Equilibration already completed for Al2O3_NMC111_pristine_001_(0, 0, 1) (P=0.005GPa, T=300K)
High temperatur

In [ ]:
--- STEP 1: Trajectoryファイルの分析と整理を開始 ---
🔍 '/home/jovyan/Kaori/MD/LiB_2/structure/output/final_interfaces' から 343 個の .traj ファイルを発見。
  -> 🔄 変換中: Interface_AlF3_on_NMC111_CotoNi_50_100_(1, 0, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_AlF3_on_NMC111_CotoNi_50_100_(1, 0, 0)_trimmed.traj
  -> 🔄 変換中: Interface_Al_bulk_on_NMC111_CotoNi_80_001_(0, 0, 1)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_bulk_on_NMC111_CotoNi_80_001_(0, 0, 1)_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al2O3_on_NMC111_CotoNi_50_012_(0, 1, 2)_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_pristine_001_(0, 0, 1)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_pristine_001_(0, 0, 1)_trimmed.traj
  -> 🔄 変換中: Interface_Al_100_(1, 0, 0)_on_NMC111_CotoNi_50_110_(1, 1, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_100_(1, 0, 0)_on_NMC111_CotoNi_50_110_(1, 1, 0)_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al_111_(1, 1, 1)_on_NMC111_selective_Co_removal_110_(1, 1, 0)_trimmed.traj
  -> 🔄 変換中: Interface_AlF3_on_NMC111_extreme_Ni_rich_010_(0, 1, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_AlF3_on_NMC111_extreme_Ni_rich_010_(0, 1, 0)_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al2O3_on_NMC111_extreme_Ni_rich_001_(0, 0, 1)_trimmed.traj
  -> 🔄 変換中: Interface_Al_100_(1, 0, 0)_on_NMC111_thermal_Co_Mn_loss_111_(1, 1, 1)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_100_(1, 0, 0)_on_NMC111_thermal_Co_Mn_loss_111_(1, 1, 1)_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_pristine_104_(1, 0, 4)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_pristine_104_(1, 0, 4)_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_on_NMC111_CotoNi_50_104_(1, 0, 4)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al2O3_on_NMC111_CotoNi_50_104_(1, 0, 4)_trimmed.traj
  -> 🔄 変換中: Interface_Al_bulk_on_NMC111_CotoNi_80_100_(1, 0, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_bulk_on_NMC111_CotoNi_80_100_(1, 0, 0)_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al2O3_2_on_NMC111_selective_Co_removal_110_(1, 1, 0)_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al_111_(1, 1, 1)_on_NMC111_pristine_100_(1, 0, 0)_trimmed.traj
  -> 🔄 変換中: Interface_Al_100_(1, 0, 0)_on_NMC111_Co_MntoNi_40_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_100_(1, 0, 0)_on_NMC111_Co_MntoNi_40_trimmed.traj
  -> 🔄 変換中: Interface_Al_110_(1, 1, 0)_on_NMC111_thermal_Co_Mn_loss_111_(1, 1, 1)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_110_(1, 1, 0)_on_NMC111_thermal_Co_Mn_loss_111_(1, 1, 1)_trimmed.traj
  -> 🔄 変換中: Interface_Al_bulk_on_NMC111_extreme_Ni_rich_104_(1, 0, 4)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_bulk_on_NMC111_extreme_Ni_rich_104_(1, 0, 4)_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_on_NMC111_Co_MntoNi_40_012_(0, 1, 2)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al2O3_on_NMC111_Co_MntoNi_40_012_(0, 1, 2)_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_thermal_Co_Mn_loss_111_(1, 1, 1)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_thermal_Co_Mn_loss_111_(1, 1, 1)_trimmed.traj
  -> 🔄 変換中: Interface_Al_100_(1, 0, 0)_on_NMC111_CotoNi_80_104_(1, 0, 4)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_100_(1, 0, 0)_on_NMC111_CotoNi_80_104_(1, 0, 4)_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_CotoNi_50_110_(1, 1, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_CotoNi_50_110_(1, 1, 0)_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_CotoNi_50_012_(0, 1, 2)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_CotoNi_50_012_(0, 1, 2)_trimmed.traj
  -> 🔄 変換中: Interface_Al_100_(1, 0, 0)_on_NMC111_extreme_Ni_rich_010_(0, 1, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_100_(1, 0, 0)_on_NMC111_extreme_Ni_rich_010_(0, 1, 0)_trimmed.traj
  -> 🔄 変換中: Interface_Al_bulk_on_NMC111_extreme_Ni_rich_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_bulk_on_NMC111_extreme_Ni_rich_trimmed.traj
  -> 🔄 変換中: Interface_Al_100_(1, 0, 0)_on_NMC111_Co_MntoNi_40_001_(0, 0, 1)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_100_(1, 0, 0)_on_NMC111_Co_MntoNi_40_001_(0, 0, 1)_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_on_NMC111_CotoNi_80_010_(0, 1, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al2O3_on_NMC111_CotoNi_80_010_(0, 1, 0)_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_on_NMC111_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al2O3_on_NMC111_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al_111_(1, 1, 1)_on_NMC111_thermal_Co_Mn_loss_trimmed.traj
  -> 🔄 変換中: Interface_AlF3_on_NMC111_pristine_010_(0, 1, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_AlF3_on_NMC111_pristine_010_(0, 1, 0)_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_CotoNi_80_001_(0, 0, 1)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_CotoNi_80_001_(0, 0, 1)_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_on_NMC111_extreme_Ni_rich_104_(1, 0, 4)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al2O3_on_NMC111_extreme_Ni_rich_104_(1, 0, 4)_trimmed.traj
  -> 🔄 変換中: Interface_Al_110_(1, 1, 0)_on_NMC111_selective_Co_removal_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_110_(1, 1, 0)_on_NMC111_selective_Co_removal_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al2O3_2_on_NMC111_pristine_104_(1, 0, 4)_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_selective_Co_removal_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_selective_Co_removal_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al2O3_2_on_NMC111_thermal_Co_Mn_loss_trimmed.traj
  -> 🔄 変換中: Interface_AlF3_on_NMC111_CotoNi_80_001_(0, 0, 1)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_AlF3_on_NMC111_CotoNi_80_001_(0, 0, 1)_trimmed.traj
  -> 🔄 変換中: Interface_AlF3_on_NMC111_CotoNi_80_100_(1, 0, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_AlF3_on_NMC111_CotoNi_80_100_(1, 0, 0)_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al_111_(1, 1, 1)_on_NMC111_Co_MntoNi_40_104_(1, 0, 4)_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al_111_(1, 1, 1)_on_NMC111_CotoNi_80_104_(1, 0, 4)_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_CotoNi_80_010_(0, 1, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_CotoNi_80_010_(0, 1, 0)_trimmed.traj
  -> 🔄 変換中: Interface_Al_110_(1, 1, 0)_on_NMC111_extreme_Ni_rich_110_(1, 1, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_110_(1, 1, 0)_on_NMC111_extreme_Ni_rich_110_(1, 1, 0)_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_CotoNi_50_010_(0, 1, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_CotoNi_50_010_(0, 1, 0)_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_on_NMC111_thermal_Co_Mn_loss_111_(1, 1, 1)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al2O3_on_NMC111_thermal_Co_Mn_loss_111_(1, 1, 1)_trimmed.traj
  -> 🔄 変換中: Interface_AlF3_on_NMC111_pristine_110_(1, 1, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_AlF3_on_NMC111_pristine_110_(1, 1, 0)_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al_111_(1, 1, 1)_on_NMC111_CotoNi_50_001_(0, 0, 1)_trimmed.traj
  -> 🔄 変換中: Interface_AlF3_on_NMC111_thermal_Co_Mn_loss_100_(1, 0, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_AlF3_on_NMC111_thermal_Co_Mn_loss_100_(1, 0, 0)_trimmed.traj
  -> 🔄 変換中: Interface_Al_100_(1, 0, 0)_on_NMC111_extreme_Ni_rich_104_(1, 0, 4)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_100_(1, 0, 0)_on_NMC111_extreme_Ni_rich_104_(1, 0, 4)_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_CotoNi_80_100_(1, 0, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_CotoNi_80_100_(1, 0, 0)_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_CotoNi_50_001_(0, 0, 1)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_CotoNi_50_001_(0, 0, 1)_trimmed.traj
  -> 🔄 変換中: Interface_Al_100_(1, 0, 0)_on_NMC111_pristine_001_(0, 0, 1)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_100_(1, 0, 0)_on_NMC111_pristine_001_(0, 0, 1)_trimmed.traj
  -> 🔄 変換中: Interface_Al_100_(1, 0, 0)_on_NMC111_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_100_(1, 0, 0)_on_NMC111_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_extreme_Ni_rich_012_(0, 1, 2)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_extreme_Ni_rich_012_(0, 1, 2)_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_on_NMC111_extreme_Ni_rich_100_(1, 0, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al2O3_on_NMC111_extreme_Ni_rich_100_(1, 0, 0)_trimmed.traj
  -> 🔄 変換中: Interface_Al_100_(1, 0, 0)_on_NMC111_selective_Co_removal_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_100_(1, 0, 0)_on_NMC111_selective_Co_removal_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_extreme_Ni_rich_100_(1, 0, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_extreme_Ni_rich_100_(1, 0, 0)_trimmed.traj
  -> 🔄 変換中: Interface_Al_bulk_on_NMC111_selective_Co_removal_110_(1, 1, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_bulk_on_NMC111_selective_Co_removal_110_(1, 1, 0)_trimmed.traj
  -> 🔄 変換中: Interface_AlF3_on_NMC111_thermal_Co_Mn_loss_110_(1, 1, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_AlF3_on_NMC111_thermal_Co_Mn_loss_110_(1, 1, 0)_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al2O3_2_on_NMC111_Co_MntoNi_40_010_(0, 1, 0)_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al2O3_2_on_NMC111_extreme_Ni_rich_110_(1, 1, 0)_trimmed.traj
  -> 🔄 変換中: Interface_Al_bulk_on_NMC111_CotoNi_50_100_(1, 0, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_bulk_on_NMC111_CotoNi_50_100_(1, 0, 0)_trimmed.traj
  -> 🔄 変換中: Interface_Al_100_(1, 0, 0)_on_NMC111_Co_MntoNi_40_100_(1, 0, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_100_(1, 0, 0)_on_NMC111_Co_MntoNi_40_100_(1, 0, 0)_trimmed.traj
  -> 🔄 変換中: Interface_AlF3_on_NMC111_pristine_104_(1, 0, 4)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_AlF3_on_NMC111_pristine_104_(1, 0, 4)_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_Co_MntoNi_40_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_Co_MntoNi_40_trimmed.traj
  -> 🔄 変換中: Interface_Al_bulk_on_NMC111_thermal_Co_Mn_loss_111_(1, 1, 1)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_bulk_on_NMC111_thermal_Co_Mn_loss_111_(1, 1, 1)_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_extreme_Ni_rich_104_(1, 0, 4)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_extreme_Ni_rich_104_(1, 0, 4)_trimmed.traj
  -> 🔄 変換中: Interface_AlF3_on_NMC111_CotoNi_80_110_(1, 1, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_AlF3_on_NMC111_CotoNi_80_110_(1, 1, 0)_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_selective_Co_removal_110_(1, 1, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_selective_Co_removal_110_(1, 1, 0)_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al2O3_on_NMC111_CotoNi_80_001_(0, 0, 1)_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al2O3_2_on_NMC111_pristine_012_(0, 1, 2)_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al2O3_2_on_NMC111_extreme_Ni_rich_100_(1, 0, 0)_trimmed.traj
  -> 🔄 変換中: Interface_AlF3_on_NMC111_Co_MntoNi_40_104_(1, 0, 4)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_AlF3_on_NMC111_Co_MntoNi_40_104_(1, 0, 4)_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_on_NMC111_Co_MntoNi_40_104_(1, 0, 4)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al2O3_on_NMC111_Co_MntoNi_40_104_(1, 0, 4)_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al2O3_2_on_NMC111_CotoNi_50_012_(0, 1, 2)_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_on_NMC111_CotoNi_80_100_(1, 0, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al2O3_on_NMC111_CotoNi_80_100_(1, 0, 0)_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al_111_(1, 1, 1)_on_NMC111_CotoNi_80_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al2O3_2_on_NMC111_CotoNi_50_010_(0, 1, 0)_trimmed.traj
  -> 🔄 変換中: Interface_Al_bulk_on_NMC111_CotoNi_80_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_bulk_on_NMC111_CotoNi_80_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_CotoNi_80_110_(1, 1, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_CotoNi_80_110_(1, 1, 0)_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_trimmed.traj
  -> 🔄 変換中: Interface_Al_bulk_on_NMC111_extreme_Ni_rich_110_(1, 1, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_bulk_on_NMC111_extreme_Ni_rich_110_(1, 1, 0)_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al2O3_on_NMC111_selective_Co_removal_111_(1, 1, 1)_trimmed.traj
  -> 🔄 変換中: Interface_Al_100_(1, 0, 0)_on_NMC111_pristine_012_(0, 1, 2)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_100_(1, 0, 0)_on_NMC111_pristine_012_(0, 1, 2)_trimmed.traj
  -> 🔄 変換中: Interface_AlF3_on_NMC111_CotoNi_80_104_(1, 0, 4)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_AlF3_on_NMC111_CotoNi_80_104_(1, 0, 4)_trimmed.traj
  -> 🔄 変換中: Interface_Al_100_(1, 0, 0)_on_NMC111_CotoNi_80_110_(1, 1, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_100_(1, 0, 0)_on_NMC111_CotoNi_80_110_(1, 1, 0)_trimmed.traj
  -> 🔄 変換中: Interface_Al_110_(1, 1, 0)_on_NMC111_CotoNi_80_104_(1, 0, 4)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_110_(1, 1, 0)_on_NMC111_CotoNi_80_104_(1, 0, 4)_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al_111_(1, 1, 1)_on_NMC111_extreme_Ni_rich_110_(1, 1, 0)_trimmed.traj
  -> 🔄 変換中: Interface_Al_110_(1, 1, 0)_on_NMC111_selective_Co_removal_110_(1, 1, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_110_(1, 1, 0)_on_NMC111_selective_Co_removal_110_(1, 1, 0)_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al_111_(1, 1, 1)_on_NMC111_extreme_Ni_rich_100_(1, 0, 0)_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al_111_(1, 1, 1)_on_NMC111_CotoNi_50_104_(1, 0, 4)_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al2O3_2_on_NMC111_extreme_Ni_rich_trimmed.traj
  -> 🔄 変換中: Interface_Al_bulk_on_NMC111_pristine_100_(1, 0, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_bulk_on_NMC111_pristine_100_(1, 0, 0)_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_selective_Co_removal_111_(1, 1, 1)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_selective_Co_removal_111_(1, 1, 1)_trimmed.traj
  -> 🔄 変換中: Interface_AlF3_on_NMC111_pristine_100_(1, 0, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_AlF3_on_NMC111_pristine_100_(1, 0, 0)_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_pristine_110_(1, 1, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_pristine_110_(1, 1, 0)_trimmed.traj
  -> 🔄 変換中: Interface_Al_100_(1, 0, 0)_on_NMC111_extreme_Ni_rich_012_(0, 1, 2)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_100_(1, 0, 0)_on_NMC111_extreme_Ni_rich_012_(0, 1, 2)_trimmed.traj
  -> 🔄 変換中: Interface_Al_bulk_on_NMC111_Co_MntoNi_40_100_(1, 0, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_bulk_on_NMC111_Co_MntoNi_40_100_(1, 0, 0)_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_on_NMC111_Co_MntoNi_40_110_(1, 1, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al2O3_on_NMC111_Co_MntoNi_40_110_(1, 1, 0)_trimmed.traj
  -> 🔄 変換中: Interface_Al_110_(1, 1, 0)_on_NMC111_Co_MntoNi_40_110_(1, 1, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_110_(1, 1, 0)_on_NMC111_Co_MntoNi_40_110_(1, 1, 0)_trimmed.traj
  -> 🔄 変換中: Interface_AlF3_on_NMC111_Co_MntoNi_40_100_(1, 0, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_AlF3_on_NMC111_Co_MntoNi_40_100_(1, 0, 0)_trimmed.traj
  -> 🔄 変換中: Interface_Al_100_(1, 0, 0)_on_NMC111_CotoNi_50_100_(1, 0, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_100_(1, 0, 0)_on_NMC111_CotoNi_50_100_(1, 0, 0)_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al_111_(1, 1, 1)_on_NMC111_extreme_Ni_rich_010_(0, 1, 0)_trimmed.traj
  -> 🔄 変換中: Interface_Al_110_(1, 1, 0)_on_NMC111_thermal_Co_Mn_loss_100_(1, 0, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_110_(1, 1, 0)_on_NMC111_thermal_Co_Mn_loss_100_(1, 0, 0)_trimmed.traj
  -> 🔄 変換中: Interface_Al_110_(1, 1, 0)_on_NMC111_extreme_Ni_rich_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_110_(1, 1, 0)_on_NMC111_extreme_Ni_rich_trimmed.traj
  -> 🔄 変換中: Interface_AlF3_on_NMC111_Co_MntoNi_40_012_(0, 1, 2)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_AlF3_on_NMC111_Co_MntoNi_40_012_(0, 1, 2)_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al_111_(1, 1, 1)_on_NMC111_pristine_010_(0, 1, 0)_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al2O3_2_on_NMC111_CotoNi_80_012_(0, 1, 2)_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al2O3_2_on_NMC111_CotoNi_80_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_selective_Co_removal_100_(1, 0, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_selective_Co_removal_100_(1, 0, 0)_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_Co_MntoNi_40_110_(1, 1, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_Co_MntoNi_40_110_(1, 1, 0)_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al_111_(1, 1, 1)_on_NMC111_pristine_001_(0, 0, 1)_trimmed.traj
  -> 🔄 変換中: Interface_Al_bulk_on_NMC111_thermal_Co_Mn_loss_100_(1, 0, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_bulk_on_NMC111_thermal_Co_Mn_loss_100_(1, 0, 0)_trimmed.traj
  -> 🔄 変換中: Interface_AlF3_on_NMC111_Co_MntoNi_40_001_(0, 0, 1)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_AlF3_on_NMC111_Co_MntoNi_40_001_(0, 0, 1)_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al2O3_2_on_NMC111_Co_MntoNi_40_012_(0, 1, 2)_trimmed.traj
  -> 🔄 変換中: Interface_Al_bulk_on_NMC111_CotoNi_80_012_(0, 1, 2)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_bulk_on_NMC111_CotoNi_80_012_(0, 1, 2)_trimmed.traj
  -> 🔄 変換中: Interface_AlF3_on_NMC111_extreme_Ni_rich_110_(1, 1, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_AlF3_on_NMC111_extreme_Ni_rich_110_(1, 1, 0)_trimmed.traj
  -> 🔄 変換中: Interface_Al_bulk_on_NMC111_Co_MntoNi_40_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_bulk_on_NMC111_Co_MntoNi_40_trimmed.traj
  -> 🔄 変換中: Interface_Al_100_(1, 0, 0)_on_NMC111_pristine_110_(1, 1, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_100_(1, 0, 0)_on_NMC111_pristine_110_(1, 1, 0)_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al2O3_2_on_NMC111_pristine_100_(1, 0, 0)_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al_111_(1, 1, 1)_on_NMC111_CotoNi_50_012_(0, 1, 2)_trimmed.traj
  -> 🔄 変換中: Interface_AlF3_on_NMC111_CotoNi_50_012_(0, 1, 2)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_AlF3_on_NMC111_CotoNi_50_012_(0, 1, 2)_trimmed.traj
  -> 🔄 変換中: Interface_Al_bulk_on_NMC111_pristine_001_(0, 0, 1)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_bulk_on_NMC111_pristine_001_(0, 0, 1)_trimmed.traj
  -> 🔄 変換中: Interface_Al_bulk_on_NMC111_Co_MntoNi_40_110_(1, 1, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_bulk_on_NMC111_Co_MntoNi_40_110_(1, 1, 0)_trimmed.traj
  -> 🔄 変換中: Interface_Al_100_(1, 0, 0)_on_NMC111_pristine_010_(0, 1, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_100_(1, 0, 0)_on_NMC111_pristine_010_(0, 1, 0)_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al2O3_on_NMC111_Co_MntoNi_40_010_(0, 1, 0)_trimmed.traj
  -> 🔄 変換中: Interface_AlF3_on_NMC111_extreme_Ni_rich_104_(1, 0, 4)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_AlF3_on_NMC111_extreme_Ni_rich_104_(1, 0, 4)_trimmed.traj
  -> 🔄 変換中: Interface_AlF3_on_NMC111_thermal_Co_Mn_loss_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_AlF3_on_NMC111_thermal_Co_Mn_loss_trimmed.traj
  -> 🔄 変換中: Interface_Al_100_(1, 0, 0)_on_NMC111_CotoNi_80_012_(0, 1, 2)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_100_(1, 0, 0)_on_NMC111_CotoNi_80_012_(0, 1, 2)_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al_111_(1, 1, 1)_on_NMC111_Co_MntoNi_40_100_(1, 0, 0)_trimmed.traj
  -> 🔄 変換中: Interface_Al_110_(1, 1, 0)_on_NMC111_CotoNi_50_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_110_(1, 1, 0)_on_NMC111_CotoNi_50_trimmed.traj
  -> 🔄 変換中: Interface_Al_bulk_on_NMC111_thermal_Co_Mn_loss_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_bulk_on_NMC111_thermal_Co_Mn_loss_trimmed.traj
  -> 🔄 変換中: Interface_Al_110_(1, 1, 0)_on_NMC111_thermal_Co_Mn_loss_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_110_(1, 1, 0)_on_NMC111_thermal_Co_Mn_loss_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_CotoNi_50_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_CotoNi_50_trimmed.traj
  -> 🔄 変換中: Interface_Al_100_(1, 0, 0)_on_NMC111_thermal_Co_Mn_loss_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_100_(1, 0, 0)_on_NMC111_thermal_Co_Mn_loss_trimmed.traj
  -> 🔄 変換中: Interface_Al_bulk_on_NMC111_CotoNi_50_010_(0, 1, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_bulk_on_NMC111_CotoNi_50_010_(0, 1, 0)_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al_111_(1, 1, 1)_on_NMC111_CotoNi_50_trimmed.traj
  -> 🔄 変換中: Interface_Al_bulk_on_NMC111_CotoNi_50_012_(0, 1, 2)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_bulk_on_NMC111_CotoNi_50_012_(0, 1, 2)_trimmed.traj
  -> 🔄 変換中: Interface_Al_110_(1, 1, 0)_on_NMC111_CotoNi_50_010_(0, 1, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_110_(1, 1, 0)_on_NMC111_CotoNi_50_010_(0, 1, 0)_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al2O3_2_on_NMC111_CotoNi_50_001_(0, 0, 1)_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al_111_(1, 1, 1)_on_NMC111_CotoNi_80_110_(1, 1, 0)_trimmed.traj
  -> 🔄 変換中: Interface_Al_110_(1, 1, 0)_on_NMC111_extreme_Ni_rich_010_(0, 1, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_110_(1, 1, 0)_on_NMC111_extreme_Ni_rich_010_(0, 1, 0)_trimmed.traj
  -> 🔄 変換中: Interface_Al_bulk_on_NMC111_pristine_010_(0, 1, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_bulk_on_NMC111_pristine_010_(0, 1, 0)_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_on_NMC111_CotoNi_50_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al2O3_on_NMC111_CotoNi_50_trimmed.traj
  -> 🔄 変換中: Interface_Al_110_(1, 1, 0)_on_NMC111_extreme_Ni_rich_104_(1, 0, 4)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_110_(1, 1, 0)_on_NMC111_extreme_Ni_rich_104_(1, 0, 4)_trimmed.traj
  -> 🔄 変換中: Interface_Al_100_(1, 0, 0)_on_NMC111_CotoNi_50_001_(0, 0, 1)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_100_(1, 0, 0)_on_NMC111_CotoNi_50_001_(0, 0, 1)_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al_111_(1, 1, 1)_on_NMC111_Co_MntoNi_40_001_(0, 0, 1)_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_on_NMC111_pristine_010_(0, 1, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al2O3_on_NMC111_pristine_010_(0, 1, 0)_trimmed.traj
  -> 🔄 変換中: Interface_Al_110_(1, 1, 0)_on_NMC111_CotoNi_50_104_(1, 0, 4)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_110_(1, 1, 0)_on_NMC111_CotoNi_50_104_(1, 0, 4)_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_on_NMC111_selective_Co_removal_100_(1, 0, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al2O3_on_NMC111_selective_Co_removal_100_(1, 0, 0)_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al2O3_2_on_NMC111_selective_Co_removal_111_(1, 1, 1)_trimmed.traj
  -> 🔄 変換中: Interface_Al_110_(1, 1, 0)_on_NMC111_extreme_Ni_rich_100_(1, 0, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_110_(1, 1, 0)_on_NMC111_extreme_Ni_rich_100_(1, 0, 0)_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al2O3_2_on_NMC111_thermal_Co_Mn_loss_100_(1, 0, 0)_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_on_NMC111_pristine_110_(1, 1, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al2O3_on_NMC111_pristine_110_(1, 1, 0)_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_on_NMC111_CotoNi_80_012_(0, 1, 2)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al2O3_on_NMC111_CotoNi_80_012_(0, 1, 2)_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_extreme_Ni_rich_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_extreme_Ni_rich_trimmed.traj
  -> 🔄 変換中: Interface_AlF3_on_NMC111_CotoNi_50_010_(0, 1, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_AlF3_on_NMC111_CotoNi_50_010_(0, 1, 0)_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al2O3_2_on_NMC111_CotoNi_80_104_(1, 0, 4)_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_on_NMC111_pristine_001_(0, 0, 1)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al2O3_on_NMC111_pristine_001_(0, 0, 1)_trimmed.traj
  -> 🔄 変換中: Interface_AlF3_on_NMC111_CotoNi_50_104_(1, 0, 4)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_AlF3_on_NMC111_CotoNi_50_104_(1, 0, 4)_trimmed.traj
  -> 🔄 変換中: Interface_Al_110_(1, 1, 0)_on_NMC111_Co_MntoNi_40_001_(0, 0, 1)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_110_(1, 1, 0)_on_NMC111_Co_MntoNi_40_001_(0, 0, 1)_trimmed.traj
  -> 🔄 変換中: Interface_Al_bulk_on_NMC111_selective_Co_removal_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_bulk_on_NMC111_selective_Co_removal_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_on_NMC111_thermal_Co_Mn_loss_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al2O3_on_NMC111_thermal_Co_Mn_loss_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_on_NMC111_extreme_Ni_rich_010_(0, 1, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al2O3_on_NMC111_extreme_Ni_rich_010_(0, 1, 0)_trimmed.traj
  -> 🔄 変換中: Interface_Al_100_(1, 0, 0)_on_NMC111_Co_MntoNi_40_010_(0, 1, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_100_(1, 0, 0)_on_NMC111_Co_MntoNi_40_010_(0, 1, 0)_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al2O3_2_on_NMC111_CotoNi_50_trimmed.traj
  -> 🔄 変換中: Interface_Al_bulk_on_NMC111_CotoNi_50_110_(1, 1, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_bulk_on_NMC111_CotoNi_50_110_(1, 1, 0)_trimmed.traj
  -> 🔄 変換中: Interface_Al_bulk_on_NMC111_Co_MntoNi_40_001_(0, 0, 1)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_bulk_on_NMC111_Co_MntoNi_40_001_(0, 0, 1)_trimmed.traj
  -> 🔄 変換中: Interface_Al_110_(1, 1, 0)_on_NMC111_Co_MntoNi_40_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_110_(1, 1, 0)_on_NMC111_Co_MntoNi_40_trimmed.traj
  -> 🔄 変換中: Interface_AlF3_on_NMC111_CotoNi_50_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_AlF3_on_NMC111_CotoNi_50_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_Co_MntoNi_40_104_(1, 0, 4)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_Co_MntoNi_40_104_(1, 0, 4)_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al_111_(1, 1, 1)_on_NMC111_extreme_Ni_rich_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al2O3_2_on_NMC111_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al2O3_2_on_NMC111_selective_Co_removal_100_(1, 0, 0)_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_thermal_Co_Mn_loss_110_(1, 1, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_thermal_Co_Mn_loss_110_(1, 1, 0)_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_CotoNi_80_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_CotoNi_80_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al_111_(1, 1, 1)_on_NMC111_selective_Co_removal_111_(1, 1, 1)_trimmed.traj
  -> 🔄 変換中: Interface_AlF3_on_NMC111_selective_Co_removal_100_(1, 0, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_AlF3_on_NMC111_selective_Co_removal_100_(1, 0, 0)_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al_111_(1, 1, 1)_on_NMC111_extreme_Ni_rich_012_(0, 1, 2)_trimmed.traj
  -> 🔄 変換中: Interface_Al_100_(1, 0, 0)_on_NMC111_extreme_Ni_rich_100_(1, 0, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_100_(1, 0, 0)_on_NMC111_extreme_Ni_rich_100_(1, 0, 0)_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_on_NMC111_CotoNi_50_110_(1, 1, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al2O3_on_NMC111_CotoNi_50_110_(1, 1, 0)_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al2O3_on_NMC111_CotoNi_80_110_(1, 1, 0)_trimmed.traj
  -> 🔄 変換中: Interface_Al_100_(1, 0, 0)_on_NMC111_extreme_Ni_rich_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_100_(1, 0, 0)_on_NMC111_extreme_Ni_rich_trimmed.traj
  -> 🔄 変換中: Interface_Al_bulk_on_NMC111_CotoNi_50_104_(1, 0, 4)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_bulk_on_NMC111_CotoNi_50_104_(1, 0, 4)_trimmed.traj
  -> 🔄 変換中: Interface_AlF3_on_NMC111_pristine_001_(0, 0, 1)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_AlF3_on_NMC111_pristine_001_(0, 0, 1)_trimmed.traj
  -> 🔄 変換中: Interface_Al_bulk_on_NMC111_Co_MntoNi_40_012_(0, 1, 2)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_bulk_on_NMC111_Co_MntoNi_40_012_(0, 1, 2)_trimmed.traj
  -> 🔄 変換中: Interface_AlF3_on_NMC111_extreme_Ni_rich_100_(1, 0, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_AlF3_on_NMC111_extreme_Ni_rich_100_(1, 0, 0)_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al2O3_2_on_NMC111_pristine_001_(0, 0, 1)_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_CotoNi_80_012_(0, 1, 2)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_CotoNi_80_012_(0, 1, 2)_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al2O3_2_on_NMC111_pristine_010_(0, 1, 0)_trimmed.traj
  -> 🔄 変換中: Interface_Al_bulk_on_NMC111_extreme_Ni_rich_010_(0, 1, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_bulk_on_NMC111_extreme_Ni_rich_010_(0, 1, 0)_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al_111_(1, 1, 1)_on_NMC111_extreme_Ni_rich_001_(0, 0, 1)_trimmed.traj
  -> 🔄 変換中: Interface_Al_100_(1, 0, 0)_on_NMC111_extreme_Ni_rich_001_(0, 0, 1)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_100_(1, 0, 0)_on_NMC111_extreme_Ni_rich_001_(0, 0, 1)_trimmed.traj
  -> 🔄 変換中: Interface_Al_100_(1, 0, 0)_on_NMC111_extreme_Ni_rich_110_(1, 1, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_100_(1, 0, 0)_on_NMC111_extreme_Ni_rich_110_(1, 1, 0)_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_on_NMC111_thermal_Co_Mn_loss_100_(1, 0, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al2O3_on_NMC111_thermal_Co_Mn_loss_100_(1, 0, 0)_trimmed.traj
  -> 🔄 変換中: Interface_Al_100_(1, 0, 0)_on_NMC111_Co_MntoNi_40_104_(1, 0, 4)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_100_(1, 0, 0)_on_NMC111_Co_MntoNi_40_104_(1, 0, 4)_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al_111_(1, 1, 1)_on_NMC111_Co_MntoNi_40_trimmed.traj
  -> 🔄 変換中: Interface_Al_bulk_on_NMC111_selective_Co_removal_111_(1, 1, 1)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_bulk_on_NMC111_selective_Co_removal_111_(1, 1, 1)_trimmed.traj
  -> 🔄 変換中: Interface_AlF3_on_NMC111_CotoNi_80_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_AlF3_on_NMC111_CotoNi_80_trimmed.traj
  -> 🔄 変換中: Interface_Al_bulk_on_NMC111_thermal_Co_Mn_loss_110_(1, 1, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_bulk_on_NMC111_thermal_Co_Mn_loss_110_(1, 1, 0)_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_CotoNi_50_100_(1, 0, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_CotoNi_50_100_(1, 0, 0)_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al_111_(1, 1, 1)_on_NMC111_CotoNi_80_012_(0, 1, 2)_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_on_NMC111_Co_MntoNi_40_001_(0, 0, 1)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al2O3_on_NMC111_Co_MntoNi_40_001_(0, 0, 1)_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_pristine_012_(0, 1, 2)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_pristine_012_(0, 1, 2)_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al_111_(1, 1, 1)_on_NMC111_selective_Co_removal_100_(1, 0, 0)_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al2O3_2_on_NMC111_CotoNi_80_100_(1, 0, 0)_trimmed.traj
  -> 🔄 変換中: Interface_Al_bulk_on_NMC111_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_bulk_on_NMC111_trimmed.traj
  -> 🔄 変換中: Interface_Al_bulk_on_NMC111_CotoNi_50_001_(0, 0, 1)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al_bulk_on_NMC111_CotoNi_50_001_(0, 0, 1)_trimmed.traj
  -> 🗑️  0KBファイルを削除します: Interface_Al_111_(1, 1, 1)_on_NMC111_pristine_104_(1, 0, 4)_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_Co_MntoNi_40_010_(0, 1, 0)_trimmed.traj
  -> ✔️ 変換成功、元のファイルを削除しました: Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_Co_MntoNi_40_010_(0, 1, 0)_trimmed.traj
  -> 🔄 変換中: Interface_Al2O3_on_NMC111_extreme_Ni_rich_110_(1, 1, 0)_trimmed.traj

In [24]:
from ase.io import read, Trajectory, write
from ase.constraints import FixAtoms
import numpy as np
import os
import time
from datetime import datetime
from sklearn.linear_model import LinearRegression
from matlantis_features.features.md import ASEMDSystem, LangevinIntegrator, MDFeature, MDExtensionBase, NPTIntegrator
from matlantis_features.features.md.md_extensions import DeformScheduler
from pfp_api_client.pfp.estimator import Estimator, EstimatorCalcMode
from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
from matlantis_features.utils.calculators import get_calculator, pfp_estimator_fn
from matlantis_features.atoms import MatlantisAtoms
import pathlib
from ase import units
import pandas as pd # ★ CSVファイルを扱うためにpandasをインポート

# =============================================
# エネルギー計算用の関数を定義
# =============================================
estimator_fn = pfp_estimator_fn(model_version='v7.0.0', calc_mode='crystal_u0_plus_d3')

# =============================================
# シミュレーションパラメータ設定
# =============================================
# ★ CSVから読み込むため、以下のリストは不要になります
# atoms_list = ["Mn3O4" ]
# compounds = [ "Al2O3",]

# パラメータの設定 - カスタマイズ可能
compression_temperatures = [300]  # 圧着シミュレーション温度（K）
compression_pressures = [0.001, 0.005, 0.01]  # 圧着圧力（GPa）
high_temperatures = [300]  # 高温シミュレーション温度（K）
tensile_temperature = 300.0  # 引張シミュレーション温度（K）

# =============================================
# ファイル・ディレクトリパス設定
# =============================================
BASE_DIR = "/home/jovyan/Kaori/MD/LiB_2/"
INPUT_DIR = os.path.join(BASE_DIR, "structure/output/final_interfaces")
OUTPUT_DIR = os.path.join(BASE_DIR, "output")
COMPRESSION_DIR = os.path.join(OUTPUT_DIR, "compression")
HEAT_DIR = os.path.join(OUTPUT_DIR, "heat")
STRESS_STRAIN_DIR = os.path.join(OUTPUT_DIR, "slow010/stress_strain")
TENSILE_DIR = os.path.join(OUTPUT_DIR, "slow010/tensile")

# ディレクトリ作成
os.makedirs(INPUT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(COMPRESSION_DIR, exist_ok=True)
os.makedirs(HEAT_DIR, exist_ok=True)
os.makedirs(STRESS_STRAIN_DIR, exist_ok=True)
os.makedirs(TENSILE_DIR, exist_ok=True)

# --- ファイルパス生成関数群 ---
def get_input_traj_path(com, atom):
    """入力trajファイルのパスを取得"""
    # ★ ファイル名の形式をCSVの内容に合わせる
    return os.path.join(INPUT_DIR, f"interface_{com}_{atom}.traj")

def get_input_file_path(com, atom):
    """入力xyzファイルのパスを取得"""
    return os.path.join(INPUT_DIR, f"interface_{com}_{atom}.xyz")

def get_compression_file_path(base_name, pressure, comp_temp):
    return os.path.join(COMPRESSION_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_compression.traj")

def get_equil_file_path(base_name, pressure, comp_temp):
    return os.path.join(HEAT_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_equil.traj")

def get_high_temp_file_path(base_name, pressure, comp_temp, high_temp):
    return os.path.join(HEAT_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K.traj")

def get_tensile_file_path(base_name, pressure, comp_temp, high_temp, tensile_temperature):
    return os.path.join(TENSILE_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K_tensile{tensile_temperature:.0f}K_early_stop.traj")

def get_tensile_analysis_file_path(base_name, pressure, comp_temp, high_temp):
    return os.path.join(TENSILE_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K_tensile_analysis.dat")

def get_tensile_summary_file_path(base_name, pressure, comp_temp, high_temp):
    return os.path.join(TENSILE_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K_tensile_summary.txt")

# =============================================
# 早期終了システム設定 & クラス定義
# (このセクションのコードは変更なしのため省略)
# =============================================
class EarlyTerminationException(Exception):
    """早期終了用のカスタム例外"""
    def __init__(self, reason, step, details=None):
        self.reason = reason
        self.step = step
        self.details = details or {}
        super().__init__(f"Early termination at step {step}: {reason}")

# ... (ComprehensiveTensileAnalyzer, run_md_with_gap_detection, save_tensile_results_with_gap_detection などのクラス・関数定義はここにそのままペーストしてください) ...
# (前回のコードから変更はありません)

# =============================================
# ユーティリティ関数
# (このセクションのコードは変更なしのため省略)
# =============================================
def check_md_completed(traj_path, expected_steps, traj_freq):
    # ... (内容は変更なし) ...
    if not os.path.exists(traj_path): return False
    try:
        traj = Trajectory(traj_path, 'r')
        return len(traj) >= (expected_steps // traj_freq) * 0.9
    except:
        return False

# ... (PrintCellShape, rotate_atoms_to_lower, prepare_structure_with_top_vacuum などの関数定義はここにそのままペーストしてください) ...
# (前回のコードから変更はありません)

# =============================================
# ★★★ メイン処理 (CSV読み込み対応版) ★★★
# =============================================

# 読み込むCSVファイルのパス
CSV_FILE_PATH = "/home/jovyan/Kaori/MD/LiB_2/structure/extracted_data_final.csv"

print(f"入力CSV: {CSV_FILE_PATH}")
print(f"入力ファイルディレクトリ: {INPUT_DIR}")
print(f"出力ディレクトリ: {OUTPUT_DIR}")
print("="*50)

# CSVファイルを読み込む
try:
    df = pd.read_csv(CSV_FILE_PATH)
    print(f"✅ CSVから {len(df)} 件の計算対象を読み込みました。")
except FileNotFoundError:
    print(f"❌ エラー: CSVファイルが見つかりません: {CSV_FILE_PATH}")
    exit() # プログラムを終了

# CSVファイルの各行に対してループ処理
for index, row in df.iterrows():
    # CSVの列名 'com', 'atom' から値を取得
    com = row['com']
    atom = row['atom']
    
    # 出力ファイル用のベース名を生成
    base_name = f"{com}_{atom}"
    
    # 入力ファイルのパスを生成
    input_traj = get_input_traj_path(com, atom)
    
    # 入力ファイルの存在をチェック
    if not os.path.exists(input_traj):
        print(f"\n入力ファイルが見つかりません: {input_traj}. スキップします...")
        continue
        
    print(f"\n{'='*20} {base_name} のシミュレーションを開始 {'='*20}")
    
    # ----------------------------------------------------------------
    # ▼▼▼ ここから下のループは、元のプログラムと同じです ▼▼▼
    # ----------------------------------------------------------------
    
    for comp_temp in compression_temperatures:
        for pressure in compression_pressures:
            # --- ステップ0: 圧着 ---
            compression_file = get_compression_file_path(base_name, pressure, comp_temp)
            if not check_md_completed(compression_file, 8000, 100):
                print(f"ステップ0: 圧着実行 (P={pressure:.3f}GPa, T={comp_temp}K)")
                ase_atoms = Trajectory(input_traj)[-1]
                ase_atoms = rotate_atoms_to_lower(ase_atoms)
                indices = [a.index for a in ase_atoms if a.position[2] <= 3.0]
                ase_atoms.set_constraint(FixAtoms(indices=indices))
                
                atoms = MatlantisAtoms.from_ase_atoms(ase_atoms)
                system = ASEMDSystem(atoms)
                system.init_temperature(comp_temp)
                
                integrator = NPTIntegrator(
                    timestep=1.0, temperature=comp_temp, pressure=pressure * units.GPa,
                    ttime=20*units.fs, pfactor=2e6*units.GPa*(units.fs**2),
                    mask=np.array([[0,0,0], [0,0,0], [0,0,1]])
                )
                md = MDFeature(integrator, n_run=8000, traj_file_name=compression_file, traj_freq=100, estimator_fn=estimator_fn)
                md(system, extensions=[(PrintCellShape(), 100)])
            else:
                print(f"ステップ0: 圧着は完了済み (P={pressure:.3f}GPa, T={comp_temp}K)")

            for high_temp in high_temperatures:
                try:
                    # --- ステップ1: 平衡化 ---
                    equil_file = get_equil_file_path(base_name, pressure, comp_temp)
                    if not check_md_completed(equil_file, 3000, 100):
                        print(f"ステップ1: 平衡化実行")
                        ase_atoms = read(compression_file, "-1")
                        ase_atoms = rotate_atoms_to_lower(ase_atoms)
                        indices = [a.index for a in ase_atoms if a.position[2] <= 3.0]
                        ase_atoms.set_constraint(FixAtoms(indices=indices))
                        atoms = MatlantisAtoms.from_ase_atoms(ase_atoms)
                        system = ASEMDSystem(atoms)
                        system.init_temperature(300)
                        integrator = LangevinIntegrator(timestep=1.0, temperature=300.0)
                        md = MDFeature(integrator, n_run=3000, traj_file_name=equil_file, traj_freq=100, estimator_fn=estimator_fn)
                        md(system, extensions=[(PrintCellShape(), 1000)])
                    else:
                        print(f"ステップ1: 平衡化は完了済み")

                    # --- ステップ2: 高温処理 ---
                    high_temp_file = get_high_temp_file_path(base_name, pressure, comp_temp, high_temp)
                    if not check_md_completed(high_temp_file, 10000, 100):
                        print(f"ステップ2: 高温処理実行 ({high_temp}K)")
                        ase_atoms = read(equil_file, "-1")
                        ase_atoms = rotate_atoms_to_lower(ase_atoms)
                        indices = [a.index for a in ase_atoms if a.position[2] <= 2.0]
                        ase_atoms.set_constraint(FixAtoms(indices=indices))
                        atoms = MatlantisAtoms.from_ase_atoms(ase_atoms)
                        system = ASEMDSystem(atoms)
                        system.init_temperature(high_temp)
                        integrator = LangevinIntegrator(timestep=1.0, temperature=float(high_temp))
                        md = MDFeature(integrator, n_run=10000, traj_file_name=high_temp_file, traj_freq=100, estimator_fn=estimator_fn)
                        md(system, extensions=[(PrintCellShape(), 500)])
                    else:
                        print(f"ステップ2: 高温処理は完了済み ({high_temp}K)")

                    # --- ステップ3: 引張シミュレーション ---
                    tensile_file = get_tensile_file_path(base_name, pressure, comp_temp, high_temp, tensile_temperature)
                    if check_md_completed(tensile_file, UNIFIED_CONDITIONS["max_steps"], UNIFIED_CONDITIONS["output_freq"]):
                        print(f"ステップ3: 引張シミュレーションは完了済み")
                        continue
                    
                    print(f"\n{'='*25}\nステップ3: 引張計算開始 (P={pressure:.3f}GPa, HT={high_temp}K)\n{'='*25}")
                    
                    ase_atoms, initial_reference, structure_info = prepare_structure_with_top_vacuum(
                        high_temp_file, 
                        UNIFIED_CONDITIONS["vacuum_top"],
                        UNIFIED_CONDITIONS["fixed_layer_thickness"]
                    )
                    
                    atoms = MatlantisAtoms.from_ase_atoms(ase_atoms)
                    system = ASEMDSystem(atoms)
                    system.init_temperature(UNIFIED_CONDITIONS["temperature"])
                    
                    strain_rate = UNIFIED_CONDITIONS["strain_rate"]
                    max_strain = UNIFIED_CONDITIONS["max_strain"]
                    timestep = UNIFIED_CONDITIONS["timestep"]
                    total_steps = min(int(max_strain / (strain_rate * timestep * 1e-3)), UNIFIED_CONDITIONS["max_steps"])
                    
                    # ★★★ ひずみ計算基準の修正 ★★★
                    # セル長ではなく、純粋な材料の厚さを基準にする
                    initial_material_thickness = structure_info["material_thickness"]
                    if UNIFIED_CONDITIONS["vacuum_top"] > 0:
                        final_material_thickness = initial_material_thickness * (1 + max_strain)
                        final_cell_length = final_material_thickness + UNIFIED_CONDITIONS["vacuum_top"]
                    else:
                        final_cell_length = initial_material_thickness * (1 + max_strain)
                    
                    latt = np.array(system.ase_atoms.cell)
                    latt[2, 2] = final_cell_length
                    
                    analysis_file = get_tensile_analysis_file_path(base_name, pressure, comp_temp, high_temp)
                    analyzer = ComprehensiveTensileAnalyzer(
                        initial_material_thickness, structure_info, UNIFIED_CONDITIONS, high_temp_file, analysis_file
                    )
                    
                    integrator = LangevinIntegrator(timestep=timestep, temperature=UNIFIED_CONDITIONS["temperature"])
                    deform = DeformScheduler(latt, total_steps)
                    md = MDFeature(integrator, n_run=total_steps, traj_file_name=tensile_file, traj_freq=UNIFIED_CONDITIONS["output_freq"], estimator_fn=estimator_fn)
                    
                    extensions = [(PrintCellShape(), 100), (deform, 1), (analyzer, 10)]
                    
                    # ★ 早期終了関数の呼び出しを修正
                    execution_info = run_md_with_gap_detection(md, system, extensions)
                    final_results = analyzer.get_final_results()
                    
                    summary_file = get_tensile_summary_file_path(base_name, pressure, comp_temp, high_temp)
                    save_tensile_results_with_gap_detection(
                        final_results, summary_file, base_name, pressure, comp_temp, high_temp, execution_info
                    )
                    
                    print(f"✅ 引張計算完了: {base_name} (P={pressure:.3f}GPa, HT={high_temp}K)")

                except Exception as e:
                    print(f"❌ エラー発生: {base_name} (P={pressure:.3f}GPa, HT={high_temp}K): {e}")
                    import traceback
                    traceback.print_exc()
                    continue

print("\n全てのシミュレーションが完了しました。")


入力CSV: /home/jovyan/Kaori/MD/LiB_2/structure/extracted_data_final.csv
入力ファイルディレクトリ: /home/jovyan/Kaori/MD/LiB_2/structure/output/final_interfaces
出力ディレクトリ: /home/jovyan/Kaori/MD/LiB_2/output
✅ CSVから 723 件の計算対象を読み込みました。

入力ファイルが見つかりません: /home/jovyan/Kaori/MD/LiB_2/structure/output/final_interfaces/interface_Al_100_(1, 0, 0)_NMC111_Co_MntoNi_40_012_(0, 1, 2).traj. スキップします...

入力ファイルが見つかりません: /home/jovyan/Kaori/MD/LiB_2/structure/output/final_interfaces/interface_AlF3_NMC111_extreme_Ni_rich.traj. スキップします...

入力ファイルが見つかりません: /home/jovyan/Kaori/MD/LiB_2/structure/output/final_interfaces/interface_Al2O3_10-12_(1, 0, 2)_NMC111_CotoNi_80_001_(0, 0, 1)_trimmed_final.traj. スキップします...

入力ファイルが見つかりません: /home/jovyan/Kaori/MD/LiB_2/structure/output/final_interfaces/interface_Al2O3_NMC111_CotoNi_80_100_(1, 0, 0).traj. スキップします...

入力ファイルが見つかりません: /home/jovyan/Kaori/MD/LiB_2/structure/output/final_interfaces/interface_Al_100_(1, 0, 0)_NMC111_thermal_Co_Mn_loss_110_(1, 1, 0).traj. スキップします...

入力ファイルが見つか

In [57]:
from ase.io import read, Trajectory, write
from ase.constraints import FixAtoms
import numpy as np
import os
import time
from datetime import datetime
from sklearn.linear_model import LinearRegression
from matlantis_features.features.md import ASEMDSystem, LangevinIntegrator, MDFeature, MDExtensionBase, NPTIntegrator
from matlantis_features.features.md.md_extensions import DeformScheduler
from pfp_api_client.pfp.estimator import Estimator, EstimatorCalcMode
from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
from matlantis_features.utils.calculators import get_calculator, pfp_estimator_fn
from matlantis_features.atoms import MatlantisAtoms
import pathlib
import pandas as pd
from ase import units
import numpy as np

# =============================================
# エネルギー計算用の関数を定義
# =============================================
estimator_fn = pfp_estimator_fn(model_version='v7.0.0', calc_mode='crystal_u0_plus_d3')

# =============================================
# シミュレーションパラメータ設定
# =============================================
# 解析対象の物質リスト
atoms_list = ["Mn3O4"]
compounds = ["Al2O3"]

# パラメータの設定
compression_temperatures = [300]  # 圧着シミュレーション温度（K）
compression_pressures = [0.001, 0.005, 0.01]  # 圧着圧力（GPa）
high_temperatures = [300]  # 高温シミュレーション温度（K）
tensile_temperature = 300.0  # 引張シミュレーション温度（K）

# =============================================
# ファイル・ディレクトリパス設定
# =============================================
BASE_DIR = "/home/jovyan/Kaori/MD/LiB_2/"
INPUT_DIR = os.path.join(BASE_DIR, "structure/output/final_interfaces")
OUTPUT_DIR = os.path.join(BASE_DIR, "output")
COMPRESSION_DIR = os.path.join(OUTPUT_DIR, "compression")
HEAT_DIR = os.path.join(OUTPUT_DIR, "heat")
STRESS_STRAIN_DIR = os.path.join(OUTPUT_DIR, "slow010/stress_strain")
TENSILE_DIR = os.path.join(OUTPUT_DIR, "slow010/tensile")

# ディレクトリ作成
os.makedirs(INPUT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(COMPRESSION_DIR, exist_ok=True)
os.makedirs(HEAT_DIR, exist_ok=True)
os.makedirs(STRESS_STRAIN_DIR, exist_ok=True)
os.makedirs(TENSILE_DIR, exist_ok=True)

# --- ファイルパス生成関数群 ---
def get_input_traj_path(com, atom):
    return os.path.join(INPUT_DIR, f"{com}_{atom}_interface_optimized.traj")

def get_input_xyz_path(com, atom):
    
    return os.path.join(INPUT_DIR, f"Interface_{com}_on_{atom}.xyz")
# Kaori/MD/LiB_2/structure/output/final_interfaces/Interface_Al_100_(1, 0, 0)_on_NMC111_CotoNi_50_trimmed_final.xyz
def get_compression_file_path(base_name, pressure, comp_temp):
    return os.path.join(COMPRESSION_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_compression.traj")

def get_equil_file_path(base_name, pressure, comp_temp):
    return os.path.join(HEAT_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_equil.traj")

def get_high_temp_file_path(base_name, pressure, comp_temp, high_temp):
    return os.path.join(HEAT_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K.traj")

def get_tensile_file_path(base_name, pressure, comp_temp, high_temp, tensile_temperature):
    return os.path.join(TENSILE_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K_tensile{tensile_temperature:.0f}K_early_stop.traj")

def get_tensile_analysis_file_path(base_name, pressure, comp_temp, high_temp):
    return os.path.join(TENSILE_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K_tensile_analysis.dat")

def get_tensile_summary_file_path(base_name, pressure, comp_temp, high_temp):
    return os.path.join(TENSILE_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K_tensile_summary.txt")

# =============================================
# 早期終了システム設定
# =============================================
UNIFIED_CONDITIONS = {
    "strain_rate": 0.01,
    "vacuum_top": 0.0,
    "temperature": 300.0,
    "max_strain": 3.0,
    "timestep": 1.0,
    "elastic_region": (0.005, 0.03),
    "quality_threshold": 0.9,
    "max_steps": 100000,
    "output_freq": 100,
    "fixed_layer_thickness": 15
}

EARLY_STOP_CONDITIONS = {
    "separation_gap": 10.0,
    "max_strain_limit": 5.0,
    "min_stress_threshold": -1.0,
    "separation_wait_steps": 500,
    "stress_drop_ratio": 0.6,
    "internal_gap_threshold": 10.0,
    "gap_search_margin": 0.1
}

class EarlyTerminationException(Exception):
    """早期終了用のカスタム例外"""
    def __init__(self, reason, step, details=None):
        self.reason = reason
        self.step = step
        self.details = details or {}
        super().__init__(f"Early termination at step {step}: {reason}")

def find_largest_gap_z(atoms, search_margin_ratio=0.1):
    """Z座標でソートした原子間の最大の隙間（ギャップ）の中間点を返す"""
    positions = atoms.get_positions()
    if len(positions) < 2:
        return atoms.get_cell()[2, 2] / 2.0, 0.0

    sorted_z = np.sort(positions[:, 2])
    start_idx = int(len(sorted_z) * search_margin_ratio)
    end_idx = int(len(sorted_z) * (1 - search_margin_ratio))
    
    if start_idx >= end_idx:
        return np.mean(sorted_z), 0.0

    search_z = sorted_z[start_idx:end_idx]
    gaps = search_z[1:] - search_z[:-1]
    
    max_gap_index = np.argmax(gaps)
    max_gap_value = gaps[max_gap_index]
    fracture_z = (search_z[max_gap_index] + search_z[max_gap_index + 1]) / 2.0
    
    return fracture_z, max_gap_value

class EarlyTerminationException(Exception):
    """早期終了用のカスタム例外"""
    def __init__(self, reason, step, details=None):
        self.reason = reason
        self.step = step
        self.details = details or {}
        super().__init__(f"Early termination at step {step}: {reason}")

class ComprehensiveTensileAnalyzer(MDExtensionBase):
    """
    早期終了機能を無効化した包括的引張解析器
    """
    
    def __init__(self, initial_thickness, structure_info, conditions, source_file, output_file):
        self.initial_thickness = initial_thickness
        self.structure_info = structure_info
        self.conditions = conditions
        self.source_file = source_file
        self.output_file = output_file
        
        # 解析データ
        self.data = []
        self.young_modulus = None
        self.young_r2 = 0.0
        self.young_intercept = 0.0
        self.max_stress = 0.0
        self.max_stress_strain = 0.0
        self.fracture_detected = False
        self.fracture_step = None
        self.complete_separation = False
        self.separation_step = None
        
        # 早期終了関連（無効化）
        self.early_termination_requested = False
        self.termination_reason = None
        self.termination_details = {}
        
        # チェック用カウンタ（記録のみ）
        self.stress_drop_count = 0
        self.separation_wait_count = 0
        
        # ヘッダー書き込み
        with open(self.output_file, 'w') as f:
            f.write(f"# Integrated Tensile Analysis (No Early Termination)\n")
            f.write(f"# Source: {source_file}\n")
            f.write(f"# Unified Strain Rate: {conditions['strain_rate']} 1/ps\n")
            f.write(f"# Detection Conditions (Record Only):\n")
            f.write(f"#   Separation Gap: {EARLY_STOP_CONDITIONS['separation_gap']} A (RECORD)\n")
            f.write(f"#   Max Strain Limit: {EARLY_STOP_CONDITIONS['max_strain_limit']} (RECORD)\n")
            f.write(f"#   Min Stress Threshold: {EARLY_STOP_CONDITIONS['min_stress_threshold']} GPa (RECORD)\n")
            f.write(f"#   Stress Drop Detection: {EARLY_STOP_CONDITIONS['stress_drop_ratio']} (RECORD)\n")
            if UNIFIED_CONDITIONS["vacuum_top"] > 0:
                f.write(f"# Step Material_Strain Stress(GPa) YoungMod(GPa) Status Time(ps) Notes\n")
            else:
                f.write(f"# Step Cell_Strain Stress(GPa) YoungMod(GPa) Status Time(ps) Notes\n")

    def detect_events(self, system, material_strain, stress_zz_GPa, z_coords=None):
        """イベント検出（記録のみ、早期終了はしない）"""
        
        # 1. 完全分離チェック（vacuum有りの場合のみ）
        if UNIFIED_CONDITIONS["vacuum_top"] > 0 and z_coords is not None:
            if system.current_total_step % 100 == 0:
                z_median = np.median(z_coords)
                upper_z = z_coords[z_coords > z_median]
                lower_z = z_coords[z_coords <= z_median]
                
                if len(upper_z) > 0 and len(lower_z) > 0:
                    gap = np.min(upper_z) - np.max(lower_z)
                    if np.max(lower_z) >= 8.0:
                        if gap > EARLY_STOP_CONDITIONS["separation_gap"]:
                            if not self.complete_separation:
                                self.complete_separation = True
                                self.separation_step = system.current_total_step
                                print(f"      🔗 完全分離検出: Step {self.separation_step}, 隙間 {gap:.1f}Å（記録のみ）")
                                
                                self.termination_details.update({
                                    "separation_gap": gap,
                                    "separation_step": self.separation_step
                                })
                            
                            self.separation_wait_count += 1
        
        # 2. 応力急減チェック（破断検出）- 記録のみ
        if len(self.data) > 100 and self.max_stress > 0:
            recent_stresses = [d[2] for d in self.data[-50:]]
            if len(recent_stresses) > 0:
                recent_max_stress = max(recent_stresses)
                stress_ratio = recent_max_stress / self.max_stress
                
                if stress_ratio < EARLY_STOP_CONDITIONS["stress_drop_ratio"]:
                    if not self.fracture_detected:
                        self.fracture_detected = True
                        self.fracture_step = system.current_total_step
                        print(f"      ⚠️  破断検出: Step {self.fracture_step}, 応力比 {stress_ratio:.3f}（記録のみ）")
                        
                        self.termination_details.update({
                            "fracture_stress_ratio": stress_ratio,
                            "fracture_step": self.fracture_step
                        })
        
        # 3. 最大ひずみ制限チェック - 記録のみ
        if material_strain > EARLY_STOP_CONDITIONS["max_strain_limit"]:
            if "max_strain_exceeded" not in self.termination_details:
                print(f"      📊 最大ひずみ制限超過: {material_strain:.3f}（記録のみ）")
                self.termination_details["max_strain_exceeded"] = {
                    "strain": material_strain,
                    "step": system.current_total_step
                }
        
        # 4. 圧縮転換チェック - 記録のみ
        if stress_zz_GPa < EARLY_STOP_CONDITIONS["min_stress_threshold"]:
            if "compression_detected" not in self.termination_details:
                print(f"      ⬇️  圧縮転換検出: {stress_zz_GPa:.3f} GPa（記録のみ）")
                self.termination_details["compression_detected"] = {
                    "stress": stress_zz_GPa,
                    "step": system.current_total_step
                }
    
    def __call__(self, system, integrator):
        # ひずみ計算の分岐（vacuum有無による）
        if UNIFIED_CONDITIONS["vacuum_top"] > 0:
            # vacuum有り：材料長基準
            positions = system.ase_atoms.get_positions()
            z_coords = positions[:, 2]
            current_material_length = np.max(z_coords) - np.min(z_coords)
            material_strain = (current_material_length - self.initial_thickness) / self.initial_thickness
            strain_method = "material_length"
        else:
            # vacuum無し：セル長基準
            current_cell_length = system.ase_atoms.get_cell()[2, 2]
            material_strain = (current_cell_length - self.initial_thickness) / self.initial_thickness
            current_material_length = current_cell_length  # セル長=材料長
            strain_method = "cell_length"
            z_coords = None
        
        # 応力取得
        try:
            stress_tensor = system.ase_atoms.get_stress(voigt=False)
            stress_zz_GPa = stress_tensor[2, 2] * 160.21766208  # eV/Å³ to GPa
        except:
            stress_zz_GPa = 0.0
        
        # セル長
        cell_length = system.ase_atoms.get_cell()[2, 2]
        time_ps = system.current_total_step * self.conditions["timestep"] / 1000
        
        if UNIFIED_CONDITIONS["vacuum_top"] > 0:
            # vacuum有り：材料長も記録
            self.data.append([
                system.current_total_step, material_strain, stress_zz_GPa,
                current_material_length, cell_length, time_ps
            ])
        else:
            # vacuum無し：材料長記録なし（セル長のみ）
            self.data.append([
                system.current_total_step, material_strain, stress_zz_GPa,
                None, cell_length, time_ps
            ])
        
        # 最大応力更新
        if stress_zz_GPa > self.max_stress:
            self.max_stress = stress_zz_GPa
            self.max_stress_strain = material_strain
        
        # ヤング率計算
        status = "Loading"
        current_young = 0.0
        
        if len(self.data) > 50 and material_strain > 0.01 and self.young_modulus is None:
            strains = np.array([d[1] for d in self.data])
            stresses = np.array([d[2] for d in self.data])
            
            elastic_min, elastic_max = self.conditions["elastic_region"]
            elastic_mask = (strains > elastic_min) & (strains < elastic_max) & (stresses > 0)
            
            if np.sum(elastic_mask) > 20:
                elastic_strains = strains[elastic_mask]
                elastic_stresses = stresses[elastic_mask]
                
                X = elastic_strains.reshape(-1, 1)
                y = elastic_stresses
                
                model = LinearRegression().fit(X, y)
                predicted = model.predict(X)
                
                self.young_modulus = model.coef_[0]
                self.young_intercept = model.intercept_
                
                ss_res = np.sum((y - predicted) ** 2)
                ss_tot = np.sum((y - np.mean(y)) ** 2)
                self.young_r2 = 1 - (ss_res / ss_tot) if ss_tot > 0 else 0
                
                print(f"      ✅ ヤング率確定: {self.young_modulus:.1f} GPa (R² = {self.young_r2:.3f})")
        
        if self.young_modulus is not None:
            current_young = self.young_modulus
            if material_strain > self.conditions["elastic_region"][1]:
                status = "Plastic"
        
        # イベント検出（記録のみ、早期終了はしない）
        self.detect_events(system, material_strain, stress_zz_GPa, z_coords)
        
        # 状態更新
        if self.complete_separation:
            status = "Separated"
        elif self.fracture_detected:
            status = "Fractured"
        
        # 早期終了処理は無効化 - 常に計算を継続
        
        # ファイル出力（計算方法も記録）
        notes = f"Method:{strain_method}"
        if self.complete_separation:
            notes += f" Sep:{self.separation_wait_count}"
        if self.fracture_detected:
            notes += f" Frac"
        
        with open(self.output_file, 'a') as f:
            f.write(f"{system.current_total_step:<8} {material_strain:<14.6f} {stress_zz_GPa:<12.6f} ")
            f.write(f"{current_young:<12.1f} {status:<10} {time_ps:<10.3f} {notes}\n")
    
    def get_final_results(self):
        """最終結果を辞書形式で返す"""
        return {
            'young_modulus': self.young_modulus or 0.0,
            'young_r2': self.young_r2,
            'young_intercept': self.young_intercept,
            'max_stress': self.max_stress,
            'max_stress_strain': self.max_stress_strain,
            'fracture_detected': self.fracture_detected,
            'fracture_step': self.fracture_step,
            'complete_separation': self.complete_separation,
            'separation_step': self.separation_step,
            'total_data_points': len(self.data),
            'structure_info': self.structure_info,
            'conditions': self.conditions,
            'early_termination': False,  # 常にFalse
            'termination_reason': None,
            'termination_details': self.termination_details
        }

# class ComprehensiveTensileAnalyzer(MDExtensionBase):
#     """隙間検出による早期終了機能付き包括的引張解析器"""
#     def __init__(self, initial_thickness, structure_info, conditions, source_file, output_file):
#         self.initial_thickness = initial_thickness
#         self.structure_info = structure_info
#         self.conditions = conditions
#         self.source_file = source_file
#         self.output_file = output_file
        
#         self.data = []
#         self.young_modulus = None
#         self.young_r2 = 0.0
#         self.max_stress = 0.0
#         self.max_stress_strain = 0.0
#         self.fracture_detected = False
#         self.fracture_step = None
#         self.complete_separation = False
#         self.separation_step = None
#         self.internal_gap_detected = False
#         self.internal_gap_step = None
#         self.max_internal_gap = 0.0
#         self.early_termination_requested = False
#         self.termination_reason = None
#         self.termination_details = {}
#         self.gap_check_frequency = 50
        
#         with open(self.output_file, 'w') as f:
#             f.write(f"# Integrated Tensile Analysis with Internal Gap Detection\n")
#             f.write(f"# Source: {source_file}\n")
#             f.write(f"# Early Termination by Internal Gap: {EARLY_STOP_CONDITIONS['internal_gap_threshold']} A\n")
#             if UNIFIED_CONDITIONS["vacuum_top"] > 0:
#                 f.write(f"# Step Material_Strain Stress(GPa) YoungMod(GPa) Max_Gap(A) Status Time(ps) Notes\n")
#             else:
#                 f.write(f"# Step Cell_Strain Stress(GPa) YoungMod(GPa) Max_Gap(A) Status Time(ps) Notes\n")

#     def detect_events(self, system, material_strain, stress_zz_GPa, z_coords=None):
#         current_step = system.current_total_step
        
#         if current_step % self.gap_check_frequency == 0:
#             fracture_z, max_gap = find_largest_gap_z(
#                 system.ase_atoms, 
#                 search_margin_ratio=EARLY_STOP_CONDITIONS["gap_search_margin"]
#             )
#             self.max_internal_gap = max(self.max_internal_gap, max_gap)
            
#             if max_gap >= EARLY_STOP_CONDITIONS["internal_gap_threshold"]:
#                 if not self.internal_gap_detected:
#                     self.internal_gap_detected = True
#                     self.internal_gap_step = current_step
#                     print(f"    🚨 内部隙間検出: Step {self.internal_gap_step}, 隙間 {max_gap:.1f}Å")
#                     self.early_termination_requested = True
#                     self.termination_reason = f"internal_gap_{max_gap:.1f}A"
#                     self.termination_details.update({
#                         "internal_gap": max_gap,
#                         "internal_gap_step": self.internal_gap_step
#                     })
    #                 raise EarlyTerminationException(
    #                     reason=f"Internal gap {max_gap:.1f}Å detected",
    #                     step=current_step,
    #                     details=self.termination_details
    #                 )

    # def __call__(self, system, integrator):
    #     if UNIFIED_CONDITIONS["vacuum_top"] > 0:
    #         positions = system.ase_atoms.get_positions()
    #         z_coords = positions[:, 2]
    #         current_material_length = np.max(z_coords) - np.min(z_coords)
    #         material_strain = (current_material_length - self.initial_thickness) / self.initial_thickness
    #         strain_method = "material_length"
    #     else:
    #         current_cell_length = system.ase_atoms.get_cell()[2, 2]
    #         material_strain = (current_cell_length - self.initial_thickness) / self.initial_thickness
    #         strain_method = "cell_length"
    #         z_coords = None
        
    #     try:
    #         stress_tensor = system.ase_atoms.get_stress(voigt=False)
    #         stress_zz_GPa = stress_tensor[2, 2] * 160.21766208
    #     except:
    #         stress_zz_GPa = 0.0
        
    #     time_ps = system.current_total_step * self.conditions["timestep"] / 1000
    #     self.data.append([system.current_total_step, material_strain, stress_zz_GPa, self.max_internal_gap, time_ps])

        # if stress_zz_GPa > self.max_stress:
        #     self.max_stress = stress_zz_GPa
        #     self.max_stress_strain = material_strain
            
        # current_young = 0.0
        # if len(self.data) > 50 and material_strain > 0.01 and self.young_modulus is None:
        #     strains = np.array([d[1] for d in self.data])
        #     stresses = np.array([d[2] for d in self.data])
        #     elastic_min, elastic_max = self.conditions["elastic_region"]
        #     elastic_mask = (strains > elastic_min) & (strains < elastic_max) & (stresses > 0)
            
        #     if np.sum(elastic_mask) > 20:
        #         elastic_strains = strains[elastic_mask].reshape(-1, 1)
        #         elastic_stresses = stresses[elastic_mask]
        #         model = LinearRegression().fit(elastic_strains, elastic_stresses)
        #         self.young_modulus = model.coef_[0]
        #         self.young_r2 = model.score(elastic_strains, elastic_stresses)
        #         print(f"    ✅ ヤング率確定: {self.young_modulus:.1f} GPa (R² = {self.young_r2:.3f})")

        # if self.young_modulus is not None:
        #     current_young = self.young_modulus
        
        # self.detect_events(system, material_strain, stress_zz_GPa, z_coords)
        
        # status = "Loading"
        # if self.young_modulus is not None and material_strain > self.conditions["elastic_region"][1]:
        #     status = "Plastic"
    #     if self.internal_gap_detected:
    #         status = "Gap_Detected"

    #     with open(self.output_file, 'a') as f:
    #         f.write(f"{system.current_total_step:<8} {material_strain:<14.6f} {stress_zz_GPa:<12.6f} ")
    #         f.write(f"{current_young:<12.1f} {self.max_internal_gap:<10.1f} {status:<15} {time_ps:<10.3f}\n")

    # def get_final_results(self):
    #     return {
    #         'young_modulus': self.young_modulus or 0.0,
    #         'young_r2': self.young_r2,
    #         'max_stress': self.max_stress,
    #         'max_stress_strain': self.max_stress_strain,
    #         'internal_gap_detected': self.internal_gap_detected,
    #         'internal_gap_step': self.internal_gap_step,
    #         'max_internal_gap': self.max_internal_gap,
    #         'total_data_points': len(self.data),
    #         'structure_info': self.structure_info,
    #         'conditions': self.conditions,
    #         'early_termination': self.early_termination_requested,
    #         'termination_reason': self.termination_reason,
    #     }

def run_md_with_gap_detection(md_feature, system, extensions):
    """隙間検出による早期終了機能付きのMD実行ラッパー"""
    execution_info = {'completed_normally': False, 'early_terminated': False}
    start_time = time.time()
    try:
        print(f"🚀 MD計算開始（隙間検出による早期終了機能付き）")
        md_feature(system, extensions=extensions)
        execution_info['completed_normally'] = True
        execution_info['total_steps_run'] = UNIFIED_CONDITIONS["max_steps"]
        print(f"✅ MD計算正常完了")
    except EarlyTerminationException as e:
        execution_info['early_terminated'] = True
        execution_info['termination_reason'] = e.reason
        execution_info['termination_step'] = e.step
        execution_info['total_steps_run'] = e.step
        print(f"🛑 MD計算早期終了: {e.reason} at step {e.step}")
    except Exception as e:
        execution_info['error_occurred'] = True
        execution_info['error_message'] = str(e)
        print(f"❌ MD計算エラー終了: {e}")
    finally:
        execution_info['execution_time'] = time.time() - start_time
    return execution_info

def save_tensile_results_with_gap_detection(results, summary_file, base_name, pressure, comp_temp, high_temp, execution_info):
    """引張計算結果をファイルに保存（隙間検出機能付き版）"""
    with open(summary_file, 'w') as f:
        f.write(f"引張計算結果（隙間検出による早期終了） - {base_name}\n")
        f.write("="*70 + "\n")
        f.write(f"完了時刻: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"計算時間: {execution_info.get('execution_time', 0)/60:.1f} 分\n\n")
        f.write(f"条件: P={pressure:.3f}GPa, CT={comp_temp}K, HT={high_temp}K\n\n")
        f.write(f"早期終了システム:\n")
        f.write(f"  内部隙間閾値: {EARLY_STOP_CONDITIONS['internal_gap_threshold']} Å（早期終了）\n")
        f.write(f"  最大検出隙間: {results['max_internal_gap']:.1f} Å\n\n")
        f.write(f"実行結果:\n")
        if execution_info.get('early_terminated'):
            f.write(f"  状態: 早期終了（{execution_info.get('termination_reason')}）\n")
            f.write(f"  終了ステップ: {execution_info.get('termination_step')}\n")
        elif execution_info.get('completed_normally'):
            f.write(f"  状態: 正常完了\n")
        elif execution_info.get('error_occurred'):
            f.write(f"  状態: エラー終了\n")
            f.write(f"  エラー: {execution_info.get('error_message')}\n")
        f.write(f"\n機械特性:\n")
        f.write(f"  ヤング率: {results['young_modulus']:.1f} GPa (R² = {results['young_r2']:.3f})\n")
        f.write(f"  最大応力: {results['max_stress']:.2f} GPa @ ひずみ {results['max_stress_strain']:.3f}\n")

# --- ユーティリティ関数 ---
def check_md_completed(traj_path, expected_steps, traj_freq):
    if not os.path.exists(traj_path): return False
    try:
        traj = Trajectory(traj_path, 'r')
        return len(traj) >= (expected_steps // traj_freq) * 0.9
    except:
        return False

class PrintCellShape(MDExtensionBase):
    def __call__(self, system, integrator):
        if system.current_total_step % 1000 == 0:
            print(f"Dyn step {system.current_total_step:4d} Cell(c): {system.ase_atoms.cell.cellpar()[2]:.2f}")

def rotate_atoms_to_lower(atoms):
    # ase_atoms = atoms.ase_atoms if hasattr(atoms, 'ase_atoms') else atoms
    positions = ase_atoms.get_positions()
    z_min = np.min(positions[:, 2])
    ase_atoms.translate([0, 0, -z_min])
    print(f"原子をZ方向に {-z_min:.2f}Å 移動しました")
    return ase_atoms
# def rotate_atoms_to_lower(ase_atoms):
#     # この関数に渡されるのは常に標準のase.Atomsオブジェクトなので、直接操作する
#     positions = ase_atoms.get_positions()
#     z_min = np.min(positions[:, 2])
#     ase_atoms.translate([0, 0, -z_min])
#     return ase_atoms

def prepare_structure_with_top_vacuum(high_temp_file, vacuum_top, fixed_layer_ratio):
    if not os.path.exists(high_temp_file):
        raise FileNotFoundError(f"高温処理ファイルが見つかりません: {high_temp_file}")
    
    ase_atoms = read(high_temp_file, "-1")
    ase_atoms = rotate_atoms_to_lower(ase_atoms)
    
    positions = ase_atoms.get_positions()
    material_thickness = np.max(positions[:, 2]) - np.min(positions[:, 2])
    print(f"  構造解析: 材料厚さ={material_thickness:.2f}Å, 原子数={len(ase_atoms)}")
    
    new_cell_z = material_thickness + vacuum_top
    cell = ase_atoms.get_cell()
    cell[2, 2] = new_cell_z
    ase_atoms.set_cell(cell)
    ase_atoms.wrap()
    
    current_z_min = np.min(ase_atoms.get_positions()[:, 2])
    current_z_max = np.max(ase_atoms.get_positions()[:, 2])
    current_z_range = current_z_max - current_z_min
    fixed_layer_thickness = current_z_range * fixed_layer_ratio / 100
    lower_fixed_threshold = current_z_min + fixed_layer_thickness
    upper_fixed_threshold = current_z_max - fixed_layer_thickness
    
    indices = [
        atom.index for atom in ase_atoms
        if atom.position[2] <= lower_fixed_threshold or atom.position[2] >= upper_fixed_threshold
    ]
    ase_atoms.set_constraint(FixAtoms(indices=indices))
    
    initial_reference = new_cell_z
    
    structure_info = {
        "material_thickness": material_thickness,
        "vacuum_top": vacuum_top,
        "total_cell_z": new_cell_z,
        "fixed_atoms": len(indices),
        "fixed_ratio": len(indices) / len(ase_atoms),
        "strain_method": "material_length" if vacuum_top > 0 else "cell_length"
    }
    
    print(f"  配置完了: セルZ長={new_cell_z:.2f}Å, 固定原子率={structure_info['fixed_ratio']*100:.1f}%")
    return ase_atoms, initial_reference, structure_info

# --- Matlantis最適化関数 (共通) ---
def run_matlantis_optimization(atoms, fmax=0.05, ):
    print(f"  -> Matlantis最適化開始 (, fmax = {fmax}) ...")
    matlantis_atoms = MatlantisAtoms(atoms)
    estimator_function = pfp_estimator_fn(
        model_version='v7.0.0', calc_mode=EstimatorCalcMode.CRYSTAL_PLUS_D3
    )
    position_optimizer = FireLBFGSASEOptFeature(
        estimator_fn=estimator_function, filter=False,n_run=5000, fmax=fmax, show_progress_bar=True)
 # trajectory=str(trajectory_path
    try:
        result = position_optimizer(matlantis_atoms)
        optimized_atoms = result.atoms.ase_atoms
        final_energy = result.output.energy_log[-1]
        print(f"  -> ✔️ 最適化完了！ エネルギー: {final_energy:.3f} eV")
        # .xyzファイルでの保存はここで継続
        # xyz_save_path = Path(trajectory_path).with_suffix('.xyz')
        print(f"  -> 💾 最適化後の中間構造を保存します: ")
        # write(str(xyz_save_path), optimized_atoms)
        return optimized_atoms  # ★変更点: 最適化後の構造オブジェクトを返す
    except Exception as e:
        print(f"  -> ❌ 最適化中にエラーが発生: {e}")
        return None  # ★変更点: 失敗した場合はNoneを返す
# =============================================
# メイン処理
# =============================================
print(f"入力ディレクトリ: {INPUT_DIR}")
print(f"出力ディレクトリ: {OUTPUT_DIR}")
print("="*50)
from matlantis_features.features.common.opt import FireLBFGSASEOptFeature
# # 読み込むCSVファイルのパス
# CSV_FILE_PATH = "/home/jovyan/Kaori/MD/LiB_2/structure/extracted_data_final.csv"

# print(f"入力CSV: {CSV_FILE_PATH}")
# print(f"入力ファイルディレクトリ: {INPUT_DIR}")
# print(f"出力ディレクトリ: {OUTPUT_DIR}")
# print("="*50)

# # CSVファイルを読み込む
# try:
#     df = pd.read_csv(CSV_FILE_PATH)
#     print(f"✅ CSVから {len(df)} 件の計算対象を読み込みました。")
# except FileNotFoundError:
#     print(f"❌ エラー: CSVファイルが見つかりません: {CSV_FILE_PATH}")
#     exit() # プログラムを終了

# # CSVファイルの各行に対してループ処理
# for index, row in df.iterrows():
#     # CSVの列名 'com', 'atom' から値を取得
#     com = row['com']
#     atom = row['atom']
    
#     # 出力ファイル用のベース名を生成
#     base_name = f"{com}_{atom}"
    
#     # 入力ファイルのパスを生成
#     input_traj = get_input_xyz_path(com, atom)
    
#     # 入力ファイルの存在をチェック
#     if not os.path.exists(input_traj):
#         print(f"\n入力ファイルが見つかりません: {input_traj}. スキップします...")
#         continue
        
#     print(f"\n{'='*20} {base_name} のシミュレーションを開始 {'='*20}")
    
#     # ----------------------------------------------------------------
#     # ▼▼▼ ここから下のループは、元のプログラムと同じです ▼▼▼
#     # ----------------------------------------------------------------
    
# for atom in atoms_list:
#     for com in compounds:
CSV_FILE_PATH = "/home/jovyan/Kaori/MD/LiB_2/structure/extracted_data_final.csv"

print(f"入力CSV: {CSV_FILE_PATH}\n入力ディレクトリ: {INPUT_DIR}\n{'='*50}")

try:
    df = pd.read_csv(CSV_FILE_PATH)
    print(f"✅ CSVから {len(df)} 件の計算対象を読み込みました。")
except FileNotFoundError:
    print(f"❌ エラー: CSVファイルが見つかりません: {CSV_FILE_PATH}")
    exit()

for index, row in df.iterrows():
    com, atom = row['com'], row['atom']
    base_name = f"{com}_{atom}"
    input_traj = get_input_xyz_path(com, atom)

    if not os.path.exists(input_traj):
        print(f"\n入力ファイルが見つかりません: {input_traj}. スキップします...")
        continue
        
    print(f"\n{'='*20} {base_name} のシミュレーションを開始 {'='*20}")

    base_name = f"{com}_{atom}"
    input_traj = get_input_xyz_path(com, atom)
    
    if not os.path.exists(input_traj):
        print(f"入力ファイルが見つかりません: {input_traj}. スキップします...")
        continue
        
    print(f"\n{'='*20} {base_name} のシミュレーションを開始 {'='*20}")
    
    for comp_temp in compression_temperatures:
        for pressure in compression_pressures:
            # --- ステップ0: 圧着 ---
            compression_file = get_compression_file_path(base_name, pressure, comp_temp)
            if not check_md_completed(compression_file, 8000, 100):
                print(f"ステップ0: 圧着実行 (P={pressure:.3f}GPa, T={comp_temp}K)")
                # ase_atoms = Trajectory(input_traj)[-1]
                ase_atoms=read(input_traj)
                # ase_atoms=atoms.ase_atomsatoms = MatlantisAtoms.from_ase_atoms(ase_atoms)
                ase_atoms = rotate_atoms_to_lower(ase_atoms)
                        # traj_path = self.interfaces_dir / f"{interface_name}.traj"
                # optimized_structure = run_matlantis_optimization(ase_atoms1, fmax=0.05)
    
                # # STEP 4: 保存
                # if optimized_structure:
                #     optimized_structure.center(vacuum=15.0, axis=2)
                #     optimized_structure.pbc = (True, True, False)
                #     # print(f"    4. 最終構造をファイルに保存中 -> {output_path.name}")
                #     # write(str(output_path), optimized_structure)
                # else:
                #     print("   -> ❌ 最適化に失敗したため、ファイルは保存されません。")

                
                # system.get_potential_energy()
                # ase_atoms=optimized_structure.copy()
                # atoms = MatlantisAtoms.from_ase_atoms(ase_atoms)
                indices = [a.index for a in ase_atoms if a.position[2] <= 3.0]
                system.ase_atoms.calc = get_calculator(estimator_fn)
                ase_atoms.set_constraint(FixAtoms(indices=indices))
                atoms = MatlantisAtoms.from_ase_atoms(ase_atoms)
                system = ASEMDSystem(atoms)
                system.init_temperature(comp_temp)
                
                integrator = NPTIntegrator(
                    timestep=1.0, temperature=comp_temp, pressure=pressure * units.GPa,
                    ttime=20*units.fs, pfactor=2e6*units.GPa*(units.fs**2),
                    mask=np.array([[0,0,0], [0,0,0], [0,0,1]]))
                md = MDFeature(integrator, n_run=8000, traj_file_name=compression_file, traj_freq=100, estimator_fn=estimator_fn)
                md(system)
                # , extensions=[(PrintCellShape(), 100)]
            else:
                print(f"ステップ0: 圧着は完了済み (P={pressure:.3f}GPa, T={comp_temp}K)")

            for high_temp in high_temperatures:
                try:
                    # --- ステップ1: 平衡化 ---
                    equil_file = get_equil_file_path(base_name, pressure, comp_temp)
                    if not check_md_completed(equil_file, 3000, 100):
                        print(f"ステップ1: 平衡化実行")
                        indices = [a.index for a in ase_atoms if a.position[2] <= 3.0]
                        ase_atoms.set_constraint(FixAtoms(indices=indices))
                        atoms = MatlantisAtoms.from_ase_atoms(ase_atoms)
                        system = ASEMDSystem(atoms)
                        system.init_temperature(300)
                        system.get_potential_energy()
                        integrator = LangevinIntegrator(timestep=1.0, temperature=300.0)
                        md = MDFeature(integrator, n_run=3000, traj_file_name=equil_file, traj_freq=100, estimator_fn=estimator_fn)
                        md(system, extensions=[(PrintCellShape(), 1000)])
                    else:
                        print(f"ステップ1: 平衡化は完了済み")

                    # --- ステップ2: 高温処理 ---
                    high_temp_file = get_high_temp_file_path(base_name, pressure, comp_temp, high_temp)
                    if not check_md_completed(high_temp_file, 10000, 100):
                        print(f"ステップ2: 高温処理実行 ({high_temp}K)")
                        ase_atoms = read(equil_file, "-1")
                        ase_atoms = rotate_atoms_to_lower(ase_atoms)
                        indices = [a.index for a in ase_atoms if a.position[2] <= 2.0]
                        ase_atoms.set_constraint(FixAtoms(indices=indices))
                        atoms = MatlantisAtoms.from_ase_atoms(ase_atoms)
                        system = ASEMDSystem(atoms)
                        system.init_temperature(high_temp)
                        integrator = LangevinIntegrator(timestep=1.0, temperature=float(high_temp))
                        md = MDFeature(integrator, n_run=10000, traj_file_name=high_temp_file, traj_freq=100, estimator_fn=estimator_fn)
                        md(system, extensions=[(PrintCellShape(), 500)])
                    else:
                        print(f"ステップ2: 高温処理は完了済み ({high_temp}K)")

                    # --- ステップ3: 引張シミュレーション ---
                    tensile_file = get_tensile_file_path(base_name, pressure, comp_temp, high_temp, tensile_temperature)
                    if check_md_completed(tensile_file, UNIFIED_CONDITIONS["max_steps"], UNIFIED_CONDITIONS["output_freq"]):
                        print(f"ステップ3: 引張シミュレーションは完了済み")
                        continue
                    
                    print(f"\n{'='*25}\nステップ3: 引張計算開始 (P={pressure:.3f}GPa, HT={high_temp}K)\n{'='*25}")
                    
                    ase_atoms, initial_reference, structure_info = prepare_structure_with_top_vacuum(
                        high_temp_file, 
                        UNIFIED_CONDITIONS["vacuum_top"],
                        UNIFIED_CONDITIONS["fixed_layer_thickness"]
                    )
                    
                    atoms = MatlantisAtoms.from_ase_atoms(ase_atoms)
                    system = ASEMDSystem(atoms)
                    system.init_temperature(UNIFIED_CONDITIONS["temperature"])
                    
                    strain_rate = UNIFIED_CONDITIONS["strain_rate"]
                    max_strain = UNIFIED_CONDITIONS["max_strain"]
                    timestep = UNIFIED_CONDITIONS["timestep"]
                    total_steps = min(int(max_strain / (strain_rate * timestep * 1e-3)), UNIFIED_CONDITIONS["max_steps"])
                    
                    if UNIFIED_CONDITIONS["vacuum_top"] > 0:
                        final_material_thickness = structure_info["material_thickness"] * (1 + max_strain)
                        final_cell_length = final_material_thickness + UNIFIED_CONDITIONS["vacuum_top"]
                    else:
                        final_cell_length = initial_reference * (1 + max_strain)
                    
                    latt = np.array(system.ase_atoms.cell)
                    latt[2, 2] = final_cell_length
                    
                    analysis_file = get_tensile_analysis_file_path(base_name, pressure, comp_temp, high_temp)
                    initial_material_thickness = structure_info["material_thickness"]
                    analyzer = ComprehensiveTensileAnalyzer(
                        initial_material_thickness, structure_info, UNIFIED_CONDITIONS, high_temp_file, analysis_file
                    )
                    
                    integrator = LangevinIntegrator(timestep=timestep, temperature=UNIFIED_CONDITIONS["temperature"])
                    deform = DeformScheduler(latt, total_steps)
                    md = MDFeature(integrator, n_run=total_steps, traj_file_name=tensile_file, traj_freq=UNIFIED_CONDITIONS["output_freq"], estimator_fn=estimator_fn)
                    
                    extensions = [(PrintCellShape(), 100), (deform, 1), (analyzer, 10)]
                    
                    execution_info = run_md_with_gap_detection(md, system, extensions)
                    final_results = analyzer.get_final_results()
                    
                    summary_file = get_tensile_summary_file_path(base_name, pressure, comp_temp, high_temp)
                    save_tensile_results_with_gap_detection(
                        final_results, summary_file, base_name, pressure, comp_temp, high_temp, execution_info
                    )
                    
                    print(f"✅ 引張計算完了: {base_name} (P={pressure:.3f}GPa, HT={high_temp}K)")

                except Exception as e:
                    print(f"❌ エラー発生: {base_name} (P={pressure:.3f}GPa, HT={high_temp}K): {e}")
                    import traceback
                    traceback.print_exc()
                    continue

print("\n全てのシミュレーションが完了しました。")


The MD trajectory will be saved at /home/jovyan/Kaori/MD/LiB_2/output/compression/Al2O3_10-12_(1, 0, 2)_NMC111_CotoNi_80_001_(0, 0, 1)_trimmed_final_P0.001_T300K_compression.traj.
Note: The max disk size of /home/jovyan is about 98G.


入力ディレクトリ: /home/jovyan/Kaori/MD/LiB_2/structure/output/final_interfaces
出力ディレクトリ: /home/jovyan/Kaori/MD/LiB_2/output
入力CSV: /home/jovyan/Kaori/MD/LiB_2/structure/extracted_data_final.csv
入力ディレクトリ: /home/jovyan/Kaori/MD/LiB_2/structure/output/final_interfaces
✅ CSVから 723 件の計算対象を読み込みました。

入力ファイルが見つかりません: /home/jovyan/Kaori/MD/LiB_2/structure/output/final_interfaces/Interface_Al_100_(1, 0, 0)_on_NMC111_Co_MntoNi_40_012_(0, 1, 2).xyz. スキップします...

入力ファイルが見つかりません: /home/jovyan/Kaori/MD/LiB_2/structure/output/final_interfaces/Interface_AlF3_on_NMC111_extreme_Ni_rich.xyz. スキップします...

==================== Al2O3_10-12_(1, 0, 2)_NMC111_CotoNi_80_001_(0, 0, 1)_trimmed_final のシミュレーションを開始 ====================

==================== Al2O3_10-12_(1, 0, 2)_NMC111_CotoNi_80_001_(0, 0, 1)_trimmed_final のシミュレーションを開始 ====================
ステップ0: 圧着実行 (P=0.001GPa, T=300K)
原子をZ方向に -4.57Å 移動しました


PropertyNotImplementedError: stress not present in this calculation

In [64]:
from ase.io import read, Trajectory, write
from ase.constraints import FixAtoms
import numpy as np
import os
import time
from datetime import datetime
from sklearn.linear_model import LinearRegression
from matlantis_features.features.md import ASEMDSystem, LangevinIntegrator, MDFeature, MDExtensionBase, NPTIntegrator
from matlantis_features.features.md.md_extensions import DeformScheduler
from pfp_api_client.pfp.estimator import Estimator, EstimatorCalcMode
from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
from matlantis_features.utils.calculators import get_calculator, pfp_estimator_fn
from matlantis_features.atoms import MatlantisAtoms
import pathlib
from ase import units
import pandas as pd # CSVファイルを扱うためにpandasをインポート

# =============================================
# エネルギー計算用の関数を定義
# =============================================
estimator_fn = pfp_estimator_fn(model_version='v7.0.0', calc_mode='crystal_u0_plus_d3')

# =============================================
# シミュレーションパラメータ設定
# =============================================
compression_temperatures = [300]
compression_pressures = [0.001, 0.005, 0.01]
high_temperatures = [300]
tensile_temperature = 300.0

# =============================================
# ファイル・ディレクトリパス設定
# =============================================
BASE_DIR = "/home/jovyan/Kaori/MD/LiB_2/"
INPUT_DIR = os.path.join(BASE_DIR, "structure/output/final_interfaces") 
OUTPUT_DIR = os.path.join(BASE_DIR, "output")
COMPRESSION_DIR = os.path.join(OUTPUT_DIR, "compression")
HEAT_DIR = os.path.join(OUTPUT_DIR, "heat")
STRESS_STRAIN_DIR = os.path.join(OUTPUT_DIR, "slow010/stress_strain")
TENSILE_DIR = os.path.join(OUTPUT_DIR, "slow010/tensile")

# ディレクトリ作成
os.makedirs(INPUT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(COMPRESSION_DIR, exist_ok=True)
os.makedirs(HEAT_DIR, exist_ok=True)
os.makedirs(STRESS_STRAIN_DIR, exist_ok=True)
os.makedirs(TENSILE_DIR, exist_ok=True)

# --- ファイルパス生成関数群 ---
def get_compression_file_path(base_name, pressure, comp_temp):
    return os.path.join(COMPRESSION_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_compression.traj")

def get_equil_file_path(base_name, pressure, comp_temp):
    return os.path.join(HEAT_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_equil.traj")

def get_high_temp_file_path(base_name, pressure, comp_temp, high_temp):
    return os.path.join(HEAT_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K.traj")

def get_tensile_file_path(base_name, pressure, comp_temp, high_temp, tensile_temperature):
    return os.path.join(TENSILE_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K_tensile{tensile_temperature:.0f}K.traj")

def get_tensile_analysis_file_path(base_name, pressure, comp_temp, high_temp):
    return os.path.join(TENSILE_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K_tensile_analysis.dat")

def get_tensile_summary_file_path(base_name, pressure, comp_temp, high_temp):
    return os.path.join(TENSILE_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K_tensile_summary.txt")

# =============================================
# 早期終了システム設定 & クラス定義
# =============================================
UNIFIED_CONDITIONS = {
    "strain_rate": 0.01, "vacuum_top": 0.0, "temperature": 300.0,
    "max_strain": 3.0, "timestep": 1.0, "elastic_region": (0.005, 0.03),
    "max_steps": 100000, "output_freq": 100, "fixed_layer_thickness": 15
}
EARLY_STOP_CONDITIONS = {
    "internal_gap_threshold": 10.0, "gap_search_margin": 0.1
}

class EarlyTerminationException(Exception):
    def __init__(self, reason, step, details=None):
        super().__init__(f"Early termination at step {step}: {reason}")
        self.reason = reason
        self.step = step
        self.details = details or {}

def find_largest_gap_z(atoms, search_margin_ratio=0.1):
    positions = atoms.get_positions()
    if len(positions) < 2: return atoms.get_cell()[2, 2] / 2.0, 0.0
    sorted_z = np.sort(positions[:, 2])
    start_idx = int(len(sorted_z) * search_margin_ratio)
    end_idx = int(len(sorted_z) * (1 - search_margin_ratio))
    if start_idx >= end_idx: return np.mean(sorted_z), 0.0
    search_z = sorted_z[start_idx:end_idx]
    gaps = search_z[1:] - search_z[:-1]
    max_gap_index = np.argmax(gaps)
    max_gap_value = gaps[max_gap_index]
    fracture_z = (search_z[max_gap_index] + search_z[max_gap_index + 1]) / 2.0
    return fracture_z, max_gap_value

class ComprehensiveTensileAnalyzer(MDExtensionBase):
    def __init__(self, initial_thickness, structure_info, conditions, source_file, output_file):
        self.initial_thickness = initial_thickness
        self.structure_info = structure_info
        self.conditions = conditions
        self.output_file = output_file
        self.data = []
        self.young_modulus = None
        self.young_r2 = 0.0
        self.max_stress = 0.0
        self.max_stress_strain = 0.0
        self.internal_gap_detected = False
        self.internal_gap_step = None
        self.max_internal_gap = 0.0
        self.early_termination_requested = False
        self.termination_reason = None
        self.termination_details = {}
        self.gap_check_frequency = 50
        with open(output_file, 'w') as f:
            f.write(f"# Tensile Analysis with Internal Gap Detection (Source: {source_file})\n")
            f.write(f"# Step Strain Stress(GPa) YoungMod(GPa) Max_Gap(A) Status Time(ps)\n")

    def detect_events(self, system):
        current_step = system.current_total_step
        if current_step % self.gap_check_frequency == 0:
            _, max_gap = find_largest_gap_z(system.ase_atoms, EARLY_STOP_CONDITIONS["gap_search_margin"])
            self.max_internal_gap = max(self.max_internal_gap, max_gap)
            if max_gap >= EARLY_STOP_CONDITIONS["internal_gap_threshold"]:
                if not self.internal_gap_detected:
                    self.internal_gap_detected = True
                    self.internal_gap_step = current_step
                    self.early_termination_requested = True
                    self.termination_reason = f"internal_gap_{max_gap:.1f}A"
                    raise EarlyTerminationException(f"Internal gap {max_gap:.1f}Å", current_step)

    def __call__(self, system, integrator):
        current_cell_length = system.ase_atoms.get_cell()[2, 2]
        material_strain = (current_cell_length - self.initial_thickness) / self.initial_thickness
        try:
            system.ase_atoms.get_potential_energy()
            stress_tensor = system.ase_atoms.get_stress(voigt=False)
            stress_zz_GPa = stress_tensor[2, 2] * 160.21766208
        except Exception:
            stress_zz_GPa = 0.0
            
        time_ps = system.current_total_step * self.conditions["timestep"] / 1000
        if stress_zz_GPa > self.max_stress:
            self.max_stress = stress_zz_GPa
            self.max_stress_strain = material_strain
        if len(self.data) > 50 and material_strain > 0.01 and self.young_modulus is None:
            strains = np.array([d[0] for d in self.data])
            stresses = np.array([d[1] for d in self.data])
            mask = (strains > self.conditions["elastic_region"][0]) & (strains < self.conditions["elastic_region"][1])
            if np.sum(mask) > 20:
                model = LinearRegression().fit(strains[mask].reshape(-1, 1), stresses[mask])
                self.young_modulus = model.coef_[0]
                self.young_r2 = model.score(strains[mask].reshape(-1, 1), stresses[mask])
        self.detect_events(system)
        status = "Plastic" if self.young_modulus is not None else "Loading"
        if self.internal_gap_detected: status = "Gap_Detected"
        self.data.append([material_strain, stress_zz_GPa])
        with open(self.output_file, 'a') as f:
            f.write(f"{system.current_total_step:<8} {material_strain:<14.6f} {stress_zz_GPa:<12.6f} "
                    f"{self.young_modulus or 0.0:<12.1f} {self.max_internal_gap:<10.1f} {status:<15} {time_ps:<10.3f}\n")

    def get_final_results(self):
        return {'young_modulus': self.young_modulus or 0.0, 'young_r2': self.young_r2,
                'max_stress': self.max_stress, 'max_stress_strain': self.max_stress_strain,
                'internal_gap_detected': self.internal_gap_detected, 'internal_gap_step': self.internal_gap_step,
                'max_internal_gap': self.max_internal_gap, 'early_termination': self.early_termination_requested,
                'termination_reason': self.termination_reason}

def run_md_with_gap_detection(md_feature, system, extensions):
    info = {'completed_normally': False, 'early_terminated': False}
    start = time.time()
    try:
        print(f"🚀 MD計算開始（隙間検出による早期終了機能付き）")
        md_feature(system, extensions=extensions)
        info.update({'completed_normally': True, 'total_steps_run': UNIFIED_CONDITIONS["max_steps"]})
    except EarlyTerminationException as e:
        info.update({'early_terminated': True, 'termination_reason': e.reason, 'termination_step': e.step, 'total_steps_run': e.step})
        print(f"🛑 MD計算早期終了: {e.reason} at step {e.step}")
    except Exception as e:
        info.update({'error_occurred': True, 'error_message': str(e)})
        print(f"❌ MD計算エラー終了: {e}")
    finally:
        info['execution_time'] = time.time() - start
    return info

def save_tensile_results(results, summary_file, base_name, p, ct, ht, exec_info):
    with open(summary_file, 'w') as f:
        f.write(f"引張計算結果 - {base_name}\n{'='*70}\n")
        f.write(f"完了時刻: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"条件: P={p:.3f}GPa, CT={ct}K, HT={ht}K\n")
        f.write(f"状態: {'早期終了' if exec_info.get('early_terminated') else '正常完了'}\n")
        if exec_info.get('early_terminated'): f.write(f"  理由: {exec_info.get('termination_reason')}\n")
        f.write(f"ヤング率: {results['young_modulus']:.1f} GPa (R² = {results['young_r2']:.3f})\n")
        f.write(f"最大応力: {results['max_stress']:.2f} GPa @ ひずみ {results['max_stress_strain']:.3f}\n")
        f.write(f"最大内部隙間: {results['max_internal_gap']:.1f} Å\n")

def check_md_completed(traj_path, steps, freq):
    if not os.path.exists(traj_path): return False
    try: return len(Trajectory(traj_path, 'r')) >= (steps // freq) * 0.9
    except: return False

class PrintCellShape(MDExtensionBase):
    def __call__(self, system, integrator):
        if system.current_total_step % 1000 == 0:
            print(f"  Dyn step {system.current_total_step:4d} Cell(c): {system.ase_atoms.cell.cellpar()[2]:.2f}")

def rotate_atoms_to_lower(ase_atoms):
    positions = ase_atoms.get_positions()
    z_min = np.min(positions[:, 2])
    ase_atoms.translate([0, 0, -z_min])
    return ase_atoms

def prepare_structure(high_temp_file):
    if not os.path.exists(high_temp_file): raise FileNotFoundError(f"File not found: {high_temp_file}")
    ase_atoms = read(high_temp_file, "-1")
    ase_atoms = rotate_atoms_to_lower(ase_atoms)
    thickness = np.max(ase_atoms.get_positions()[:, 2]) - np.min(ase_atoms.get_positions()[:, 2])
    cell = ase_atoms.get_cell()
    cell[2, 2] = thickness + UNIFIED_CONDITIONS["vacuum_top"]
    ase_atoms.set_cell(cell)
    ase_atoms.wrap()
    z_min, z_max = np.min(ase_atoms.get_positions()[:, 2]), np.max(ase_atoms.get_positions()[:, 2])
    fixed_thickness = (z_max - z_min) * UNIFIED_CONDITIONS["fixed_layer_thickness"] / 100.0
    indices = [a.index for a in ase_atoms if a.position[2] <= z_min + fixed_thickness or a.position[2] >= z_max - fixed_thickness]
    ase_atoms.set_constraint(FixAtoms(indices=indices))
    info = {"material_thickness": thickness}
    return ase_atoms, info

# =============================================
# メイン処理 (CSVファイル名指定版)
# =============================================
CSV_FILE_PATH = "/home/jovyan/Kaori/MD/LiB_2/structure/extracted_data_final_with_filenames.csv"

print(f"入力CSV: {CSV_FILE_PATH}\n入力ファイルディレクトリ: {INPUT_DIR}\n{'='*50}")

try:
    df = pd.read_csv(CSV_FILE_PATH)
    print(f"✅ CSVから {len(df)} 件の計算対象を読み込みました。")
except FileNotFoundError:
    print(f"❌ エラー: CSVファイルが見つかりません: {CSV_FILE_PATH}")
    exit()

for index, row in df.iterrows():
    original_filename = row['original_filename']
    com = row['com']
    atom = row['atom']
    base_name = f"{com}_{atom}"
    input_xyz_path = os.path.join(INPUT_DIR, original_filename)

    if not os.path.exists(input_xyz_path):
        print(f"\n入力ファイルが見つかりません: {input_xyz_path}. スキップします...")
        continue
        
    print(f"\n{'='*20} {base_name} ({original_filename}) のシミュレーションを開始 {'='*20}")
    
    for comp_temp in compression_temperatures:
        for pressure in compression_pressures:
            # --- ステップ0: 圧着 ---
            comp_file = get_compression_file_path(base_name, pressure, comp_temp)
            if not check_md_completed(comp_file, 8000, 100):
                print(f"ステップ0: 圧着実行 (P={pressure:.3f}GPa, T={comp_temp}K)")
                
                ase_atoms = read(input_xyz_path) 
                
                ase_atoms = rotate_atoms_to_lower(ase_atoms)
                ase_atoms.set_constraint(FixAtoms(indices=[a.index for a in ase_atoms if a.position[2] <= 3.0]))
                ase_atoms.pbc = (True, True, True)
                system = ASEMDSystem(MatlantisAtoms.from_ase_atoms(ase_atoms))
                system.init_temperature(comp_temp)
                
                # ★★★ 修正点 ★★★
                # MDFeatureを呼び出す前に、手動で計算機をシステムにアタッチします
                system.ase_atoms.calc = get_calculator(estimator_fn)
                
                integrator = NPTIntegrator(timestep=1.0, temperature=comp_temp, pressure=pressure*units.GPa, ttime=20*units.fs, pfactor=2e6*units.GPa*(units.fs**2), mask=np.array([[0,0,0],[0,0,0],[0,0,1]]))
                # 事前に計算機を明示的にアタッチ
                atoms.calc = get_calculator(estimator_fn)
                system = ASEMDSystem(atoms)
                # その後MDFeatureに estimator_fn を渡す
                md = MDFeature(integrator, n_run=8000, traj_file_name=comp_file, traj_freq=100, estimator_fn=estimator_fn)
                md(system)
            else: 
                print(f"ステップ0: 圧着は完了済み (P={pressure:.3f}GPa, T={comp_temp}K)")

            for high_temp in high_temperatures:
                try:
                    # --- ステップ1: 平衡化 ---
                    equil_file = get_equil_file_path(base_name, pressure, comp_temp)
                    if not check_md_completed(equil_file, 3000, 100):
                        print(f"ステップ1: 平衡化実行")
                        ase_atoms = read(comp_file, "-1")
                        ase_atoms = rotate_atoms_to_lower(ase_atoms)
                        ase_atoms.set_constraint(FixAtoms(indices=[a.index for a in ase_atoms if a.position[2] <= 3.0]))
                        system = ASEMDSystem(MatlantisAtoms.from_ase_atoms(ase_atoms))
                        system.init_temperature(300)
                        md = MDFeature(LangevinIntegrator(1.0, 300.0), n_run=3000, traj_file_name=equil_file, traj_freq=100, estimator_fn=estimator_fn)
                        md(system)
                    else: print(f"ステップ1: 平衡化は完了済み")

                    # --- ステップ2: 高温処理 ---
                    ht_file = get_high_temp_file_path(base_name, pressure, comp_temp, high_temp)
                    if not check_md_completed(ht_file, 10000, 100):
                        print(f"ステップ2: 高温処理実行 ({high_temp}K)")
                        ase_atoms = read(equil_file, "-1")
                        ase_atoms = rotate_atoms_to_lower(ase_atoms)
                        ase_atoms.set_constraint(FixAtoms(indices=[a.index for a in ase_atoms if a.position[2] <= 2.0]))
                        system = ASEMDSystem(MatlantisAtoms.from_ase_atoms(ase_atoms))
                        system.init_temperature(high_temp)
                        md = MDFeature(LangevinIntegrator(1.0, float(high_temp)), n_run=10000, traj_file_name=ht_file, traj_freq=100, estimator_fn=estimator_fn)
                        md(system)
                    else: print(f"ステップ2: 高温処理は完了済み")

                    # --- ステップ3: 引張シミュレーション ---
                    tensile_file = get_tensile_file_path(base_name, pressure, comp_temp, high_temp, tensile_temperature)
                    if check_md_completed(tensile_file, UNIFIED_CONDITIONS["max_steps"], 100):
                        print(f"ステップ3: 引張シミュレーションは完了済み")
                        continue
                    
                    print(f"\nステップ3: 引張計算開始 (P={pressure:.3f}GPa, HT={high_temp}K)")
                    ase_atoms, structure_info = prepare_structure(ht_file)
                    system = ASEMDSystem(MatlantisAtoms.from_ase_atoms(ase_atoms))
                    system.init_temperature(UNIFIED_CONDITIONS["temperature"])
                    
                    total_steps = int(UNIFIED_CONDITIONS["max_strain"] / (UNIFIED_CONDITIONS["strain_rate"] * UNIFIED_CONDITIONS["timestep"] * 1e-3))
                    final_cell_length = structure_info["material_thickness"] * (1 + UNIFIED_CONDITIONS["max_strain"])
                    latt = np.array(system.ase_atoms.cell); latt[2, 2] = final_cell_length
                    
                    analyzer = ComprehensiveTensileAnalyzer(structure_info["material_thickness"], {}, UNIFIED_CONDITIONS, ht_file, get_tensile_analysis_file_path(base_name, pressure, comp_temp, high_temp))
                    md = MDFeature(LangevinIntegrator(1.0, UNIFIED_CONDITIONS["temperature"]), n_run=total_steps, traj_file_name=tensile_file, traj_freq=100, estimator_fn=estimator_fn)
                    
                    execution_info = run_md_with_gap_detection(md, system, [(PrintCellShape(), 1000), (DeformScheduler(latt, total_steps), 1), (analyzer, 10)])
                    final_results = analyzer.get_final_results()
                    
                    save_tensile_results(final_results, get_tensile_summary_file_path(base_name, pressure, comp_temp, high_temp), base_name, pressure, comp_temp, high_temp, execution_info)
                    print(f"✅ 引張計算完了: {base_name} (P={pressure:.3f}GPa, HT={high_temp}K)")

                except Exception as e:
                    print(f"❌ エラー発生: {base_name} (P={pressure:.3f}GPa, HT={high_temp}K): {e}")
                    import traceback; traceback.print_exc()
                    continue

print("\n全てのシミュレーションが完了しました。")


The MD trajectory will be saved at /home/jovyan/Kaori/MD/LiB_2/output/compression/Al_100_(1, 0, 0)_NMC111_Co_MntoNi_40_012_(0, 1, 2)_P0.001_T300K_compression.traj.
Note: The max disk size of /home/jovyan is about 98G.


入力CSV: /home/jovyan/Kaori/MD/LiB_2/structure/extracted_data_final_with_filenames.csv
入力ファイルディレクトリ: /home/jovyan/Kaori/MD/LiB_2/structure/output/final_interfaces
✅ CSVから 723 件の計算対象を読み込みました。

==================== Al_100_(1, 0, 0)_NMC111_Co_MntoNi_40_012_(0, 1, 2) (Interface_Al_100_(1, 0, 0)_on_NMC111_Co_MntoNi_40_012_(0, 1, 2)_trimmed_initial.xyz) のシミュレーションを開始 ====================
ステップ0: 圧着実行 (P=0.001GPa, T=300K)


PropertyNotImplementedError: stress not present in this calculation

In [63]:
# 主要な修正箇所を抜粋

# 修正1: ComprehensiveTensileAnalyzer クラスの __call__ メソッド
class ComprehensiveTensileAnalyzer(MDExtensionBase):
    def __call__(self, system, integrator):
        current_cell_length = system.ase_atoms.get_cell()[2, 2]
        material_strain = (current_cell_length - self.initial_thickness) / self.initial_thickness
        
        # ★★★ 修正箇所: 応力計算の改善 ★★★
        try:
            # まずエネルギー計算を確実に実行して計算機を初期化
            system.ase_atoms.get_potential_energy()
            
            # 応力計算を実行（Matlantisの適切な前処理後）
            stress_tensor = system.ase_atoms.get_stress(voigt=False)
            stress_zz_GPa = stress_tensor[2, 2] * 160.21766208
            
        except (AttributeError, NotImplementedError, Exception) as e:
            # 応力計算が失敗した場合は0にする
            print(f"応力計算エラー（計算は継続）: {e}")
            stress_zz_GPa = 0.0
            
        # 以下は既存のコードと同じ...

# 修正2: メイン処理での原子系準備
for index, row in df.iterrows():
    # ... 既存のコード ...
    
    # ステップ0: 圧着
    if not check_md_completed(comp_file, 8000, 100):
        print(f"ステップ0: 圧着実行 (P={pressure:.3f}GPa, T={comp_temp}K)")
        
        ase_atoms = read(input_xyz_path)
        ase_atoms = rotate_atoms_to_lower(ase_atoms)
        ase_atoms.set_constraint(FixAtoms(indices=[a.index for a in ase_atoms if a.position[2] <= 3.0]))
        
        # ★★★ 修正箇所: Matlantisの適切な前処理 ★★★
        atoms = MatlantisAtoms.from_ase_atoms(ase_atoms)
        atoms.rotate_atoms_to_upper()  # この処理が重要
        
        # 計算機を明示的にアタッチ
        atoms.calc = get_calculator(estimator_fn)
        atoms.set_pbc=(True,True,True)
        
        system = ASEMDSystem(atoms)
        system.init_temperature(comp_temp)
        
        # 残りのMDFeature設定は同じ
        integrator = NPTIntegrator(
            timestep=1.0, 
            temperature=comp_temp, 
            pressure=pressure*units.GPa, 
            ttime=20*units.fs, 
            pfactor=2e6*units.GPa*(units.fs**2), 
            mask=np.array([[0,0,0],[0,0,0],[0,0,1]])
        )
        md = MDFeature(integrator, n_run=8000, traj_file_name=comp_file, traj_freq=100, estimator_fn=estimator_fn)
        md(system, extensions=[(PrintCellShape(), 100)])

# 修正3: ステップ1とステップ2でも同様の処理を追加
# ステップ1: 平衡化
if not check_md_completed(equil_file, 3000, 100):
    ase_atoms = read(comp_file, "-1")
    ase_atoms = rotate_atoms_to_lower(ase_atoms)
    ase_atoms.set_constraint(FixAtoms(indices=[a.index for a in ase_atoms if a.position[2] <= 3.0]))
    
    # ★★★ 修正箇所 ★★★
    atoms = MatlantisAtoms.from_ase_atoms(ase_atoms)
    atoms.rotate_atoms_to_upper()
    atoms.calc = get_calculator(estimator_fn)
    
    system = ASEMDSystem(atoms)
    system.init_temperature(300)
    md = MDFeature(LangevinIntegrator(1.0, 300.0), n_run=3000, traj_file_name=equil_file, traj_freq=100, estimator_fn=estimator_fn)
    md(system)

# ステップ2: 高温処理
if not check_md_completed(ht_file, 10000, 100):
    ase_atoms = read(equil_file, "-1")
    ase_atoms = rotate_atoms_to_lower(ase_atoms)
    ase_atoms.set_constraint(FixAtoms(indices=[a.index for a in ase_atoms if a.position[2] <= 2.0]))
    
    # ★★★ 修正箇所 ★★★
    atoms = MatlantisAtoms.from_ase_atoms(ase_atoms)
    atoms.rotate_atoms_to_upper()
    atoms.calc = get_calculator(estimator_fn)
    
    system = ASEMDSystem(atoms)
    system.init_temperature(high_temp)
    md = MDFeature(LangevinIntegrator(1.0, float(high_temp)), n_run=10000, traj_file_name=ht_file, traj_freq=100, estimator_fn=estimator_fn)
    md(system)

# ステップ3: 引張シミュレーション
print(f"\nステップ3: 引張計算開始 (P={pressure:.3f}GPa, HT={high_temp}K)")
ase_atoms, structure_info = prepare_structure(ht_file)

# ★★★ 修正箇所 ★★★
atoms = MatlantisAtoms.from_ase_atoms(ase_atoms)
atoms.rotate_atoms_to_upper()
atoms.calc = get_calculator(estimator_fn)

system = ASEMDSystem(atoms)
system.init_temperature(UNIFIED_CONDITIONS["temperature"])

ステップ0: 圧着実行 (P=0.001GPa, T=300K)


ValueError: The method is only applicable for the structure with PBC.

In [41]:
import re
import csv
from pathlib import Path

def parse_compound_block(block_string: str) -> tuple[str, str]:
    """
    化合物のブロック文字列を解析し、名前と表面情報を分離するヘルパー関数。
    例: "NMC111_extreme_Ni_rich_trimmed" -> ("NMC111_extreme_Ni_rich", "N/A")
    例: "Al_111" -> ("Al", "111")
    """
    # 除去する接尾辞のリスト (必要に応じて追加してください)
    SUFFIXES_TO_REMOVE = ['trimmed', 'initial', 'optimized', 'interface', 'final']
    
    parts = block_string.split('_')
    
    # 後方から接尾辞を削除
    cleaned_parts = []
    # partsを逆順にチェック
    for part in reversed(parts):
        if part in SUFFIXES_TO_REMOVE and not cleaned_parts:
            # 末尾にある接尾辞だけをスキップ
            continue
        cleaned_parts.append(part)
    
    # 順序を元に戻す
    cleaned_parts.reverse()
    
    if not cleaned_parts:
        return "unknown", "N/A"

    # 末尾の要素が3桁の数字なら、それを表面情報とする
    if len(cleaned_parts[-1]) == 3 and cleaned_parts[-1].isdigit():
        surface = cleaned_parts.pop() # 末尾をsurfaceとして取り出す
        name = "_".join(cleaned_parts)
        return name, surface
    else:
        # 表面情報がない場合
        name = "_".join(cleaned_parts)
        return name, "N/A" # Not Applicable


def extract_interface_data_flexible():
    """
    柔軟な命名規則に対応し、ファイル名から化合物と表面情報を抽出し、
    元のファイル名も一緒にCSVファイルに出力する。
    """
    # --- 設定項目 ---
    INTERFACES_DIR = Path("/home/jovyan/Kaori/MD/LiB_2/structure/output/final_interfaces")
    OUTPUT_CSV = "extracted_data_final_with_filenames.csv" # 出力ファイル名を変更
    FILE_EXTENSION = "*.xyz"
    # --- プログラム本体 ---

    print(f"📁 解析ディレクトリ: {INTERFACES_DIR}")
    if not INTERFACES_DIR.is_dir():
        print(f"❌ エラー: ディレクトリが見つかりません。")
        return

    extracted_data = []
    skipped_files = []

    file_paths = list(INTERFACES_DIR.glob(FILE_EXTENSION))
    print(f"🔍 {len(file_paths)}個のファイルを検出しました。解析を開始します...")

    for file_path in file_paths:
        stem = file_path.stem.replace("Interface_", "") # 先頭の"Interface_"を除去

        # "_on_" で前半と後半に分割
        if "_on_" not in stem:
            skipped_files.append(file_path.name)
            continue
            
        com_block, atom_block = stem.split("_on_", 1)
        
        # 各ブロックを解析
        com_name, com_surface = parse_compound_block(com_block)
        atom_name, atom_surface = parse_compound_block(atom_block)
        
        if com_name != "unknown" and atom_name != "unknown":
            # ★ 変更点1: 元のファイル名も辞書に追加
            extracted_data.append({
                "original_filename": file_path.name,
                "com": com_name,
                "com_surface": com_surface,
                "atom": atom_name,
                "atom_surface": atom_surface
            })
        else:
            skipped_files.append(file_path.name)
            
    # --- 結果の出力 ---
    if not extracted_data:
        print("⚠️ 警告: 解析可能なファイルが見つかりませんでした。")
        return

    try:
        with open(OUTPUT_CSV, 'w', newline='') as f:
            # ★ 変更点2: CSVのヘッダーに 'original_filename' を追加
            fieldnames = ['original_filename', 'com', 'com_surface', 'atom', 'atom_surface']
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(extracted_data)

        print("\n" + "="*60)
        print(f"✅ 成功: {len(extracted_data)}件のデータを'{OUTPUT_CSV}'に保存しました。")
        print("="*60)
        
        print("ファイル内容のプレビュー:")
        with open(OUTPUT_CSV, 'r') as f:
            for i, line in enumerate(f):
                if i > 5: break
                print(f"  {line.strip()}")
        if len(extracted_data) > 5: print("  ...")

    except IOError as e:
        print(f"❌ エラー: ファイル '{OUTPUT_CSV}' の書き込みに失敗しました。{e}")

    if skipped_files:
        print("\n⚠️ 以下のファイルは命名規則に一致しなかったためスキップされました:")
        for name in skipped_files: print(f"  - {name}")

if __name__ == "__main__":
    extract_interface_data_flexible()
# ```

### 実行後のCSVファイル例

# 新しいCSV (`extracted_data_final_with_filenames.csv`) は以下のようになります。

# ```csv
# original_filename,com,com_surface,atom,atom_surface
# Interface_Al_100_on_NMC111_012.xyz,Al,100,NMC111,012
# Interface_AlF3_on_NMC111_extreme_Ni_rich.xyz,AlF3,N/A,NMC111_extreme_Ni_rich,N/A
# ...
# ```

# このCSVファイルを使えば、次のMDシミュレーションプログラムで `original_filename` 列を読むだけで、確実に入力ファイルを指定でき

📁 解析ディレクトリ: /home/jovyan/Kaori/MD/LiB_2/structure/output/final_interfaces
🔍 723個のファイルを検出しました。解析を開始します...

✅ 成功: 723件のデータを'extracted_data_final_with_filenames.csv'に保存しました。
ファイル内容のプレビュー:
  original_filename,com,com_surface,atom,atom_surface
  "Interface_Al_100_(1, 0, 0)_on_NMC111_Co_MntoNi_40_012_(0, 1, 2)_trimmed_initial.xyz","Al_100_(1, 0, 0)",N/A,"NMC111_Co_MntoNi_40_012_(0, 1, 2)",N/A
  Interface_AlF3_on_NMC111_extreme_Ni_rich_trimmed_initial.xyz,AlF3,N/A,NMC111_extreme_Ni_rich,N/A
  "Interface_Al2O3_10-12_(1, 0, 2)_on_NMC111_CotoNi_80_001_(0, 0, 1)_trimmed_final.xyz","Al2O3_10-12_(1, 0, 2)",N/A,"NMC111_CotoNi_80_001_(0, 0, 1)",N/A
  "Interface_Al2O3_on_NMC111_CotoNi_80_100_(1, 0, 0)_trimmed_initial.xyz",Al2O3,N/A,"NMC111_CotoNi_80_100_(1, 0, 0)",N/A
  "Interface_Al_100_(1, 0, 0)_on_NMC111_thermal_Co_Mn_loss_110_(1, 1, 0)_trimmed_initial.xyz","Al_100_(1, 0, 0)",N/A,"NMC111_thermal_Co_Mn_loss_110_(1, 1, 0)",N/A
  ...


In [43]:
from ase.io import read, Trajectory, write
from ase.constraints import FixAtoms
import numpy as np
import os
import time
from datetime import datetime
from sklearn.linear_model import LinearRegression
from matlantis_features.features.md import ASEMDSystem, LangevinIntegrator, MDFeature, MDExtensionBase, NPTIntegrator
from matlantis_features.features.md.md_extensions import DeformScheduler
from pfp_api_client.pfp.estimator import Estimator, EstimatorCalcMode
from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
from matlantis_features.utils.calculators import get_calculator, pfp_estimator_fn
from matlantis_features.atoms import MatlantisAtoms
import pathlib
from ase import units
import pandas as pd # CSVファイルを扱うためにpandasをインポート

# =============================================
# エネルギー計算用の関数を定義
# =============================================
estimator_fn = pfp_estimator_fn(model_version='v7.0.0', calc_mode='crystal_u0_plus_d3')

# =============================================
# シミュレーションパラメータ設定
# =============================================
compression_temperatures = [300]
compression_pressures = [0.001, 0.005, 0.01]
high_temperatures = [300]
tensile_temperature = 300.0

# =============================================
# ファイル・ディレクトリパス設定
# =============================================
BASE_DIR = "/home/jovyan/Kaori/MD/LiB_2/"
# ★★★ 注意: このディレクトリはCSV内のファイル名が実際に存在する場所を指定してください ★★★
INPUT_DIR = os.path.join(BASE_DIR, "structure/output/final_interfaces") 
OUTPUT_DIR = os.path.join(BASE_DIR, "output")
COMPRESSION_DIR = os.path.join(OUTPUT_DIR, "compression")
HEAT_DIR = os.path.join(OUTPUT_DIR, "heat")
STRESS_STRAIN_DIR = os.path.join(OUTPUT_DIR, "slow010/stress_strain")
TENSILE_DIR = os.path.join(OUTPUT_DIR, "slow010/tensile")

# ディレクトリ作成
os.makedirs(INPUT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(COMPRESSION_DIR, exist_ok=True)
os.makedirs(HEAT_DIR, exist_ok=True)
os.makedirs(STRESS_STRAIN_DIR, exist_ok=True)
os.makedirs(TENSILE_DIR, exist_ok=True)

# --- ファイルパス生成関数群 (出力ファイル用) ---
def get_compression_file_path(base_name, pressure, comp_temp):
    return os.path.join(COMPRESSION_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_compression.traj")

def get_equil_file_path(base_name, pressure, comp_temp):
    return os.path.join(HEAT_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_equil.traj")

def get_high_temp_file_path(base_name, pressure, comp_temp, high_temp):
    return os.path.join(HEAT_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K.traj")

def get_tensile_file_path(base_name, pressure, comp_temp, high_temp, tensile_temperature):
    return os.path.join(TENSILE_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K_tensile{tensile_temperature:.0f}K.traj")

def get_tensile_analysis_file_path(base_name, pressure, comp_temp, high_temp):
    return os.path.join(TENSILE_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K_tensile_analysis.dat")

def get_tensile_summary_file_path(base_name, pressure, comp_temp, high_temp):
    return os.path.join(TENSILE_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K_tensile_summary.txt")

# =============================================
# 早期終了システム設定 & クラス定義 (変更なし)
# =============================================
# (このセクションのコードは変更なしのため省略します。前回のコードをそのままお使いください)
# ... ComprehensiveTensileAnalyzer, run_md_with_gap_detection, etc. ...

# =============================================
# ユーティリティ関数 (変更なし)
# =============================================
# (このセクションのコードは変更なしのため省略します。前回のコードをそのままお使いください)
# ... check_md_completed, rotate_atoms_to_lower, etc. ...

# =============================================
# ★★★ メイン処理 (CSVファイル名指定版) ★★★
# =============================================
CSV_FILE_PATH = "/home/jovyan/Kaori/MD/LiB_2/structure/extracted_data_final_with_filenames.csv"

print(f"入力CSV: {CSV_FILE_PATH}\n入力ファイルディレクトリ: {INPUT_DIR}\n{'='*50}")

try:
    df = pd.read_csv(CSV_FILE_PATH)
    print(f"✅ CSVから {len(df)} 件の計算対象を読み込みました。")
except FileNotFoundError:
    print(f"❌ エラー: CSVファイルが見つかりません: {CSV_FILE_PATH}")
    exit()

for index, row in df.iterrows():
    # --- CSVから情報を取得 ---
    original_filename = row['original_filename']
    com = row['com']
    atom = row['atom']
    
    # --- パスと名前を生成 ---
    base_name = f"{com}_{atom}" # 出力ファイル用のベース名
    input_xyz_path = os.path.join(INPUT_DIR, original_filename) # ★ 入力ファイルのフルパス

    # --- シミュレーション開始 ---
    if not os.path.exists(input_xyz_path):
        print(f"\n入力ファイルが見つかりません: {input_xyz_path}. スキップします...")
        continue
        
    print(f"\n{'='*20} {base_name} ({original_filename}) のシミュレーションを開始 {'='*20}")
    
    for comp_temp in compression_temperatures:
        for pressure in compression_pressures:
            # --- ステップ0: 圧着 ---
            comp_file = get_compression_file_path(base_name, pressure, comp_temp)
            if not check_md_completed(comp_file, 8000, 100):
                print(f"ステップ0: 圧着実行 (P={pressure:.3f}GPa, T={comp_temp}K)")
                
                # ★★★ 元のファイル名を指定して読み込む ★★★
                ase_atoms = read(input_xyz_path) 
                
                ase_atoms = rotate_atoms_to_lower(ase_atoms)
                ase_atoms.set_constraint(FixAtoms(indices=[a.index for a in ase_atoms if a.position[2] <= 3.0]))
                system = ASEMDSystem(MatlantisAtoms.from_ase_atoms(ase_atoms))
                system.init_temperature(comp_temp)
                # system.get_potential_energy() # NPTの前にstressを計算させる
                
                integrator = NPTIntegrator(timestep=1.0, temperature=comp_temp, pressure=pressure*units.GPa, ttime=20*units.fs, pfactor=2e6*units.GPa*(units.fs**2), mask=np.array([[0,0,0],[0,0,0],[0,0,1]]))
                md = MDFeature(integrator, n_run=8000, traj_file_name=comp_file, traj_freq=100, estimator_fn=estimator_fn)
                md(system, extensions=[(PrintCellShape(), 100)])
            else: 
                print(f"ステップ0: 圧着は完了済み (P={pressure:.3f}GPa, T={comp_temp}K)")

            # ----------------------------------------------------------------
            # ▼▼▼ ステップ1以降の処理 (変更なし) ▼▼▼
            # ----------------------------------------------------------------
            for high_temp in high_temperatures:
                try:
                    # (平衡化、高温処理、引張シミュレーションのコードはここに続く)
                    # ...
                    pass # この行は実際のコードに置き換えてください
                except Exception as e:
                    print(f"❌ エラー発生: {base_name} (P={pressure:.3f}GPa, HT={high_temp}K): {e}")
                    import traceback; traceback.print_exc()
                    continue

print("\n全てのシミュレーションが完了しました。")



The MD trajectory will be saved at /home/jovyan/Kaori/MD/LiB_2/output/compression/Al_100_(1, 0, 0)_NMC111_Co_MntoNi_40_012_(0, 1, 2)_P0.001_T300K_compression.traj.
Note: The max disk size of /home/jovyan is about 98G.


入力CSV: /home/jovyan/Kaori/MD/LiB_2/structure/extracted_data_final_with_filenames.csv
入力ファイルディレクトリ: /home/jovyan/Kaori/MD/LiB_2/structure/output/final_interfaces
✅ CSVから 723 件の計算対象を読み込みました。

==================== Al_100_(1, 0, 0)_NMC111_Co_MntoNi_40_012_(0, 1, 2) (Interface_Al_100_(1, 0, 0)_on_NMC111_Co_MntoNi_40_012_(0, 1, 2)_trimmed_initial.xyz) のシミュレーションを開始 ====================
ステップ0: 圧着実行 (P=0.001GPa, T=300K)
  Dyn step    0 Cell(c): 53.00


PropertyNotImplementedError: stress not present in this calculation

In [60]:
from ase.io import read, Trajectory, write
from ase.constraints import FixAtoms
import numpy as np
import os
import time
from datetime import datetime
from sklearn.linear_model import LinearRegression
from matlantis_features.features.md import ASEMDSystem, LangevinIntegrator, MDFeature, MDExtensionBase, NPTIntegrator
from matlantis_features.features.md.md_extensions import DeformScheduler
from pfp_api_client.pfp.estimator import Estimator, EstimatorCalcMode
from pfp_api_client.pfp.calculators.ase_calculator import ASECalculator
from matlantis_features.utils.calculators import get_calculator, pfp_estimator_fn
from matlantis_features.atoms import MatlantisAtoms
import pathlib
from ase import units
import pandas as pd

# =============================================
# エネルギー計算用の関数を定義
# =============================================
estimator_fn = pfp_estimator_fn(model_version='v7.0.0', calc_mode='crystal_u0_plus_d3')

# =============================================
# シミュレーションパラメータ設定
# =============================================
compression_temperatures = [300]
compression_pressures = [0.001, 0.005, 0.01]
high_temperatures = [300]
tensile_temperature = 300.0

# =============================================
# ファイル・ディレクトリパス設定
# =============================================
BASE_DIR = "/home/jovyan/Kaori/MD/LiB_2/"
INPUT_DIR = os.path.join(BASE_DIR, "structure/output/final_interfaces") 
OUTPUT_DIR = os.path.join(BASE_DIR, "output")
COMPRESSION_DIR = os.path.join(OUTPUT_DIR, "compression")
HEAT_DIR = os.path.join(OUTPUT_DIR, "heat")
STRESS_STRAIN_DIR = os.path.join(OUTPUT_DIR, "slow010/stress_strain")
TENSILE_DIR = os.path.join(OUTPUT_DIR, "slow010/tensile")

# ディレクトリ作成
os.makedirs(INPUT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(COMPRESSION_DIR, exist_ok=True)
os.makedirs(HEAT_DIR, exist_ok=True)
os.makedirs(STRESS_STRAIN_DIR, exist_ok=True)
os.makedirs(TENSILE_DIR, exist_ok=True)

# --- ファイルパス生成関数群 ---
def get_compression_file_path(base_name, pressure, comp_temp):
    return os.path.join(COMPRESSION_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_compression.traj")

def get_equil_file_path(base_name, pressure, comp_temp):
    return os.path.join(HEAT_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_equil.traj")

def get_high_temp_file_path(base_name, pressure, comp_temp, high_temp):
    return os.path.join(HEAT_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K.traj")

def get_tensile_file_path(base_name, pressure, comp_temp, high_temp, tensile_temperature):
    return os.path.join(TENSILE_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K_tensile{tensile_temperature:.0f}K.traj")

def get_tensile_analysis_file_path(base_name, pressure, comp_temp, high_temp):
    return os.path.join(TENSILE_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K_tensile_analysis.dat")

def get_tensile_summary_file_path(base_name, pressure, comp_temp, high_temp):
    return os.path.join(TENSILE_DIR, f"{base_name}_P{pressure:.3f}_T{comp_temp:.0f}K_HT{high_temp}K_tensile_summary.txt")

# =============================================
# 早期終了システム設定 & クラス定義
# =============================================
UNIFIED_CONDITIONS = {
    "strain_rate": 0.01, "vacuum_top": 0.0, "temperature": 300.0,
    "max_strain": 3.0, "timestep": 1.0, "elastic_region": (0.005, 0.03),
    "max_steps": 100000, "output_freq": 100, "fixed_layer_thickness": 15
}
EARLY_STOP_CONDITIONS = {
    "internal_gap_threshold": 10.0, "gap_search_margin": 0.1
}

class EarlyTerminationException(Exception):
    def __init__(self, reason, step, details=None):
        super().__init__(f"Early termination at step {step}: {reason}")
        self.reason = reason
        self.step = step
        self.details = details or {}

def find_largest_gap_z(atoms, search_margin_ratio=0.1):
    positions = atoms.get_positions()
    if len(positions) < 2: return atoms.get_cell()[2, 2] / 2.0, 0.0
    sorted_z = np.sort(positions[:, 2])
    start_idx = int(len(sorted_z) * search_margin_ratio)
    end_idx = int(len(sorted_z) * (1 - search_margin_ratio))
    if start_idx >= end_idx: return np.mean(sorted_z), 0.0
    search_z = sorted_z[start_idx:end_idx]
    gaps = search_z[1:] - search_z[:-1]
    max_gap_index = np.argmax(gaps)
    max_gap_value = gaps[max_gap_index]
    fracture_z = (search_z[max_gap_index] + search_z[max_gap_index + 1]) / 2.0
    return fracture_z, max_gap_value

def calculate_stress_from_forces(atoms, cross_sectional_area):
    """
    力からz方向の応力を近似計算する関数
    """
    try:
        forces = atoms.get_forces()
        # z方向の力の合計を計算
        total_force_z = np.sum(forces[:, 2])
        # 応力 = 力 / 断面積 (単位変換: eV/Å → GPa)
        stress_z = abs(total_force_z) / cross_sectional_area * 0.1602176634  # eV/Å² → GPa
        return stress_z
    except Exception as e:
        print(f"力による応力計算でエラー: {e}")
        return 0.0

class ComprehensiveTensileAnalyzer(MDExtensionBase):
    def __init__(self, initial_thickness, structure_info, conditions, source_file, output_file):
        self.initial_thickness = initial_thickness
        self.structure_info = structure_info
        self.conditions = conditions
        self.output_file = output_file
        self.data = []
        self.young_modulus = None
        self.young_r2 = 0.0
        self.max_stress = 0.0
        self.max_stress_strain = 0.0
        self.internal_gap_detected = False
        self.internal_gap_step = None
        self.max_internal_gap = 0.0
        self.early_termination_requested = False
        self.termination_reason = None
        self.termination_details = {}
        self.gap_check_frequency = 50
        
        # 断面積の計算（xy平面）
        cell = structure_info.get('cell_params')
        if cell is not None:
            self.cross_sectional_area = cell[0] * cell[1]  # x * y
        else:
            self.cross_sectional_area = 100.0  # デフォルト値
            
        with open(output_file, 'w') as f:
            f.write(f"# Tensile Analysis with Force-based Stress (Source: {source_file})\n")
            f.write(f"# Step Strain Stress(GPa) YoungMod(GPa) Max_Gap(A) Status Time(ps)\n")

    def detect_events(self, system):
        current_step = system.current_total_step
        if current_step % self.gap_check_frequency == 0:
            _, max_gap = find_largest_gap_z(system.ase_atoms, EARLY_STOP_CONDITIONS["gap_search_margin"])
            self.max_internal_gap = max(self.max_internal_gap, max_gap)
            if max_gap >= EARLY_STOP_CONDITIONS["internal_gap_threshold"]:
                if not self.internal_gap_detected:
                    self.internal_gap_detected = True
                    self.internal_gap_step = current_step
                    self.early_termination_requested = True
                    self.termination_reason = f"internal_gap_{max_gap:.1f}A"
                    raise EarlyTerminationException(f"Internal gap {max_gap:.1f}Å", current_step)

    def __call__(self, system, integrator):
        current_cell_length = system.ase_atoms.get_cell()[2, 2]
        material_strain = (current_cell_length - self.initial_thickness) / self.initial_thickness
        
        # ★★★ 修正点: 応力計算の変更 ★★★
        try:
            # まずエネルギー計算を確実に実行
            system.ase_atoms.get_potential_energy()
            
            # 応力テンソル計算を試行
            try:
                stress_tensor = system.ase_atoms.get_stress(voigt=False)
                stress_zz_GPa = stress_tensor[2, 2] * 160.21766208
                stress_calculation_method = "tensor"
            except (AttributeError, NotImplementedError, Exception) as stress_error:
                # 応力テンソルが利用できない場合は力ベースの計算に切り替え
                print(f"応力テンソル計算不可、力ベース計算に切り替え: {stress_error}")
                stress_zz_GPa = calculate_stress_from_forces(system.ase_atoms, self.cross_sectional_area)
                stress_calculation_method = "force"
                
        except Exception as e:
            print(f"エネルギー計算エラー: {e}")
            stress_zz_GPa = 0.0
            stress_calculation_method = "error"
            
        time_ps = system.current_total_step * self.conditions["timestep"] / 1000
        if stress_zz_GPa > self.max_stress:
            self.max_stress = stress_zz_GPa
            self.max_stress_strain = material_strain
            
        if len(self.data) > 50 and material_strain > 0.01 and self.young_modulus is None:
            strains = np.array([d[0] for d in self.data])
            stresses = np.array([d[1] for d in self.data])
            mask = (strains > self.conditions["elastic_region"][0]) & (strains < self.conditions["elastic_region"][1])
            if np.sum(mask) > 20:
                model = LinearRegression().fit(strains[mask].reshape(-1, 1), stresses[mask])
                self.young_modulus = model.coef_[0]
                self.young_r2 = model.score(strains[mask].reshape(-1, 1), stresses[mask])
                
        self.detect_events(system)
        status = "Plastic" if self.young_modulus is not None else "Loading"
        if self.internal_gap_detected: status = "Gap_Detected"
        
        self.data.append([material_strain, stress_zz_GPa])
        with open(self.output_file, 'a') as f:
            f.write(f"{system.current_total_step:<8} {material_strain:<14.6f} {stress_zz_GPa:<12.6f} "
                    f"{self.young_modulus or 0.0:<12.1f} {self.max_internal_gap:<10.1f} {status:<15} {time_ps:<10.3f}\n")

    def get_final_results(self):
        return {'young_modulus': self.young_modulus or 0.0, 'young_r2': self.young_r2,
                'max_stress': self.max_stress, 'max_stress_strain': self.max_stress_strain,
                'internal_gap_detected': self.internal_gap_detected, 'internal_gap_step': self.internal_gap_step,
                'max_internal_gap': self.max_internal_gap, 'early_termination': self.early_termination_requested,
                'termination_reason': self.termination_reason}

def run_md_with_gap_detection(md_feature, system, extensions):
    info = {'completed_normally': False, 'early_terminated': False}
    start = time.time()
    try:
        print(f"🚀 MD計算開始（隙間検出による早期終了機能付き）")
        md_feature(system, extensions=extensions)
        info.update({'completed_normally': True, 'total_steps_run': UNIFIED_CONDITIONS["max_steps"]})
    except EarlyTerminationException as e:
        info.update({'early_terminated': True, 'termination_reason': e.reason, 'termination_step': e.step, 'total_steps_run': e.step})
        print(f"🛑 MD計算早期終了: {e.reason} at step {e.step}")
    except Exception as e:
        info.update({'error_occurred': True, 'error_message': str(e)})
        print(f"❌ MD計算エラー終了: {e}")
    finally:
        info['execution_time'] = time.time() - start
    return info

def save_tensile_results(results, summary_file, base_name, p, ct, ht, exec_info):
    with open(summary_file, 'w') as f:
        f.write(f"引張計算結果 - {base_name}\n{'='*70}\n")
        f.write(f"完了時刻: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"条件: P={p:.3f}GPa, CT={ct}K, HT={ht}K\n")
        f.write(f"状態: {'早期終了' if exec_info.get('early_terminated') else '正常完了'}\n")
        if exec_info.get('early_terminated'): f.write(f"  理由: {exec_info.get('termination_reason')}\n")
        f.write(f"ヤング率: {results['young_modulus']:.1f} GPa (R² = {results['young_r2']:.3f})\n")
        f.write(f"最大応力: {results['max_stress']:.2f} GPa @ ひずみ {results['max_stress_strain']:.3f}\n")
        f.write(f"最大内部隙間: {results['max_internal_gap']:.1f} Å\n")

def check_md_completed(traj_path, steps, freq):
    if not os.path.exists(traj_path): return False
    try: return len(Trajectory(traj_path, 'r')) >= (steps // freq) * 0.9
    except: return False

class PrintCellShape(MDExtensionBase):
    def __call__(self, system, integrator):
        if system.current_total_step % 1000 == 0:
            print(f"  Dyn step {system.current_total_step:4d} Cell(c): {system.ase_atoms.cell.cellpar()[2]:.2f}")

def rotate_atoms_to_lower(ase_atoms):
    positions = ase_atoms.get_positions()
    z_min = np.min(positions[:, 2])
    ase_atoms.translate([0, 0, -z_min])
    return ase_atoms

def prepare_structure(high_temp_file):
    if not os.path.exists(high_temp_file): raise FileNotFoundError(f"File not found: {high_temp_file}")
    ase_atoms = read(high_temp_file, "-1")
    ase_atoms = rotate_atoms_to_lower(ase_atoms)
    thickness = np.max(ase_atoms.get_positions()[:, 2]) - np.min(ase_atoms.get_positions()[:, 2])
    cell = ase_atoms.get_cell()
    cell[2, 2] = thickness + UNIFIED_CONDITIONS["vacuum_top"]
    ase_atoms.set_cell(cell)
    ase_atoms.wrap()
    z_min, z_max = np.min(ase_atoms.get_positions()[:, 2]), np.max(ase_atoms.get_positions()[:, 2])
    fixed_thickness = (z_max - z_min) * UNIFIED_CONDITIONS["fixed_layer_thickness"] / 100.0
    indices = [a.index for a in ase_atoms if a.position[2] <= z_min + fixed_thickness or a.position[2] >= z_max - fixed_thickness]
    ase_atoms.set_constraint(FixAtoms(indices=indices))
    
    # セル情報を追加
    info = {
        "material_thickness": thickness,
        "cell_params": ase_atoms.get_cell().cellpar()
    }
    return ase_atoms, info

# =============================================
# メイン処理 (CSVファイル名指定版)
# =============================================
CSV_FILE_PATH = "/home/jovyan/Kaori/MD/LiB_2/structure/extracted_data_final_with_filenames.csv"

print(f"入力CSV: {CSV_FILE_PATH}\n入力ファイルディレクトリ: {INPUT_DIR}\n{'='*50}")

try:
    df = pd.read_csv(CSV_FILE_PATH)
    print(f"✅ CSVから {len(df)} 件の計算対象を読み込みました。")
except FileNotFoundError:
    print(f"❌ エラー: CSVファイルが見つかりません: {CSV_FILE_PATH}")
    exit()

for index, row in df.iterrows():
    original_filename = row['original_filename']
    com = row['com']
    atom = row['atom']
    base_name = f"{com}_{atom}"
    input_xyz_path = os.path.join(INPUT_DIR, original_filename)

    if not os.path.exists(input_xyz_path):
        print(f"\n入力ファイルが見つかりません: {input_xyz_path}. スキップします...")
        continue
        
    print(f"\n{'='*20} {base_name} ({original_filename}) のシミュレーションを開始 {'='*20}")
    
    for comp_temp in compression_temperatures:
        for pressure in compression_pressures:
            # --- ステップ0: 圧着 ---
            comp_file = get_compression_file_path(base_name, pressure, comp_temp)
            if not check_md_completed(comp_file, 8000, 100):
                print(f"ステップ0: 圧着実行 (P={pressure:.3f}GPa, T={comp_temp}K)")
                
                ase_atoms = read(input_xyz_path) 
                ase_atoms = rotate_atoms_to_lower(ase_atoms)
                ase_atoms.set_constraint(FixAtoms(indices=[a.index for a in ase_atoms if a.position[2] <= 3.0]))
                
                system = ASEMDSystem(MatlantisAtoms.from_ase_atoms(ase_atoms))
                system.init_temperature(comp_temp)
                
                integrator = NPTIntegrator(timestep=1.0, temperature=comp_temp, pressure=pressure*units.GPa, ttime=20*units.fs, pfactor=2e6*units.GPa*(units.fs**2), mask=np.array([[0,0,0],[0,0,0],[0,0,1]]))
                md = MDFeature(integrator, n_run=8000, traj_file_name=comp_file, traj_freq=100, estimator_fn=estimator_fn)
                md(system, extensions=[(PrintCellShape(), 100)])
            else: 
                print(f"ステップ0: 圧着は完了済み (P={pressure:.3f}GPa, T={comp_temp}K)")

            for high_temp in high_temperatures:
                try:
                    # --- ステップ1: 平衡化 ---
                    equil_file = get_equil_file_path(base_name, pressure, comp_temp)
                    if not check_md_completed(equil_file, 3000, 100):
                        print(f"ステップ1: 平衡化実行")
                        ase_atoms = read(comp_file, "-1")
                        ase_atoms = rotate_atoms_to_lower(ase_atoms)
                        ase_atoms.set_constraint(FixAtoms(indices=[a.index for a in ase_atoms if a.position[2] <= 3.0]))
                        system = ASEMDSystem(MatlantisAtoms.from_ase_atoms(ase_atoms))
                        system.init_temperature(300)
                        md = MDFeature(LangevinIntegrator(1.0, 300.0), n_run=3000, traj_file_name=equil_file, traj_freq=100, estimator_fn=estimator_fn)
                        md(system)
                    else: print(f"ステップ1: 平衡化は完了済み")

                    # --- ステップ2: 高温処理 ---
                    ht_file = get_high_temp_file_path(base_name, pressure, comp_temp, high_temp)
                    if not check_md_completed(ht_file, 10000, 100):
                        print(f"ステップ2: 高温処理実行 ({high_temp}K)")
                        ase_atoms = read(equil_file, "-1")
                        ase_atoms = rotate_atoms_to_lower(ase_atoms)
                        ase_atoms.set_constraint(FixAtoms(indices=[a.index for a in ase_atoms if a.position[2] <= 2.0]))
                        system = ASEMDSystem(MatlantisAtoms.from_ase_atoms(ase_atoms))
                        system.init_temperature(high_temp)
                        md = MDFeature(LangevinIntegrator(1.0, float(high_temp)), n_run=10000, traj_file_name=ht_file, traj_freq=100, estimator_fn=estimator_fn)
                        md(system)
                    else: print(f"ステップ2: 高温処理は完了済み")

                    # --- ステップ3: 引張シミュレーション ---
                    tensile_file = get_tensile_file_path(base_name, pressure, comp_temp, high_temp, tensile_temperature)
                    if check_md_completed(tensile_file, UNIFIED_CONDITIONS["max_steps"], 100):
                        print(f"ステップ3: 引張シミュレーションは完了済み")
                        continue
                    
                    print(f"\nステップ3: 引張計算開始 (P={pressure:.3f}GPa, HT={high_temp}K)")
                    ase_atoms, structure_info = prepare_structure(ht_file)
                    system = ASEMDSystem(MatlantisAtoms.from_ase_atoms(ase_atoms))
                    system.init_temperature(UNIFIED_CONDITIONS["temperature"])
                    
                    total_steps = int(UNIFIED_CONDITIONS["max_strain"] / (UNIFIED_CONDITIONS["strain_rate"] * UNIFIED_CONDITIONS["timestep"] * 1e-3))
                    final_cell_length = structure_info["material_thickness"] * (1 + UNIFIED_CONDITIONS["max_strain"])
                    latt = np.array(system.ase_atoms.cell); latt[2, 2] = final_cell_length
                    
                    analyzer = ComprehensiveTensileAnalyzer(structure_info["material_thickness"], structure_info, UNIFIED_CONDITIONS, ht_file, get_tensile_analysis_file_path(base_name, pressure, comp_temp, high_temp))
                    md = MDFeature(LangevinIntegrator(1.0, UNIFIED_CONDITIONS["temperature"]), n_run=total_steps, traj_file_name=tensile_file, traj_freq=100, estimator_fn=estimator_fn)
                    
                    execution_info = run_md_with_gap_detection(md, system, [(PrintCellShape(), 1000), (DeformScheduler(latt, total_steps), 1), (analyzer, 10)])
                    final_results = analyzer.get_final_results()
                    
                    save_tensile_results(final_results, get_tensile_summary_file_path(base_name, pressure, comp_temp, high_temp), base_name, pressure, comp_temp, high_temp, execution_info)
                    print(f"✅ 引張計算完了: {base_name} (P={pressure:.3f}GPa, HT={high_temp}K)")

                except Exception as e:
                    print(f"❌ エラー発生: {base_name} (P={pressure:.3f}GPa, HT={high_temp}K): {e}")
                    import traceback; traceback.print_exc()
                    continue

print("\n全てのシミュレーションが完了しました。")

The MD trajectory will be saved at /home/jovyan/Kaori/MD/LiB_2/output/compression/Al_100_(1, 0, 0)_NMC111_Co_MntoNi_40_012_(0, 1, 2)_P0.001_T300K_compression.traj.
Note: The max disk size of /home/jovyan is about 98G.


入力CSV: /home/jovyan/Kaori/MD/LiB_2/structure/extracted_data_final_with_filenames.csv
入力ファイルディレクトリ: /home/jovyan/Kaori/MD/LiB_2/structure/output/final_interfaces
✅ CSVから 723 件の計算対象を読み込みました。

==================== Al_100_(1, 0, 0)_NMC111_Co_MntoNi_40_012_(0, 1, 2) (Interface_Al_100_(1, 0, 0)_on_NMC111_Co_MntoNi_40_012_(0, 1, 2)_trimmed_initial.xyz) のシミュレーションを開始 ====================
ステップ0: 圧着実行 (P=0.001GPa, T=300K)
  Dyn step    0 Cell(c): 53.00


PropertyNotImplementedError: stress not present in this calculation